In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# RDKit を Colab にインストール
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 73.4 MB/s eta 0:00:00


In [ ]:
%%bash
set -euo pipefail
echo "⏳  Installing PyTorch-Geometric + RDKit (Py3.12 friendly)…"

# 最新 pip
python -m pip install -q --upgrade pip

# ★ ここを「バイナリ拡張なし」に簡略化する
#   - torch_geometric は PyTorch さえ入っていれば単体で動作可能
python -m pip install -q torch_geometric rdkit captum matplotlib pandas

# 動作確認
python - <<'PY'
import torch, torch_geometric, captum
from rdkit import Chem, RDLogger; RDLogger.DisableLog('rdApp.*')

print("✅ torch         :", torch.__version__)
print("✅ torch_geometric:", torch_geometric.__version__)
print("✅ captum        :", captum.__version__)
print("✅ rdkit import OK | MolFromSmiles:",
      Chem.MolFromSmiles("CCO") is not None)
PY

echo "🎉  All deps installed"

⏳  Installing PyTorch-Geometric + RDKit (Py3.12 friendly)…
✅ torch         : 2.9.0+cu126
✅ torch_geometric: 2.7.0
✅ captum        : 0.8.0
✅ rdkit import OK | MolFromSmiles: True
🎉  All deps installed


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np

# 入力
p6 = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/AID_1346986_datatable_all.csv"  # KB-3-1
p7 = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/AID_1346987_datatable_all.csv"  # KB-8-5-11
df6 = pd.read_csv(p6)
df7 = pd.read_csv(p7)

def out2lab(x):
    if pd.isna(x): return np.nan
    x = str(x).strip().lower()
    return 1 if x=='active' else (0 if x=='inactive' else np.nan)

# 必要列抽出＆命名
a6 = df6[['PUBCHEM_CID','PUBCHEM_EXT_DATASOURCE_SMILES',
          'PUBCHEM_ACTIVITY_OUTCOME','Fit_LogAC50','Max_Response']].copy()
a6['Active6'] = a6['PUBCHEM_ACTIVITY_OUTCOME'].apply(out2lab)
a6 = a6.rename(columns={'Fit_LogAC50':'logAC50_6','Max_Response':'MaxResp_6'})

a7 = df7[['PUBCHEM_CID','PUBCHEM_EXT_DATASOURCE_SMILES',
          'PUBCHEM_ACTIVITY_OUTCOME','Fit_LogAC50','Max_Response']].copy()
a7['Active7'] = a7['PUBCHEM_ACTIVITY_OUTCOME'].apply(out2lab)
a7 = a7.rename(columns={'Fit_LogAC50':'logAC50_7','Max_Response':'MaxResp_7'})

# --- ここが重要：数値化（非数は NaN に）
for col in ['logAC50_6','MaxResp_6']:
    a6[col] = pd.to_numeric(a6[col], errors='coerce')
for col in ['logAC50_7','MaxResp_7']:
    a7[col] = pd.to_numeric(a7[col], errors='coerce')

print("dtypes after cast:")
print(a6[['logAC50_6','MaxResp_6']].dtypes)
print(a7[['logAC50_7','MaxResp_7']].dtypes)

# 変換でNaNになった件数を把握
print("coerce→NaN counts:",
      (a6['logAC50_6'].isna()).sum(), (a7['logAC50_7'].isna()).sum())

# マージ
m = pd.merge(a6.drop(columns=['PUBCHEM_ACTIVITY_OUTCOME']),
             a7.drop(columns=['PUBCHEM_ACTIVITY_OUTCOME']),
             on=['PUBCHEM_CID','PUBCHEM_EXT_DATASOURCE_SMILES'],
             how='outer')

# 差分
m['dlogAC50'] = m['logAC50_7'] - m['logAC50_6']

# 4倍閾値
LOG_FR_THR = np.log10(4.0)  # 0.60206...

def label_row(r):
    s6, s7, d = r['Active6'], r['Active7'], r['dlogAC50']
    # substrate
    if s6 == 1 and (s7 == 0 or (pd.notna(d) and d >= LOG_FR_THR)):
        return 1
    # non-substrate
    if (s7 == 1 and (s6 != 1 or (pd.notna(d) and d < LOG_FR_THR))) or (s6 == 0 and s7 == 0):
        return 0
    # その他は除外（Inconclusive含む）
    return np.nan

m['Substrate'] = m.apply(label_row, axis=1)

out = m.rename(columns={'PUBCHEM_CID':'CID',
                        'PUBCHEM_EXT_DATASOURCE_SMILES':'SMILES'})[['CID','SMILES','Substrate','dlogAC50']]
out = out.dropna(subset=['Substrate']).drop_duplicates()

print(out['Substrate'].value_counts(dropna=False))

# --- ここから追加入り --- #
# CIDが空白（欠損）を削除し、SampleWeight列(全て0.7)を追加
out['CID'] = pd.to_numeric(out['CID'], errors='coerce')  # 数値化して欠損判定を安定化
out = out.dropna(subset=['CID'])                         # CID欠損行を削除
out['SampleWeight'] = 0.7                                # 追加: 全行 0.7
# --- 追加入りここまで --- #

# --- 型の整備（任意だがオススメ） ---
out['CID'] = out['CID'].astype('Int64')   # 欠損対応の整数
out['Substrate'] = out['Substrate'].astype('Int64')
out['dlogAC50'] = pd.to_numeric(out['dlogAC50'], errors='coerce')

# 並べ替え（任意：陽性を上に、dlogAC50大きい順）
out = out.sort_values(['Substrate','dlogAC50'], ascending=[False, False])

# --- 保存 ---
save_dir = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
out_path = f"{save_dir}/AID_1346986_1346987_labels_fold4.csv"
out.to_csv(out_path, index=False)

print("✅ 保存:", out_path)
print("shape:", out.shape)
print("label counts:\n", out['Substrate'].value_counts(dropna=False))

audit_cols = [
    'PUBCHEM_CID','PUBCHEM_EXT_DATASOURCE_SMILES',
    'Active6','Active7','logAC50_6','logAC50_7','dlogAC50','Substrate'
]
audit = m[audit_cols].rename(columns={
    'PUBCHEM_CID':'CID','PUBCHEM_EXT_DATASOURCE_SMILES':'SMILES'
})
audit['CID'] = pd.to_numeric(audit['CID'], errors='coerce').astype('Int64')
audit_path = f"{save_dir}/AID_1346986_1346987_labels_fold4_audit.csv"
audit.to_csv(audit_path, index=False)

print("📝 監査用も保存:", audit_path)

dtypes after cast:
logAC50_6    float64
MaxResp_6    float64
dtype: object
logAC50_7    float64
MaxResp_7    float64
dtype: object
coerce→NaN counts: 6512 5900
Substrate
0.0    8397
1.0     399
Name: count, dtype: int64
✅ 保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/AID_1346986_1346987_labels_fold4.csv
shape: (8708, 5)
label counts:
 Substrate
0    8325
1     383
Name: count, dtype: Int64
📝 監査用も保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/AID_1346986_1346987_labels_fold4_audit.csv


In [ ]:
# =========================================
# Drug名（英語）→ PubChem CID → SMILES（多段フォールバック）
# 1) CanonicalSMILES > 2) IsomericSMILES > 3) PUG View > 4) SDF→RDKit
# 取得元は SMILES_Source に記録 / 無塩本体の正規化オプションあり
# ★ 元Excelの「ABCB1 Substrate」相当列を最終CSVへ保存（列名は元のまま保持）
#    重複時の優先ルールは SUBSTRATE_RESOLVE_POLICY で選択可（"first" / "positive"）
# =========================================

# 0) 必要パッケージ（Colab想定。不要ならコメントアウト）
try:
    import requests, pandas, openpyxl, rdkit
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "requests", "pandas", "openpyxl", "rdkit-pypi"])

# 1) 設定
INPUT_XLSX = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/Drug_Transporters_(ABCB1).xlsx"
DRUG_COL   = "Drug"
OUT_CSV    = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles.csv"
UNRES_CSV  = OUT_CSV.replace(".csv", "_unresolved.csv")

UNSALT       = False              # True: 最大フラグメントで“無塩本体”に正規化
KEEP_METALS  = True               # 金属を含むときはフラグメント化しない
PER_ITEM_SLEEP_SEC = 0.25

# ▼ Substrateの重複時優先ルール: "first"（先勝ち） or "positive"（陽性優先）
SUBSTRATE_RESOLVE_POLICY = "first"

# 2) 実装
import time, random, re, csv
import pandas as pd
import requests
from typing import List, Optional, Tuple
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

PUG_BASE   = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
PUGVIEW    = "https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound"
HDRS = {"User-Agent":"drugname2cid-fallback/1.2 (colab)", "Accept":"application/json"}
METALS = {"Li","Na","K","Mg","Ca","Fe","Cu","Zn","Pt","Bi","Al","Mn","Ni","Co","Sr","Ba","Ag","Au"}

def build_session():
    s = requests.Session()
    retry = Retry(total=6, connect=6, read=6, backoff_factor=0.6,
                  status_forcelist=[429,500,502,503,504],
                  allowed_methods=frozenset(["GET"]), raise_on_status=False)
    ad = HTTPAdapter(max_retries=retry, pool_connections=32, pool_maxsize=32)
    s.mount("https://", ad); s.mount("http://", ad)
    s.headers.update(HDRS)
    return s

S = build_session()

def sleep():
    time.sleep(PER_ITEM_SLEEP_SEC + random.uniform(0, PER_ITEM_SLEEP_SEC))

def get_json(url, timeout=60):
    try:
        r = S.get(url, timeout=timeout)
        if r.status_code == 200:
            try: return r.json()
            except Exception: return None
        if r.status_code in (400,404):
            return None
    except requests.RequestException:
        return None
    return None

def get_bytes(url, timeout=60):
    try:
        r = S.get(url, timeout=timeout)
        if r.status_code == 200:
            return r.content
        if r.status_code in (400,404):
            return None
    except requests.RequestException:
        return None
    return None

def normalize_name(x:str)->str:
    return re.sub(r"\s+"," ", (x or "").strip())

def find_real_col(df: pd.DataFrame, target: str)->Optional[str]:
    t = target.strip().lower()
    for c in df.columns:
        if str(c).strip().lower() == t:
            return c
    return None

def find_substrate_col(df: pd.DataFrame)->Optional[str]:
    # 「ABCB1 Substrate」相当列を“緩く”検出
    norm = {str(c): str(c).strip().lower().replace(" ", "").replace("_","") for c in df.columns}
    inv  = {v: k for k, v in norm.items()}
    keys = [
        "abcb1substrate", "abcb1_substrate", "abcb1 substrate",
        "substrate", "p-gpsubstrate", "pgpsubstrate"
    ]
    for k in keys:
        kk = k.replace(" ", "").replace("_","")
        if kk in inv:
            return inv[kk]
    return None

# ---- PubChem basic ----
def name_to_cids(name:str)->List[int]:
    from urllib.parse import quote
    js = get_json(f"{PUG_BASE}/compound/name/{quote(name)}/cids/JSON")
    if not js: return []
    return list(map(int, js.get("IdentifierList",{}).get("CID",[])))

def cid_title(cid:int)->Optional[str]:
    js = get_json(f"{PUG_BASE}/compound/cid/{cid}/property/Title/JSON")
    if not js: return None
    rows = js.get("PropertyTable",{}).get("Properties",[])
    return rows[0].get("Title") if rows else None

def cid_synonyms(cid:int)->List[str]:
    js = get_json(f"{PUG_BASE}/compound/cid/{cid}/synonyms/JSON")
    if not js: return []
    info = js.get("InformationList",{}).get("Information",[])
    syns = info[0].get("Synonym",[]) if info else []
    return [normalize_name(s).lower() for s in syns if isinstance(s,str)]

# ---- SMILES getters with fallback ----
def cid_canonical_smiles(cid:int)->Optional[str]:
    js = get_json(f"{PUG_BASE}/compound/cid/{cid}/property/CanonicalSMILES/JSON")
    if not js: return None
    rows = js.get("PropertyTable",{}).get("Properties",[])
    return rows[0].get("CanonicalSMILES") if rows else None

def cid_isomeric_smiles(cid:int)->Optional[str]:
    js = get_json(f"{PUG_BASE}/compound/cid/{cid}/property/IsomericSMILES/JSON")
    if not js: return None
    rows = js.get("PropertyTable",{}).get("Properties",[])
    return rows[0].get("IsomericSMILES") if rows else None

def cid_smiles_from_pugview(cid:int)->Tuple[Optional[str], Optional[str]]:
    js = get_json(f"{PUGVIEW}/{cid}/JSON")
    if not js: return (None, None)
    iso, can = None, None
    def walk(obj):
        nonlocal iso, can
        if isinstance(obj, dict):
            head = obj.get("TOCHeading") or obj.get("Name")
            if head in ("Isomeric SMILES","Canonical SMILES"):
                for info in obj.get("Information", []):
                    for swm in info.get("Value", {}).get("StringWithMarkup", []):
                        s = swm.get("String")
                        if isinstance(s, str) and s:
                            if head == "Isomeric SMILES" and not iso: iso = s
                            if head == "Canonical SMILES" and not can: can = s
            for v in obj.values(): walk(v)
        elif isinstance(obj, list):
            for v in obj: walk(v)
    walk(js)
    return iso, can

def cid_smiles_from_sdf(cid:int)->Optional[str]:
    b = get_bytes(f"{PUG_BASE}/compound/cid/{cid}/SDF?record_type=2d")
    if not b: return None
    try:
        txt = b.decode("utf-8","ignore")
        mol = Chem.MolFromMolBlock(txt, sanitize=True)
        if mol is None: return None
        return Chem.MolToSmiles(mol, isomericSmiles=True, canonical=True)
    except Exception:
        return None

def best_available_smiles(cid:int)->Tuple[Optional[str], str, Optional[str]]:
    """
    戻り: (BestAvailableSMILES, SMILES_Source, CanonicalSMILES_raw)
    Source: 'canonical' | 'isomeric' | 'pugview_isomeric' | 'pugview_canonical' | 'sdf_rdkit'
    """
    can = cid_canonical_smiles(cid); sleep()
    if can: return can, "canonical", can
    iso = cid_isomeric_smiles(cid); sleep()
    if iso: return iso, "isomeric", None
    iso2, can2 = cid_smiles_from_pugview(cid); sleep()
    if iso2: return iso2, "pugview_isomeric", can2
    if can2: return can2, "pugview_canonical", can2
    sdf = cid_smiles_from_sdf(cid)
    if sdf: return sdf, "sdf_rdkit", None
    return None, "none", None

# ---- 無塩正規化 ----
def elements_in(smiles:str):
    m = Chem.MolFromSmiles(smiles)
    if not m: return set()
    return {a.GetSymbol() for a in m.GetAtoms()}

def largest_fragment_smiles(smiles:str)->Optional[str]:
    m = Chem.MolFromSmiles(smiles)
    if m is None: return None
    frags = Chem.GetMolFrags(m, asMols=True, sanitizeFrags=False)
    if len(frags)==1:
        return Chem.MolToSmiles(frags[0], isomericSmiles=True, canonical=True)
    best, heavy = None, -1
    for f in frags:
        h = f.GetNumHeavyAtoms()
        if h>heavy: best, heavy = f, h
    return Chem.MolToSmiles(best, isomericSmiles=True, canonical=True) if best else None

def maybe_unsalt(smi:str)->str:
    if not UNSALT: return smi
    if any(e in METALS for e in elements_in(smi)):
        return smi if KEEP_METALS else (largest_fragment_smiles(smi) or smi)
    return largest_fragment_smiles(smi) or smi

# ---- 候補から最良CIDを選ぶ ----
def choose_best_cid(name:str, cids:List[int])->Tuple[Optional[int], str]:
    """優先度: Title 完全一致 > Synonym 完全一致 > 先頭候補"""
    q = normalize_name(name).lower()
    for cid in cids:
        t = cid_title(cid)
        if t and normalize_name(t).lower() == q:
            return cid, "exact_title"
        sleep()
    for cid in cids:
        syns = cid_synonyms(cid)
        if q in syns:
            return cid, "exact_synonym"
        sleep()
    return (cids[0] if cids else None), "first_hit"

# ---- Substrate 値の整形（文字でも数値でも“人間のYes/No”に頑健）----
def normalize_substrate_value(v) -> str:
    if v is None or (isinstance(v, float) and pd.isna(v)): return ""
    s = str(v).strip()
    if not s: return ""
    low = s.lower()
    # 典型的な肯定語
    if low in {"1","yes","y","true","substrate","positive","pos"}:
        return "1"
    # 典型的な否定語
    if low in {"0","no","n","false","non-substrate","negative","neg"}:
        return "0"
    # それ以外は元の文字列を残す
    return s

def resolve_substrate(prev: str, new: str) -> str:
    """重複時の解決。SUBSTRATE_RESOLVE_POLICYに従う。"""
    prev = prev or ""
    new  = new or ""
    if not prev: return new
    if not new:  return prev
    if SUBSTRATE_RESOLVE_POLICY == "positive":
        # どちらかが1/Yes系なら "1" を優先
        if normalize_substrate_value(prev) == "1" or normalize_substrate_value(new) == "1":
            return "1"
        # どちらも明確でない場合は先勝ち
        return prev
    # default: first（先勝ち）
    return prev

# ---- メイン ----
def main():
    xf = pd.ExcelFile(INPUT_XLSX)
    chosen_sheet, df, drug_col = None, None, None
    for sh in xf.sheet_names:
        tmp = pd.read_excel(INPUT_XLSX, sheet_name=sh)
        real = find_real_col(tmp, DRUG_COL)
        if real is not None:
            chosen_sheet, df, drug_col = sh, tmp, real
            break
    if df is None:
        raise RuntimeError(f"Excel内に '{DRUG_COL}' 列が見つかりません：{INPUT_XLSX}")
    print(f"✅ 使用シート: {chosen_sheet} / Drug列: {drug_col}")

    # Substrate列を特定（無い場合は None）
    substrate_col = find_substrate_col(df)
    if substrate_col:
        print(f"✅ 見つかった Substrate 列: {substrate_col}")
    else:
        print("ℹ Substrate 列は見つかりませんでした（出力には含めません）")

    # Drug名→Substrate値（正規化）の辞書を構築（重複はポリシーで解決）
    name_to_sub = {}
    if substrate_col:
        for _, r in df.iterrows():
            name = normalize_name(str(r.get(drug_col, "")))
            if not name:
                continue
            val_raw = r.get(substrate_col, "")
            val_norm = normalize_substrate_value(val_raw)
            if name in name_to_sub:
                name_to_sub[name] = resolve_substrate(name_to_sub[name], val_norm)
            else:
                name_to_sub[name] = val_norm

    names = [normalize_name(x) for x in df[drug_col].astype(str).tolist()]
    names = [n for n in names if n]
    seen = set()
    rows, unresolved = [], []

    for i, name in enumerate(names, 1):
        if name in seen:
            continue
        seen.add(name)

        cids = name_to_cids(name); sleep()
        if not cids:
            rec = {"Drug_name": name, "reason":"no_cid_from_name"}
            if substrate_col: rec[substrate_col] = name_to_sub.get(name, "")
            unresolved.append(rec)
            continue

        best_cid, match = choose_best_cid(name, cids)
        if not best_cid:
            rec = {"Drug_name": name, "candidates":";".join(map(str,cids)), "reason":"could_not_choose"}
            if substrate_col: rec[substrate_col] = name_to_sub.get(name, "")
            unresolved.append(rec)
            continue

        best_smi, src, can_raw = best_available_smiles(best_cid); sleep()
        if not best_smi:
            rec = {"Drug_name": name, "Chosen_CID": best_cid, "reason":"no_smiles_all_sources"}
            if substrate_col: rec[substrate_col] = name_to_sub.get(name, "")
            unresolved.append(rec)
            continue

        best_smi_norm = maybe_unsalt(best_smi)

        row = {
            "Drug_name": name,
            "Chosen_CID": best_cid,
            "BestAvailableSMILES": best_smi_norm,
            "SMILES_Source": src,                 # canonical / isomeric / pugview_* / sdf_rdkit
            "CanonicalSMILES_raw": can_raw or "", # 参考
            "MatchType": match,
            "Candidates": ";".join(map(str,cids))
        }
        # 出力に元の列名のまま Substrate を追加
        if substrate_col:
            row[substrate_col] = name_to_sub.get(name, "")

        rows.append(row)

        if i % 25 == 0 or i == len(names):
            print(f"[prog] {i}/{len(names)}  resolved={len(rows)}  unresolved={len(unresolved)}")

    # 出力列（Substrate列は見つかった場合のみ追加）
    base_cols = ["Drug_name","Chosen_CID","BestAvailableSMILES","SMILES_Source",
                 "CanonicalSMILES_raw","MatchType","Candidates"]
    out_cols = base_cols + ([substrate_col] if substrate_col else [])

    out_df = pd.DataFrame(rows, columns=out_cols)
    out_df.to_csv(OUT_CSV, index=False, quoting=csv.QUOTE_MINIMAL, encoding="utf-8")
    print(f"✅ 保存: {OUT_CSV} (rows={len(out_df)})")

    if unresolved:
        unresolved_df = pd.DataFrame(unresolved)
        # 表示のための列順整形
        un_cols = ["Drug_name", "reason", "Chosen_CID", "candidates"]
        if substrate_col and substrate_col not in un_cols:
            un_cols.append(substrate_col)
        unresolved_df = unresolved_df[[c for c in un_cols if c in unresolved_df.columns]]
        unresolved_df.to_csv(UNRES_CSV, index=False, encoding="utf-8")
        print(f"⚠ 未解決: {UNRES_CSV} (rows={len(unresolved_df)})")

if __name__ == "__main__":
    main()

✅ 使用シート: Sheet 1 / Drug列: Drug
✅ 見つかった Substrate 列: ABCB1 Substrate
[prog] 25/235  resolved=24  unresolved=0
[prog] 50/235  resolved=49  unresolved=0
[prog] 75/235  resolved=74  unresolved=0
[prog] 100/235  resolved=99  unresolved=0
[prog] 125/235  resolved=123  unresolved=0
[prog] 150/235  resolved=146  unresolved=0
[prog] 175/235  resolved=170  unresolved=1
[prog] 200/235  resolved=195  unresolved=1
[prog] 225/235  resolved=220  unresolved=1


[prog] 235/235  resolved=230  unresolved=1
✅ 保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles.csv (rows=230)
⚠ 未解決: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_unresolved.csv (rows=1)


In [ ]:
# === Merge 'kegg_pubchem_smiles.csv' with uploaded FDA file ===
# 仕様:
#  - 主キー: CID優先 (cid ↔ Chosen_CID)、欠ける場合は名前の正規化一致 (name ↔ Drug_name)
#  - SMILES不一致時: 既存(BestAvailableSMILES)を優先（FDA値は最終CSVに出さず、別diffで監査）
#  - 出力: 元と同じフォルダに kegg_pubchem_smiles_merged.csv
#  - 列: 元CSVの列を保持 + MergeSource（"base" | "base+fda" | "fda_file"）+ SampleWeight（★追加）
#  - 監査: 追加・コンフリクトは _diff_report.csv にまとめる（FDA_SMILESはここだけに出力）
#  - 追加要件: FDA由来データには「ABCB1 Substrate」列に 1 を入力（既存値が空のときのみ補完）
#  - 追加要件(重み): FDA由来（base+fda / fda_file）は SampleWeight=5.0、その他=1.0

import os, csv
import numpy as np
import pandas as pd

BASE_CSV   = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles.csv"   # 既存
FDA_CSV    = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/fda_ex_mdr_sub.csv"
OUT_MERGED = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged.csv"
OUT_DIFF   = OUT_MERGED.replace(".csv", "_diff_report.csv")

# --- load ---
base = pd.read_csv(BASE_CSV)
fda  = pd.read_csv(FDA_CSV)

# --- substrate列の同定/整備（最終列名は "ABCB1 Substrate" に統一） ---
def _norm_key(s: str) -> str:
    return str(s).strip().lower().replace(" ", "").replace("_", "")

sub_col_found = None
for c in base.columns:
    if _norm_key(c) in {"abcb1substrate", "pgpsubstrate", "p-gpsubstrate", "substrate"}:
        sub_col_found = c
        break

# ベース側に対象列が無ければ空列を新規作成
if sub_col_found is None and "ABCB1 Substrate" not in base.columns:
    base["ABCB1 Substrate"] = ""
elif sub_col_found is not None and sub_col_found != "ABCB1 Substrate":
    # 別名で存在する場合は "ABCB1 Substrate" にコピーして統一
    base["ABCB1 Substrate"] = base[sub_col_found].astype(object)
elif "ABCB1 Substrate" in base.columns:
    pass
else:
    base["ABCB1 Substrate"] = ""

# --- ensure base columns exist ---
base_cols = [
    "Drug_name","Chosen_CID","BestAvailableSMILES","SMILES_Source",
    "CanonicalSMILES_raw","MatchType","Candidates","ABCB1 Substrate"
]
for c in base_cols:
    if c not in base.columns:
        base[c] = ""

# --- normalize helpers ---
def _to_int_or_nan(x):
    try:
        if pd.isna(x): return np.nan
        s = str(x).strip()
        if s == "": return np.nan
        return int(s)
    except:
        return np.nan

def _norm_name(x:str)->str:
    s = str(x).strip()
    s = " ".join(s.split())
    return s.lower()

base["Chosen_CID_norm"] = base["Chosen_CID"].apply(_to_int_or_nan)
base["Drug_name_norm"]  = base["Drug_name"].astype(str).map(_norm_name)

# FDA側 期待列: name, cid, smiles
for col in ["name","cid","smiles"]:
    if col not in fda.columns:
        raise RuntimeError(f"添付CSVに必要列がありません: {col}")

fda["cid_norm"]  = fda["cid"].apply(_to_int_or_nan)
fda["name_norm"] = fda["name"].astype(str).map(_norm_name)

# --- step1: CIDでマージ（左外結合: 既存優先） ---
m = base.merge(
    fda.rename(columns={"name":"FDA_name","cid":"FDA_cid","smiles":"FDA_SMILES"}),
    left_on="Chosen_CID_norm", right_on="cid_norm", how="left"
)

# MergeSource の初期値（ベースにFDAが何かしら結びついたら base+fda、無ければ base）
m["MergeSource"] = np.where(m["FDA_name"].notna(), "base+fda", "base")

# --- step2: CIDで結びつかなかったFDA行を「名前一致」で補完/追加 ---
linked_fda_cids = set(m["cid_norm"].dropna().astype(int).unique().tolist())

base_name_to_rows = {}
for i, nm in enumerate(m["Drug_name_norm"].tolist()):
    base_name_to_rows.setdefault(nm, []).append(i)

extra_rows = []
for _, fr in fda.iterrows():
    nm = fr["name_norm"]
    cidn = fr["cid_norm"]
    fda_smiles = fr["FDA_SMILES"] if "FDA_SMILES" in fr else (fr["smiles"] if isinstance(fr["smiles"], str) else "")
    if pd.notna(cidn) and int(cidn) in linked_fda_cids:
        continue

    if nm in base_name_to_rows:
        # 既存行にFDA情報を補完（名前一致）
        idxs = base_name_to_rows[nm]
        m.loc[idxs, "FDA_name"]   = fr["name"]
        m.loc[idxs, "FDA_cid"]    = fr["cid"]
        m.loc[idxs, "FDA_SMILES"] = fda_smiles
        m.loc[idxs, "MergeSource"] = "base+fda"
        # ★ FDA由来 → 「ABCB1 Substrate」が空なら 1 を補完
        if "ABCB1 Substrate" not in m.columns:
            m["ABCB1 Substrate"] = ""
        empty_mask = m.loc[idxs, "ABCB1 Substrate"].astype(str).str.strip().isin(["", "nan", "None", "NA"])
        m.loc[np.array(idxs)[empty_mask.values], "ABCB1 Substrate"] = "1"
    else:
        # 新規追加（from FDA）
        new_row = {
            "Drug_name": fr["name"],
            "Chosen_CID": int(fr["cid"]) if pd.notna(cidn) else "",
            "BestAvailableSMILES": fda_smiles,
            "SMILES_Source": "fda_file",
            "CanonicalSMILES_raw": "",
            "MatchType": "from_fda",
            "Candidates": "",
            "Chosen_CID_norm": cidn,
            "Drug_name_norm": nm,
            "FDA_name": fr["name"],
            "FDA_cid": fr["cid"],
            "FDA_SMILES": fda_smiles,
            "cid_norm": cidn,
            "MergeSource": "fda_file",
            # ★ 追加行はFDA由来なので常に 1
            "ABCB1 Substrate": "1",
        }
        extra_rows.append(new_row)

if extra_rows:
    m = pd.concat([m, pd.DataFrame(extra_rows)], ignore_index=True)

# --- SMILES不一致処理：既存を優先、最終CSVにはFDA_SMILESは載せない（diffにのみ出力） ---
m["BestAvailableSMILES"] = m["BestAvailableSMILES"].astype(str)
m["FDA_SMILES"]          = m.get("FDA_SMILES", pd.Series("", index=m.index)).astype(str)

# 既存空＆FDAあり → 補完（SMILES_Sourceもfda_file_fillへ）
need_fill = (m["BestAvailableSMILES"].str.len()==0) & (m["FDA_SMILES"].str.len()>0)
m.loc[need_fill, "BestAvailableSMILES"] = m.loc[need_fill, "FDA_SMILES"]
m.loc[need_fill, "SMILES_Source"] = m.loc[need_fill, "SMILES_Source"].mask(
    m.loc[need_fill, "SMILES_Source"].astype(str).str.len()==0, "fda_file_fill"
)
# SMILES補完が発生した行がFDA由来なら Substrate も 1（空のときのみ）に補完
if "ABCB1 Substrate" not in m.columns:
    m["ABCB1 Substrate"] = ""
fill_sub_mask = need_fill & m["MergeSource"].isin(["base+fda","fda_file"])
empty_sub = m["ABCB1 Substrate"].astype(str).str.strip().isin(["", "nan", "None", "NA"])
m.loc[fill_sub_mask & empty_sub, "ABCB1 Substrate"] = "1"

# 衝突フラグ（監査用）
conflict = (
    (m["BestAvailableSMILES"].str.len()>0) &
    (m["FDA_SMILES"].str.len()>0) &
    (m["BestAvailableSMILES"] != m["FDA_SMILES"])
)
m["SMILES_conflict"] = np.where(conflict, 1, 0)

# === ★ ここから：重み列を追加（FDA由来=5.0、その他=1.0） ===
m["SampleWeight"] = np.where(m["MergeSource"].isin(["base+fda", "fda_file"]), 5.0, 1.0)

# --- 出力カラム（元CSVの列を保持し、MergeSourceとSampleWeightを追加。FDA_SMILESは監査CSVにのみ） ---
out_cols = [c for c in base_cols if c in m.columns] + ["MergeSource", "SampleWeight"]  # ★ SampleWeight 追加
out = m[out_cols].copy()

# 完全重複除去（保守的に主要3列で）
out = out.drop_duplicates(subset=["Drug_name","Chosen_CID","BestAvailableSMILES"], keep="first").reset_index(drop=True)

# 保存
out.to_csv(OUT_MERGED, index=False, quoting=csv.QUOTE_MINIMAL, encoding="utf-8")

# --- 差分レポート（新規追加・SMILESコンフリクト） ---
diff_rows = []
# 追加（from_fda）
added_mask = (m.get("MatchType","")=="from_fda") | (m["MergeSource"]=="fda_file")
for _, r in m[added_mask].iterrows():
    diff_rows.append({
        "type":"added_from_fda",
        "Drug_name": r.get("Drug_name",""),
        "Chosen_CID": r.get("Chosen_CID",""),
        "BestAvailableSMILES": r.get("BestAvailableSMILES",""),
        "FDA_SMILES": r.get("FDA_SMILES",""),
        "ABCB1_Substrate_out": r.get("ABCB1 Substrate",""),
        "MergeSource": r.get("MergeSource",""),
        "SampleWeight": r.get("SampleWeight",""),
    })

# SMILESコンフリクト
for _, r in m[m["SMILES_conflict"]==1].iterrows():
    diff_rows.append({
        "type":"smiles_conflict",
        "Drug_name": r.get("Drug_name",""),
        "Chosen_CID": r.get("Chosen_CID",""),
        "BestAvailableSMILES": r.get("BestAvailableSMILES",""),
        "FDA_SMILES": r.get("FDA_SMILES",""),
        "ABCB1_Substrate_out": r.get("ABCB1 Substrate",""),
        "MergeSource": r.get("MergeSource",""),
        "SampleWeight": r.get("SampleWeight",""),
    })

pd.DataFrame(diff_rows).to_csv(OUT_DIFF, index=False, encoding="utf-8")

print(f"✅ Merged CSV saved: {OUT_MERGED}  (rows={len(out)})")
print(f"📝 Diff report    : {OUT_DIFF}   (rows={len(diff_rows)})")

✅ Merged CSV saved: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged.csv  (rows=235)
📝 Diff report    : /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged_diff_report.csv   (rows=235)


In [ ]:
# =========================================
# 塩/水和物/溶媒和を「同一薬物（塩でない親分子）」に統合（優先薬物対応 + ABCB1集計 + Sample_Weight出力）
# =========================================

# 1) 設定（省略部はそのまま）
IN_CSV   = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged.csv"
OUT_BASE = IN_CSV.rsplit(".",1)[0]
OUT_CONS = OUT_BASE + "_consolidated.csv"
OUT_CHILD= OUT_BASE + "_children_expanded.csv"

# 2) 実装（省略部はそのまま）
import re, csv, time, random, numpy as np
import pandas as pd
import requests
from typing import Optional, List, Tuple, Dict, Any
from functools import lru_cache
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize as STD

# ...（PRIORITY_PAIRS, _norm_name など既存の定義はそのまま）...

# ---- 列名ゆれを吸収 ----
def find_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    low = {str(c).strip().lower(): c for c in df.columns}
    for name in candidates:
        c = low.get(name.strip().lower())
        if c: return c
    return None

# ...（PubChem utils と RDKit 正規化関数はそのまま）...

# ---- ABCB1 Substrate 正規化（0/1/欠損）----
def norm_abcb1(x):
    if pd.isna(x): return pd.NA
    s = str(x).strip().lower()
    if s in {"1", "true", "yes", "y"}: return 1
    if s in {"0", "false", "no", "n"}: return 0
    return pd.NA

# 3) 入力読み込み＆列の同定
df = pd.read_csv(IN_CSV)

col_name  = find_col(df, ["Drug_name","Drug","Drug Name"])
col_cid   = find_col(df, ["PubChem_CID","Chosen_CID","CID"])
col_smi   = find_col(df, ["SMILES","BestAvailableSMILES","CanonicalSMILES","CanonicalSMILES_raw"])
col_abcb1 = find_col(df, ["ABCB1 Substrate","abcb1 substrate","ABCB1_Substrate","abcb1_substrate"])
# ★ SampleWeight 列の同定（なければ作成して 1.0）
col_sw_in = find_col(df, ["SampleWeight","Sample_Weight","sampleweight"])
if col_sw_in is None:
    df["Sample_Weight"] = 1.0
    col_sw_in = "Sample_Weight"
else:
    # 出力は列名を "Sample_Weight" に統一
    df["Sample_Weight"] = pd.to_numeric(df[col_sw_in], errors="coerce").fillna(1.0).astype(float).clip(lower=0.0)

if not (col_name and col_smi):
    raise RuntimeError(f"必要列が見つかりません: Drug_name/Drug と SMILES系（見つかった列: {df.columns.tolist()})")

# ABCB1列が無くても動くように
if col_abcb1 is None:
    df["__ABCB1_tmp__"] = pd.NA
    col_abcb1 = "__ABCB1_tmp__"

# 4) 子→親のマッピング生成（★ 子に Sample_Weight を保持）
rows_child: List[Dict[str,Any]] = []
for i, r in df.iterrows():
    name = str(r[col_name]).strip()
    smi  = str(r[col_smi]).strip()
    cid  = int(r[col_cid]) if (col_cid and pd.notna(r[col_cid])) else None
    if not smi or smi.lower() in ("nan","none"):
        continue

    pkey, psm = parent_key_and_smiles(smi)
    if not (pkey and psm):
        continue

    pcid = None
    if LOOKUP_PARENT_CID:
        cands = smiles_to_cids(psm); pcid = int(cands[0]) if cands else None

    rows_child.append({
        "Child_Name": name,
        "Child_CID": cid,
        "Child_SMILES": smi,
        "Parent_Key": pkey,
        "Parent_SMILES": psm,
        "Parent_CID": pcid,
        "Name_Stripped": strip_saltish(name),
        "PreferredFlag": False,
        "ABCB1_Substrate": norm_abcb1(r[col_abcb1]),
        # ★ ここで各子行の Sample_Weight を保持（既に1.0補完済）
        "Sample_Weight": float(r["Sample_Weight"])
    })

    if (i+1) % 50 == 0:
        print(f"[prog] processed {i+1}/{len(df)}")

child_df = pd.DataFrame(rows_child)

# 5) 親へ集約（★ Sample_Weight は子の最大値を採用）
cons_rows: List[Dict[str,Any]] = []
pref_indices: List[int] = []

for pkey, g in child_df.groupby("Parent_Key", dropna=True):
    parent_smiles = g["Parent_SMILES"].iloc[0]

    # --- 代表候補の決定（既存ロジックのまま）---
    norm_names = g["Child_Name"].astype(str).map(_norm_name)
    cid_series = g["Child_CID"]
    mask_cid  = cid_series.isin(PRIORITY_CID_SET)
    mask_name = norm_names.isin({s.lower() for s in [_norm_name(d["name"]) for d in PRIORITY_PAIRS]})
    mask_pref = (mask_cid | mask_name)

    if mask_pref.any():
        pref_idx = None
        for pr in PRIORITY_PAIRS:
            cand_mask = (cid_series.eq(pr["cid"])) | (norm_names.eq(_norm_name(pr["name"])))
            hit = g.index[cand_mask]
            if len(hit) > 0:
                pref_idx = int(hit[0]); break
        if pref_idx is None: pref_idx = int(g.index[0])
        rep_row = child_df.loc[pref_idx]
        rep_name = str(rep_row["Child_Name"])
        rep_cid  = int(rep_row["Child_CID"]) if pd.notna(rep_row["Child_CID"]) else None
        pref_indices.append(pref_idx)
    else:
        parent_cid_mode = g["Parent_CID"].dropna().astype(int)
        rep_cid = parent_cid_mode.mode().iloc[0] if len(parent_cid_mode)>0 else None
        rep_name = None
        if rep_cid is not None:
            t = cid_title(rep_cid);
            if t: rep_name = t
        if not rep_name:
            stripped = g["Name_Stripped"].str.lower().value_counts()
            rep_name = stripped.index[0] if len(stripped)>0 else g["Child_Name"].iloc[0]

    if "rep_cid" not in locals() or rep_cid is None:
        parent_cid_mode = g["Parent_CID"].dropna().astype(int)
        rep_cid = parent_cid_mode.mode().iloc[0] if len(parent_cid_mode)>0 else None

    # --- ABCB1(A=any=1) 集約 ---
    ab = g["ABCB1_Substrate"]
    vals = ab[ab.isin([0,1])]
    if (vals == 1).any():
        parent_abcb1 = 1
    elif (vals == 0).any():
        parent_abcb1 = 0
    else:
        parent_abcb1 = pd.NA

    # ★ Sample_Weight 集約（max：FDA=5.0が1つでもあれば親も5.0）
    parent_sw = float(g["Sample_Weight"].fillna(1.0).max())

    cons_rows.append({
        "Drug_Name": rep_name,
        "PubChem_CID": rep_cid,
        "SMILES": parent_smiles,
        "Parent_Key": pkey,
        "Children_N": int(len(g)),
        "Children_Names": "; ".join(sorted(set(g["Child_Name"]))),
        "Children_CIDs": "; ".join(sorted({str(int(x)) for x in g["Child_CID"].dropna().tolist()})) if g["Child_CID"].notna().any() else "",
        "ABCB1 Substrate": parent_abcb1,
        "Sample_Weight": parent_sw,  # ★ 追加
    })

# 優先行フラグ
if len(pref_indices)>0:
    child_df.loc[pref_indices, "PreferredFlag"] = True

# 6) 保存（★ 子/親ともに Sample_Weight を含めて出力）
child_cols = [
    "Child_Name","Child_CID","Child_SMILES",
    "Parent_Key","Parent_SMILES","Parent_CID",
    "PreferredFlag","ABCB1_Substrate","Name_Stripped",
    "Sample_Weight"  # ★ 追加
]
child_df.to_csv(OUT_CHILD, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)

cons_df = pd.DataFrame(cons_rows, columns=[
    "Drug_Name","PubChem_CID","SMILES","Parent_Key",
    "Children_N","Children_Names","Children_CIDs",
    "ABCB1 Substrate","Sample_Weight"  # ★ 追加
]).sort_values(["Drug_Name","Children_N"], ascending=[True, False])

cons_df.to_csv(OUT_CONS, index=False, encoding="utf-8", quoting=csv.QUOTE_MINIMAL)

print(f"✅ 親分子へ統合して保存: {OUT_CONS}  (rows={len(cons_df)})")
print(f"   子展開（優先フラグ + ABCB1保持 + Sample_Weight）: {OUT_CHILD}  (rows={len(child_df)})")

[14:04:34] Running LargestFragmentChooser
[14:04:34] Running Normalizer
[14:04:34] Running Uncharger
[14:04:34] Running LargestFragmentChooser
[14:04:34] Fragment: Nc1nc([O-])c2ncn(COCCO)c2n1
[14:04:34] New largest fragment: Nc1nc([O-])c2ncn(COCCO)c2n1 (26)
[14:04:34] Fragment: [Na+]
[14:04:34] Running Normalizer
[14:04:34] Running Uncharger
[14:04:34] Removed negative charge.
[14:04:34] Running LargestFragmentChooser
[14:04:34] Running Normalizer
[14:04:34] Running Uncharger
[14:04:34] Running LargestFragmentChooser
[14:04:34] Fragment: CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1
[14:04:34] New largest fragment: CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1O[C@H]1CCOC1 (59)
[14:04:34] Fragment: O=C(O)/C=C\C(=O)O
[14:04:34] Fragment: O=C(O)/C=C\C(=O)O
[14:04:34] Running Normalizer
[14:04:34] Running Uncharger
[14:04:34] Running LargestFragmentChooser
[14:04:34] Running Normalizer
[14:04:34] Running Uncharger
[14:04:34] Running LargestFragmentChooser
[14:04:34] Fra

[prog] processed 50/235


[14:04:38] Running LargestFragmentChooser
[14:04:38] Running Normalizer
[14:04:38] Running Uncharger
[14:04:38] Running LargestFragmentChooser
[14:04:38] Running Normalizer
[14:04:38] Running Uncharger
[14:04:38] Running LargestFragmentChooser
[14:04:38] Fragment: CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1)OCCO2
[14:04:38] New largest fragment: CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1)OCCO2 (65)
[14:04:38] Fragment: CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1)OCCO2
[14:04:38] New largest fragment: CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@H](O)c1ccc2c(c1)OCCO2 (65)
[14:04:38] Fragment: O=C(O)[C@H](O)[C@@H](O)C(=O)O
[14:04:38] Running Normalizer
[14:04:38] Running Uncharger
[14:04:38] Running LargestFragmentChooser
[14:04:38] Running Normalizer
[14:04:38] Running Uncharger
[14:04:38] Running LargestFragmentChooser
[14:04:38] Fragment: CCCCC1(CCCC)CN(c2ccccc2)c2cc(SC)c(OCC(=O)N[C@@H](C(=O)NCC(=O)O)c3ccccc3)cc2S(=O)(=O)C1
[14:04:38] New largest fragment: CCCCC1(CCCC)CN(c2ccccc2)c2cc(SC)

[prog] processed 100/235


[14:04:40] Can't kekulize mol.  Unkekulized atoms: 4 32
[14:04:40] Can't kekulize mol.  Unkekulized atoms: 4 32
[14:04:40] Running LargestFragmentChooser
[14:04:40] Fragment: CCc1nc(C(N)=O)c(Nc2ccc(N3CCC(N4CCN(C)CC4)CC3)c(OC)c2)nc1NC1CCOCC1
[14:04:40] New largest fragment: CCc1nc(C(N)=O)c(Nc2ccc(N3CCC(N4CCN(C)CC4)CC3)c(OC)c2)nc1NC1CCOCC1 (84)
[14:04:40] Fragment: CCc1nc(C(N)=O)c(Nc2ccc(N3CCC(N4CCN(C)CC4)CC3)c(OC)c2)nc1NC1CCOCC1
[14:04:40] New largest fragment: CCc1nc(C(N)=O)c(Nc2ccc(N3CCC(N4CCN(C)CC4)CC3)c(OC)c2)nc1NC1CCOCC1 (84)
[14:04:40] Fragment: O=C(O)/C=C/C(=O)O
[14:04:40] Running Normalizer
[14:04:40] Running Uncharger
[14:04:40] Can't kekulize mol.  Unkekulized atoms: 4 32
[14:04:40] Can't kekulize mol.  Unkekulized atoms: 4 32
[14:04:40] Running LargestFragmentChooser
[14:04:40] Running Normalizer
[14:04:40] Running Uncharger
[14:04:40] Running LargestFragmentChooser
[14:04:40] Running Normalizer
[14:04:40] Running Uncharger
[14:04:40] Running LargestFragmentChooser
[14:04:40]

[prog] processed 150/235


[14:04:41] Running LargestFragmentChooser
[14:04:41] Running Normalizer
[14:04:41] Running Uncharger
[14:04:41] Running LargestFragmentChooser
[14:04:41] Fragment: CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3c([O-])c(NC(=O)c4ccc(O)c(CC=C(C)C)c4)c(=O)oc3c2C)OC1(C)C
[14:04:41] New largest fragment: CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3c([O-])c(NC(=O)c4ccc(O)c(CC=C(C)C)c4)c(=O)oc3c2C)OC1(C)C (79)
[14:04:41] Fragment: [Na+]
[14:04:41] Running Normalizer
[14:04:41] Running Uncharger
[14:04:41] Removed negative charge.
[14:04:41] Running LargestFragmentChooser
[14:04:41] Running Normalizer
[14:04:41] Running Uncharger
[14:04:41] Running LargestFragmentChooser
[14:04:41] Running Normalizer
[14:04:41] Running Uncharger
[14:04:41] Running LargestFragmentChooser
[14:04:41] Running Normalizer
[14:04:41] Running Uncharger
[14:04:41] Running LargestFragmentChooser
[14:04:41] Fragment: C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C
[14:04:41] New largest fragment: C=CC(=O)

[prog] processed 200/235


[14:04:45] Running LargestFragmentChooser
[14:04:45] Fragment: CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)c1ccc2ccccc2n1
[14:04:45] New largest fragment: CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C@@H](O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)c1ccc2ccccc2n1 (99)
[14:04:45] Fragment: CS(=O)(=O)O
[14:04:45] Running Normalizer
[14:04:45] Running Uncharger
[14:04:45] Running LargestFragmentChooser
[14:04:45] Running Normalizer
[14:04:45] Running Uncharger
[14:04:45] Running LargestFragmentChooser
[14:04:45] Fragment: COc1ccc2c(O[C@@H]3C[C@H]4C(=O)N[C@]5(C(=O)[N-]S(=O)(=O)C6CC6)C[C@H]5/C=C\CCCCN(C)C(=O)[C@@H]4C3)cc(-c3nc(C(C)C)cs3)nc2c1C
[14:04:45] New largest fragment: COc1ccc2c(O[C@@H]3C[C@H]4C(=O)N[C@]5(C(=O)[N-]S(=O)(=O)C6CC6)C[C@H]5/C=C\CCCCN(C)C(=O)[C@@H]4C3)cc(-c3nc(C(C)C)cs3)nc2c1C (98)
[14:04:45] Fragment: [Na+]
[14:04:45] Running Normalizer
[14:04:45] Running Uncharger
[14:04:45] Removed negative charge.
[14:04:45] Running 

✅ 親分子へ統合して保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged_consolidated.csv  (rows=148)
   子展開（優先フラグ + ABCB1保持 + Sample_Weight）: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged_children_expanded.csv  (rows=235)


In [ ]:
# === Check presence of specific PubChem CIDs in a CSV column ===
# Target file/column:
CSV_PATH = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged_consolidated.csv"
CID_COL  = "PubChem_CID"

# CIDs to check (as provided)
TARGET_CIDS = [3348, 3955, 2724385, 10315094, 68770, 13342, 441074]

import pandas as pd
import numpy as np

# --- load ---
df = pd.read_csv(CSV_PATH)

# --- safety: column existence ---
if CID_COL not in df.columns:
    raise RuntimeError(f"列 '{CID_COL}' が見つかりません。存在列: {list(df.columns)}")

# --- normalize the PubChem_CID column to a comparable set (both int and str) ---
# 1) 数値化できるものは数値（Int64）に、できないものはNaN
cid_num = pd.to_numeric(df[CID_COL], errors="coerce")
# 2) 数値としての集合（欠損除外）
cid_int_set = set(int(x) for x in cid_num.dropna().astype("int64").tolist())
# 3) 文字列としての集合（前後空白削除・小文字化）
cid_str_set = set(str(x).strip().lower() for x in df[CID_COL].astype(str).tolist())

def present(cid):
    # 数値一致 or 文字列一致（"3348"のようなケース）
    return (cid in cid_int_set) or (str(cid).strip().lower() in cid_str_set)

# --- report ---
print(f"File: {CSV_PATH}")
print(f"Rows: {len(df)}  | Unique {CID_COL} (numeric): {len(cid_int_set)}")

results = []
for cid in TARGET_CIDS:
    is_present = present(cid)
    results.append((cid, is_present))
    print(f"CID {cid}: {'FOUND' if is_present else 'NOT FOUND'}")

# 追加: 見つかった行番号（上位数件のみ表示）
show_examples = True
if show_examples:
    for cid, ok in results:
        if ok:
            mask = (pd.to_numeric(df[CID_COL], errors="coerce")==cid) | \
                   (df[CID_COL].astype(str).str.strip().str.lower()==str(cid).strip().lower())
            idxs = df.index[mask].tolist()
            print(f"  -> indices for CID {cid}: {idxs[:10]}{' ...' if len(idxs)>10 else ''}")

# オプション: サマリーをDataFrame化（必要なら保存してください）
summary = pd.DataFrame(results, columns=["CID", "present"])
# summary.to_csv(CSV_PATH.replace(".csv", "_cid_check_summary.csv"), index=False)

File: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged_consolidated.csv
Rows: 148  | Unique PubChem_CID (numeric): 124
CID 3348: FOUND
CID 3955: FOUND
CID 2724385: FOUND
CID 10315094: FOUND
CID 68770: FOUND
CID 13342: FOUND
CID 441074: FOUND
  -> indices for CID 3348: [46]
  -> indices for CID 3955: [66]
  -> indices for CID 2724385: [35]
  -> indices for CID 10315094: [77]
  -> indices for CID 68770: [105]
  -> indices for CID 13342: [114]
  -> indices for CID 441074: [94]


In [ ]:
# ============================
# KEGG + AID → master 生成スクリプト
#  - 出力の列順はスクリプト内で固定（テンプレ依存なし）
#  - ラベル: KEGG最優先 (ABCB1 Substrate=1) / AIDは弱補完
#  - SampleWeight: KEGGの値 > AIDの値 > 既定（AIDのみ既定=0.7、最終フォールバック=1.0）
#  - AIDのSampleWeight列を引き継ぐ（無い場合はAID行は0.7で補完）
#  - ★ y列: Substrate_kegg があればそれを、無ければ Substrate_aid を転記
# ============================

import sys, subprocess, pandas as pd, numpy as np

# ===== 入出力パス =====
FILE1_KEGG = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/kegg_pubchem_smiles_merged_consolidated.csv"
FILE2_AID  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/AID_1346986_1346987_labels_fold4.csv"
FILE3_OUT  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/integrated_labels_master.csv"

# ===== 固定の出力列順 =====
OUTPUT_COLS = [
    "join_key",
    "CID","SMILES_aid","dlogAC50",
    "PubChem_CID","SMILES_kegg","Drug_name",
    "Substrate_aid","Substrate_kegg",
    "tier","y","weight","from_AID","from_KEGG",
    "SampleWeight"
]

# ===== オプション =====
AID_SAMPLE_WEIGHT_DEFAULT = 0.7
GLOBAL_SAMPLE_WEIGHT_FALLBACK = 1.0
WEAK_WEIGHT = 1.0
AID_NEG_COUNTS_AS_NEG = False  # tier/weight算出にのみ影響（yの定義には影響しません）

# ===== RDKit（あれば使う／無ければフォールバック） =====
HAS_RDKIT = False
try:
    from rdkit import Chem, RDLogger
    HAS_RDKIT = True
except Exception:
    for pkg in ("rdkit", "rdkit-pypi"):
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
            from rdkit import Chem, RDLogger
            HAS_RDKIT = True
            break
        except Exception:
            pass
if HAS_RDKIT:
    RDLogger.DisableLog('rdApp.*')

def safe_str_strip(s):
    if pd.isna(s): return ""
    s = str(s).strip()
    return "" if s.lower() in {"nan","none","null"} else s

def smiles_to_ikey1(smiles: str) -> str:
    if not HAS_RDKIT or not smiles: return ""
    try:
        m = Chem.MolFromSmiles(smiles)
        if m is None: return ""
        ik = Chem.inchi.MolToInchiKey(m)
        return ik.split("-")[0] if ik else ""
    except Exception:
        return ""

def norm_label_aid_weak(x):
    if pd.isna(x): return np.nan
    try:
        v = int(x); return v if v in (0,1) else np.nan
    except Exception:
        return np.nan

def norm_label_kegg_strong(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().lower()
    return 1 if s in {"1","true","yes","y","pos","positive","substrate"} else np.nan

def make_join_key(df, cid_col, smiles_col, ikey_col):
    keys = []
    iks = df[ikey_col] if ikey_col in df.columns else ["" for _ in range(len(df))]
    smis = df[smiles_col] if smiles_col in df.columns else ["" for _ in range(len(df))]
    cids = df[cid_col] if cid_col in df.columns else [np.nan for _ in range(len(df))]
    for ik, smi, cid in zip(iks, smis, cids):
        if isinstance(ik, str) and ik:
            keys.append(f"IK:{ik}")
        elif isinstance(smi, str) and smi:
            keys.append(f"SMI:{smi}")
        else:
            keys.append(f"CID:{cid}" if pd.notna(cid) else "")
    return keys

# ===== 1) 読み込み =====
kegg = pd.read_csv(FILE1_KEGG)
aid  = pd.read_csv(FILE2_AID)

# ===== 2) 正規化（AID） =====
req_aid = ["CID","SMILES","Substrate"]
missing = [c for c in req_aid if c not in aid.columns]
if missing:
    raise ValueError(f"AIDに必須列が不足: {missing}")

aid = aid.copy()
aid["CID"] = pd.to_numeric(aid["CID"], errors="coerce").astype("Int64")
aid["SMILES"] = aid["SMILES"].apply(safe_str_strip)
aid["dlogAC50"] = pd.to_numeric(aid.get("dlogAC50", np.nan), errors="coerce")
aid["Substrate_aid"] = aid["Substrate"].apply(norm_label_aid_weak)

# AID側 SampleWeight
if "SampleWeight" in aid.columns:
    aid["SampleWeight_AID"] = pd.to_numeric(aid["SampleWeight"], errors="coerce")
else:
    aid["SampleWeight_AID"] = np.nan
aid["SampleWeight_AID"] = aid["SampleWeight_AID"].fillna(AID_SAMPLE_WEIGHT_DEFAULT).astype(float).clip(lower=0.0)

# ===== 3) 正規化（KEGG） =====
if "PubChem_CID" not in kegg.columns or "SMILES" not in kegg.columns:
    raise ValueError("KEGGに必須列 PubChem_CID / SMILES がありません")

# ABCB1 Substrate 列の探索
label_col = None
for c in kegg.columns:
    if c.strip().lower() in {"abcb1 substrate","abcb1_substrate","abcb1substrate"}:
        label_col = c; break

kegg = kegg.copy()
kegg["PubChem_CID"] = pd.to_numeric(kegg["PubChem_CID"], errors="coerce").astype("Int64")
kegg["SMILES"] = kegg["SMILES"].apply(safe_str_strip)
kegg["Drug_name"] = kegg["Drug_name"].apply(safe_str_strip) if "Drug_name" in kegg.columns else ""
kegg["Substrate_kegg"] = kegg[label_col].apply(norm_label_kegg_strong) if label_col else np.nan

# KEGGのSampleWeight（あれば尊重、なければ1.0）
sw_in = None
for c in kegg.columns:
    if c.strip().lower() in {"sampleweight","sample_weight"}:
        sw_in = c; break
kegg["SampleWeight"] = (
    pd.to_numeric(kegg[sw_in], errors="coerce").fillna(1.0).astype(float).clip(lower=0.0)
    if sw_in else 1.0
)

# ===== 4) キー生成（IK→SMILES→CID の優先順） =====
aid["ikey1"]  = aid["SMILES"].apply(smiles_to_ikey1) if HAS_RDKIT else ""
kegg["ikey1"] = kegg["SMILES"].apply(smiles_to_ikey1) if HAS_RDKIT else ""

aid["join_key"]  = make_join_key(aid,  "CID",         "SMILES", "ikey1")
kegg["join_key"] = make_join_key(kegg, "PubChem_CID", "SMILES", "ikey1")

aid  = aid[aid["join_key"]!=""].copy()
kegg = kegg[kegg["join_key"]!=""].copy()

# 重複排除
aid_u  = aid.sort_values("join_key").drop_duplicates(subset=["join_key"], keep="first")
kegg_u = kegg.sort_values("join_key").drop_duplicates(subset=["join_key"], keep="first")

# ===== 5) マージ（outer） =====
master = pd.merge(
    aid_u[["join_key","CID","SMILES","dlogAC50","Substrate_aid","SampleWeight_AID"]],
    kegg_u[["join_key","PubChem_CID","SMILES","Drug_name","Substrate_kegg","SampleWeight"]],
    on="join_key", how="outer", suffixes=("_aid","_kegg")
)

# ===== 6) tier（参考情報） =====
def decide_tier(row):
    s_kegg = row["Substrate_kegg"]  # 1 or NaN
    s_aid  = row["Substrate_aid"]   # 0/1/NaN
    if pd.notna(s_kegg) and int(s_kegg)==1:
        return "kegg_pos"
    if pd.notna(s_aid) and int(s_aid)==1:
        return "aid_weak_pos"
    if pd.notna(s_aid) and int(s_aid)==0:
        return "aid_weak_neg"
    return "unknown"

master["tier"] = master.apply(decide_tier, axis=1)

# ===== ★ 7) y の定義（ご指定仕様） =====
# Substrate_kegg に値があればそれを、欠損なら Substrate_aid を使う（両方欠損なら y も欠損）
y_raw = master["Substrate_kegg"].where(master["Substrate_kegg"].notna(), master["Substrate_aid"])
master["y"] = pd.to_numeric(y_raw, errors="coerce").astype("Int64")

# ===== 8) weight/tags（tierベース：y定義とは独立） =====
def tier_to_weight(t):
    if t == "kegg_pos": return 1.0
    if t.startswith("aid_weak"): return WEAK_WEIGHT
    return np.nan

master["weight"] = master["tier"].map(tier_to_weight)
master["from_AID"]  = master["Substrate_aid"].notna().astype(int)
master["from_KEGG"] = master["Substrate_kegg"].notna().astype(int)

# ===== 9) SampleWeight の最終確定 =====
#   優先度: KEGG > AID > 最終フォールバック
master["SampleWeight"] = pd.to_numeric(master.get("SampleWeight", np.nan), errors="coerce")
master["SampleWeight"] = np.where(
    master["SampleWeight"].notna(), master["SampleWeight"], master.get("SampleWeight_AID", np.nan)
)
master["SampleWeight"] = master["SampleWeight"].fillna(GLOBAL_SAMPLE_WEIGHT_FALLBACK).astype(float).clip(lower=0.0)

# ===== 10) 出力列の固定 =====
for c in OUTPUT_COLS:
    if c not in master.columns:
        master[c] = np.nan

out = master[OUTPUT_COLS].copy()
out.to_csv(FILE3_OUT, index=False)

print("✅ 統合完了 → 保存:", FILE3_OUT)
print("行数/列数:", out.shape)
print("tier内訳:\n", out["tier"].value_counts(dropna=False))
print("y集計:\n", out["y"].value_counts(dropna=False))


✅ 統合完了 → 保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/integrated_labels_master.csv
行数/列数: (7397, 15)
tier内訳:
 tier
aid_weak_neg    7097
aid_weak_pos     212
kegg_pos          57
unknown           31
Name: count, dtype: int64
y集計:
 y
0       7097
1        269
<NA>      31
Name: count, dtype: Int64


In [ ]:
# ==========================================
# Murckoスキャフォールド分割（外部20%）＋近縁遮断
# かつ 外部に strong_pos=15, weak_pos=5 を確保（可能な限り厳守）
# 優先薬物は必ず学習へ固定
# 入力 : integrated_labels_master.csv
# 出力 :
#   train_split.csv
#   external10_union.csv        ← 既存名を踏襲（中身は20%）
#   external10_kegg_only.csv
#   split_summary.txt
#   （★ すべての出力に SampleWeight を残す）
# ==========================================

import sys, subprocess, pandas as pd, numpy as np, math, io, re

# RDKit
try:
    from rdkit import Chem
    from rdkit.Chem.Scaffolds import MurckoScaffold
    from rdkit.Chem import AllChem, DataStructs
    from rdkit import RDLogger
    RDLogger.DisableLog('rdApp.*')
    print("✅ RDKit available")
except Exception as e:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rdkit"])
    from rdkit import Chem
    from rdkit.Chem.Scaffolds import MurckoScaffold
    from rdkit.Chem import AllChem, DataStructs
    from rdkit import RDLogger
    RDLogger.DisableLog('rdApp.*')
    print("✅ RDKit installed")

ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
MASTER = f"{ROOT}/integrated_labels_master.csv"

OUT_TRAIN   = f"{ROOT}/train_split.csv"
OUT_EXT_U   = f"{ROOT}/external10_union.csv"       # ファイル名は既存踏襲
OUT_EXT_K   = f"{ROOT}/external10_kegg_only.csv"
OUT_REPORT  = f"{ROOT}/split_summary.txt"

# ---- パラメータ ----
SEED = 42
TARGET_EXT_RATIO = 0.20
TARGET_STRONG_POS = 15
TARGET_WEAK_POS   = 5
MIN_SCAF_SIZE = 3
SIM_THRESHOLD = 0.70
FP_RADIUS = 2
FP_BITS = 2048
MAX_REPAIR_ITERS = 120

PRIORITY_PAIRS = [
    {"name": "Digoxin",              "cid": 2724385},
    {"name": "Loperamide",           "cid": 3955},
    {"name": "Fexofenadine",         "cid": 3348},
    {"name": "N-Methyl-Quinidine",   "cid": 10315094},
    {"name": "Talinolol",            "cid": 68770},
    {"name": "Vinblastine",          "cid": 13342},
    {"name": "Quinidine",            "cid": 441074},
]

rng = np.random.default_rng(SEED)

# ---- 列名ゆれ対応（CID/Drug名） ----
def find_col(df, candidates):
    low = {str(c).strip().lower(): c for c in df.columns}
    for nm in candidates:
        c = low.get(nm.strip().lower())
        if c: return c
    return None

def pick_smiles(row):
    sa = str(row.get("SMILES_aid")) if pd.notna(row.get("SMILES_aid")) else ""
    sk = str(row.get("SMILES_kegg")) if pd.notna(row.get("SMILES_kegg")) else ""
    sa = "" if sa.lower() in {"nan","none",""} else sa
    sk = "" if sk.lower() in {"nan","none",""} else sk
    return sa if sa else sk

def norm_name(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", " ", s)
    return s.lower()

PRIORITY_NAME_SET = {norm_name(d["name"]) for d in PRIORITY_PAIRS}
PRIORITY_CID_SET  = {int(d["cid"]) for d in PRIORITY_PAIRS if d.get("cid") is not None}

def to_mol(smi):
    if not isinstance(smi, str) or smi=="":
        return None
    try:
        return Chem.MolFromSmiles(smi)
    except Exception:
        return None

def to_scaffold_smiles(smi):
    m = to_mol(smi)
    if m is None:
        return ""
    try:
        scaf = MurckoScaffold.GetScaffoldForMol(m)
        return Chem.MolToSmiles(scaf) if scaf is not None else ""
    except Exception:
        return ""

def morgan_fp(smi):
    m = to_mol(smi)
    if m is None:
        return None
    try:
        return AllChem.GetMorganFingerprintAsBitVect(m, FP_RADIUS, nBits=FP_BITS)
    except Exception:
        return None

def max_tanimoto(fp, fps):
    if fp is None or not fps:
        return 0.0
    sims = DataStructs.BulkTanimotoSimilarity(fp, fps)
    return max(sims) if sims else 0.0

# ====== 0) 読み込み & 前処理 ======
df = pd.read_csv(MASTER)

# ★ SampleWeight の正規化（無ければ 1.0 を付与）
sw_col = find_col(df, ["SampleWeight","Sample_Weight"])
if sw_col is None:
    df["SampleWeight"] = 1.0
else:
    df["SampleWeight"] = pd.to_numeric(df[sw_col], errors="coerce").fillna(1.0).astype(float).clip(lower=0.0)

# 統一SMILES列を付与
df["SMILES"] = df.apply(pick_smiles, axis=1)
df = df[(df["SMILES"].notna()) & (df["SMILES"]!="")].copy()

# tier 必須
if "tier" not in df.columns:
    raise RuntimeError("MASTERに 'tier' 列が必要です（strong_pos/strong_neg/weak_pos/unknown）。")

# CID/Drug名
col_cid  = find_col(df, ["PubChem_CID","CID","Chosen_CID"])
col_name = find_col(df, ["Drug_name","Drug","Drug Name"])

if col_name:
    df["_name_norm"] = df[col_name].astype(str).map(norm_name)
else:
    df["_name_norm"] = ""

if col_cid:
    def _to_int_or_nan(x):
        try:
            return int(x)
        except Exception:
            return np.nan
    df["_cid_int"] = df[col_cid].apply(_to_int_or_nan)
else:
    df["_cid_int"] = np.nan

# 優先フラグ
df["_priority_flag"] = (
    df["_cid_int"].isin(PRIORITY_CID_SET) |
    df["_name_norm"].isin(PRIORITY_NAME_SET)
)

# ====== 1) スキャフォールド計算 ======
df["scaffold"] = df["SMILES"].apply(to_scaffold_smiles)
mask_empty_scaf = (df["scaffold"]=="")
df.loc[mask_empty_scaf, "scaffold"] = df.loc[mask_empty_scaf, "SMILES"].apply(lambda s: f"NOSCAF::{s[:24]}")
protected_scaffolds = set(df.loc[df["_priority_flag"], "scaffold"])

scaf_counts = df["scaffold"].value_counts()
candidates = [
    s for s,n in scaf_counts.items()
    if (n >= MIN_SCAF_SIZE) and (s not in protected_scaffolds)
]

# ====== 2) スキャフォールドごとのラベル集計 ======
def tier_counts_for_scaffold(s):
    g = df[df["scaffold"]==s]
    sp = int((g["tier"]=="strong_pos").sum())
    sn = int((g["tier"]=="strong_neg").sum())
    wp = int((g["tier"]=="weak_pos").sum())
    un = int((g["tier"]=="unknown").sum())
    total = len(g)
    return {"strong_pos":sp, "strong_neg":sn, "weak_pos":wp, "unknown":un, "total":total}

scaf_stats = {s: tier_counts_for_scaffold(s) for s in candidates}
target_total = int(round(len(df) * TARGET_EXT_RATIO))

# ====== 3) 初期選択 ======
picked = set()
need_sp = TARGET_STRONG_POS
need_wp = TARGET_WEAK_POS

for s in rng.permutation(candidates):
    if s in picked: continue
    if need_sp <= 0: break
    if scaf_stats[s]["strong_pos"] > 0:
        picked.add(s)
        need_sp -= scaf_stats[s]["strong_pos"]

for s in rng.permutation([x for x in candidates if x not in picked]):
    if need_wp <= 0: break
    if scaf_stats[s]["weak_pos"] > 0:
        picked.add(s)
        need_wp -= scaf_stats[s]["weak_pos"]

def current_tot(scafs):
    return sum(scaf_stats[s]["total"] for s in scafs)

while current_tot(picked) < target_total:
    remaining = [x for x in candidates if x not in picked]
    if not remaining: break
    best, best_gap = None, float("inf")
    for s in rng.permutation(remaining):
        new_tot = current_tot(picked | {s})
        gap = abs(target_total - new_tot)
        if gap < best_gap:
            best_gap = gap; best = s
    if best is None: break
    picked.add(best)

# ====== 4) 一旦 ext/train ======
ext_mask = df["scaffold"].isin(picked)
ext = df[ext_mask].copy().reset_index(drop=True)
train = df[~ext_mask].copy().reset_index(drop=True)

# 優先薬物は必ずtrainへ
move_priority = ext[ext["_priority_flag"]].copy()
if len(move_priority) > 0:
    ext = ext[~ext.index.isin(move_priority.index)].copy()
    train = pd.concat([train, move_priority], ignore_index=True)

# ====== 5) 近縁遮断 ======
train_fps = [morgan_fp(s) for s in train["SMILES"]]
train_fps = [fp for fp in train_fps if fp is not None]

def too_close(smi):
    fp = morgan_fp(smi)
    if fp is None or len(train_fps)==0:
        return False
    return max_tanimoto(fp, train_fps) >= SIM_THRESHOLD

ext["too_close"] = ext["SMILES"].apply(too_close)
move_nn = ext[ext["too_close"]].copy()
ext  = ext[~ext["too_close"]].copy()
train = pd.concat([train, move_nn.drop(columns=["too_close"])], ignore_index=True)

# ====== 6) 修復ループ ======
def count_labels(df_part):
    sp = int((df_part["tier"]=="strong_pos").sum())
    wp = int((df_part["tier"]=="weak_pos").sum())
    tot = len(df_part)
    return sp, wp, tot

def scaffold_of_rows(df_part):
    return set(df_part["scaffold"].unique().tolist())

def pick_scaffolds_to_add(have_scafs, need_sp_left, need_wp_left, max_add=3):
    pool = [s for s in candidates if s not in have_scafs]
    rng.shuffle(pool)
    chosen = []
    for s in pool:
        if len(chosen) >= max_add: break
        add_sp = scaf_stats[s]["strong_pos"]
        add_wp = scaf_stats[s]["weak_pos"]
        if (need_sp_left > 0 and add_sp > 0) or (need_wp_left > 0 and add_wp > 0) or (need_sp_left<=0 and need_wp_left<=0):
            chosen.append(s)
    return chosen

def drop_scaffolds_to_reduce(have_scafs, target_size):
    cur_size = sum(scaf_stats[s]["total"] for s in have_scafs)
    if cur_size <= target_size: return []
    order = sorted(list(have_scafs), key=lambda s: (scaf_stats[s]["strong_pos"], scaf_stats[s]["weak_pos"], scaf_stats[s]["total"]))
    dropping, size_now = [], cur_size
    for s in order:
        if size_now <= target_size: break
        dropping.append(s); size_now -= scaf_stats[s]["total"]
    return dropping

ext_scafs = scaffold_of_rows(ext)
for it in range(MAX_REPAIR_ITERS):
    sp, wp, tot = count_labels(ext)
    need_sp = max(0, TARGET_STRONG_POS - sp)
    need_wp = max(0, TARGET_WEAK_POS   - wp)
    if (sp >= TARGET_STRONG_POS) and (wp >= TARGET_WEAK_POS) and (tot == target_total):
        break
    changed = False
    if need_sp > 0 or need_wp > 0:
        to_add = pick_scaffolds_to_add(ext_scafs, need_sp, need_wp, max_add=3)
        if to_add:
            ext_scafs |= set(to_add)
            ext = df[df["scaffold"].isin(ext_scafs)].copy().reset_index(drop=True)
            if ext["_priority_flag"].any():
                move_back = ext[ext["_priority_flag"]].copy()
                ext = ext[~ext["_priority_flag"]].copy()
                train = pd.concat([train, move_back], ignore_index=True)
            changed = True
    sp, wp, tot = count_labels(ext)
    size_gap = target_total - tot
    if size_gap < 0:
        drops = drop_scaffolds_to_reduce(ext_scafs, target_total)
        if drops:
            ext_scafs -= set(drops)
            ext = df[df["scaffold"].isin(ext_scafs)].copy().reset_index(drop=True)
            changed = True
    elif size_gap > 0:
        add_pool = [s for s in candidates if s not in ext_scafs]
        add_pool = sorted(add_pool, key=lambda s: (scaf_stats[s]["strong_pos"], scaf_stats[s]["weak_pos"], scaf_stats[s]["total"]))
        for s in add_pool:
            if sum(scaf_stats[x]["total"] for x in ext_scafs) >= target_total:
                break
            ext_scafs.add(s); changed = True
        if changed:
            ext = df[df["scaffold"].isin(ext_scafs)].copy().reset_index(drop=True)
    if not changed: break

if ext["_priority_flag"].any():
    move_priority2 = ext[ext["_priority_flag"]].copy()
    ext = ext[~ext["_priority_flag"]].copy()
    train = pd.concat([train, move_priority2], ignore_index=True)

# ====== 7) 出力 ======
keep_cols = [
    "join_key","SMILES",
    "CID","SMILES_aid","PubChem_CID","SMILES_kegg","Drug_name",
    "tier","y","weight","from_AID","from_KEGG",
    "Substrate_strong","Substrate_weak","dlogAC50","scaffold",
    "SampleWeight"  # ★ 追加：重みを出力に残す
]
keep_cols = [c for c in keep_cols if c in df.columns] + ["SMILES","scaffold"]

train_out = train.drop(columns=[c for c in train.columns if c not in keep_cols]).copy()
ext_out   = ext.drop(columns=[c for c in ext.columns if c not in keep_cols]).copy()

# KEGG-only（weak_posのみ）を外部から抽出（PU評価用）
ext_kegg_only = ext_out[(ext_out["tier"]=="weak_pos")].copy() if "tier" in ext_out.columns else ext_out.iloc[0:0].copy()

train_out.to_csv(OUT_TRAIN, index=False)
ext_out.to_csv(OUT_EXT_U, index=False)
ext_kegg_only.to_csv(OUT_EXT_K, index=False)

# ====== 8) レポート ======
def _count_priority(df_part):
    coln = find_col(df_part, ["Drug_name","Drug","Drug Name"])
    colc = find_col(df_part, ["PubChem_CID","CID","Chosen_CID"])
    name_norm = df_part[coln].astype(str).map(norm_name) if coln else pd.Series([""]*len(df_part))
    cid_int = pd.to_numeric(df_part[colc], errors="coerce").astype("Int64") if colc else pd.Series([pd.NA]*len(df_part), dtype="Int64")
    return int(((cid_int.isin(list(PRIORITY_CID_SET))) | (name_norm.isin(list(PRIORITY_NAME_SET)))).sum())

def _lbl_counts(df_part):
    return df_part["tier"].value_counts(dropna=False).to_dict() if "tier" in df_part.columns else {}

sp_e, wp_e, tot_e = (ext_out["tier"]=="strong_pos").sum(), (ext_out["tier"]=="weak_pos").sum(), len(ext_out)

buf = io.StringIO()
def wln(s=""): buf.write(str(s)+"\n")
wln("=== Split Summary (20% target with label constraints) ===")
wln(f"All (with usable SMILES)      : {len(df):,}")
wln(f"Target external total (20%)   : {target_total:,}")
wln(f"Protected scaffolds (priority): {len(protected_scaffolds):,}")
wln(f"Moved ext→train by NN filter  : {len(move_nn):,} (Tanimoto≥{SIM_THRESHOLD})")
wln("")
wln(f"[External final] size={tot_e:,} | strong_pos={int(sp_e)} (target={TARGET_STRONG_POS}) | weak_pos={int(wp_e)} (target={TARGET_WEAK_POS})")
wln(f"  priority in ext = {_count_priority(ext_out)}")
wln(f"[Train   final] size={len(train_out):,}")
wln(f"  priority in train = {_count_priority(train_out)}")
wln("")
if "tier" in ext_out.columns:
    wln("[External tier counts]"); wln(ext_out["tier"].value_counts(dropna=False).to_string()); wln("")
if "tier" in train_out.columns:
    wln("[Train tier counts]"); wln(train_out["tier"].value_counts(dropna=False).to_string()); wln("")
wln(f"KEGG-only in External (weak_pos): {len(ext_kegg_only):,}")

if sp_e != TARGET_STRONG_POS or wp_e != TARGET_WEAK_POS or tot_e != target_total:
    wln("\n[NOTE] Exact constraints not fully met. This can happen if dataset composition makes it impossible.")
    wln("      The script selected the closest feasible split under scaffold grouping & NN filtering.")

with open(OUT_REPORT, "w") as f:
    f.write(buf.getvalue())

print(buf.getvalue())
print("保存:", OUT_TRAIN)
print("保存:", OUT_EXT_U)
print("保存:", OUT_EXT_K)
print("保存:", OUT_REPORT)

✅ RDKit available
=== Split Summary (20% target with label constraints) ===
All (with usable SMILES)      : 7,397
Target external total (20%)   : 1,479
Protected scaffolds (priority): 7
Moved ext→train by NN filter  : 163 (Tanimoto≥0.7)

[External final] size=1,478 | strong_pos=0 (target=15) | weak_pos=0 (target=5)
  priority in ext = 0
[Train   final] size=6,081
  priority in train = 7

[External tier counts]
tier
aid_weak_neg    1453
aid_weak_pos      16
kegg_pos           6
unknown            3

[Train tier counts]
tier
aid_weak_neg    5799
aid_weak_pos     201
kegg_pos          53
unknown           28

KEGG-only in External (weak_pos): 0

[NOTE] Exact constraints not fully met. This can happen if dataset composition makes it impossible.
      The script selected the closest feasible split under scaffold grouping & NN filtering.

保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/train_split.csv
保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/external10_union.csv
保存: /content/drive/MyDri

In [ ]:
# ===============================
# PyGデータ生成（d.x + d.g + d.r + cid）ver7対応
# 入力: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/train_split.csv
# - 必須列: SMILES, tier, y(0/1), weight
# - CIDは row ごとに「CID を最優先、欠損なら PubChem_CID」を使用
# 仕様:
# - unknownを学習から除外
# - y==1（陽性）のみSMILESランダム化で拡張（5倍）
# - RDKit記述子 (gfeat7, rdesc10) を標準化して d.g, d.r に格納
# - サンプル重みを d.w に保存
# - CIDを d.cid として保存（欠損は -1）
# 出力: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt
# ===============================
# ===============================
# PyGデータ生成（d.x + d.g + d.r + cid）ver7対応 + SampleWeight保持
# ===============================
import torch, pandas as pd, numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdchem
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data

ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PATH = f"{ROOT}/train_split.csv"
SAVE_PATH = f"{ROOT}/data_graph_with_smiles.pt"
INDEX_CSV = f"{ROOT}/data_graph_with_smiles_index.csv"  # ★ 追加: インデックス出力（列: SampleWeight 含む）

# --- 分子特徴量（d.g, d.r） ---
def gfeat7(mol):
    return [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.TPSA(mol),
        Descriptors.HeavyAtomCount(mol),
        Descriptors.RingCount(mol),
    ]

def rdesc10(mol):
    return [
        Descriptors.FpDensityMorgan1(mol),
        Descriptors.FpDensityMorgan2(mol),
        Descriptors.FpDensityMorgan3(mol),
        Descriptors.NumAliphaticRings(mol),
        Descriptors.NumAromaticRings(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.NumValenceElectrons(mol),
        Descriptors.BalabanJ(mol),
        Descriptors.BertzCT(mol),
        Descriptors.FractionCSP3(mol),
    ]

# --- 原子・結合特徴量 ---
def atom_f(atom: rdchem.Atom):
    return torch.tensor([
        atom.GetAtomicNum(),
        atom.GetTotalDegree(),
        atom.GetFormalCharge(),
        float(atom.GetIsAromatic()),
        atom.GetTotalNumHs(includeNeighbors=True),
    ], dtype=torch.float)

def bond_f(bond: rdchem.Bond):
    return torch.tensor([
        bond.GetBondTypeAsDouble(),
        float(bond.GetIsConjugated()),
        float(bond.IsInRing()),
        int(bond.GetStereo()),
        bond.GetBeginAtom().GetAtomicNum(),
        bond.GetEndAtom().GetAtomicNum(),
    ], dtype=torch.float)

# --- 1) 読み込み＆CID列の優先ルール適用（CID -> PubChem_CID） ---
df0 = pd.read_csv(DATA_PATH)

# 必須列チェック（CID列は後で確認）
need_cols = {"SMILES","tier","y","weight"}
missing = need_cols - set(df0.columns)
if missing:
    raise ValueError(f"必須列が不足しています: {missing}")

# ★ SampleWeight 列を正規化（無ければ 1.0 で追加）
sw_col = None
for c in df0.columns:
    if str(c).strip().lower() in {"sampleweight","sample_weight"}:
        sw_col = c; break
if sw_col is None:
    df0["SampleWeight"] = 1.0
else:
    df0["SampleWeight"] = pd.to_numeric(df0[sw_col], errors="coerce").fillna(1.0).astype(float).clip(lower=0.0)

has_cid = "CID" in df0.columns
has_pub = "PubChem_CID" in df0.columns
if not (has_cid or has_pub):
    raise ValueError("CID 列が見つかりません。少なくとも 'CID' か 'PubChem_CID' のどちらかを含めてください。")

print("✅ CID優先順: CID → PubChem_CID")
print(f"  - CID 列あり? {has_cid}")
print(f"  - PubChem_CID 列あり? {has_pub}")

def choose_cid_row(row):
    v1 = pd.to_numeric(row["CID"], errors="coerce") if has_cid else np.nan
    if pd.notna(v1):
        return int(v1)
    v2 = pd.to_numeric(row["PubChem_CID"], errors="coerce") if has_pub else np.nan
    if pd.notna(v2):
        return int(v2)
    return -1  # いずれも欠損

# unknownを除外、y∈{0,1}のみ
df0 = df0[(df0["tier"]!="unknown") & df0["y"].isin([0,1])].copy()
df0["y"] = df0["y"].astype(int)
df0["weight"] = pd.to_numeric(df0["weight"], errors="coerce").fillna(1.0)

# 優先順でCID決定
df0["_cid_int"] = df0.apply(choose_cid_row, axis=1)

# 統計表示（情報源の内訳）
def cid_source(row):
    v1 = pd.to_numeric(row["CID"], errors="coerce") if has_cid else np.nan
    if pd.notna(v1): return "CID"
    v2 = pd.to_numeric(row["PubChem_CID"], errors="coerce") if has_pub else np.nan
    if pd.notna(v2): return "PubChem_CID"
    return "none"

src_counts = df0.apply(cid_source, axis=1).value_counts()
print("🔎 CID ソース別件数:\n", src_counts.to_string())
print(f"🔎 CID欠損(-1)件数: {(df0['_cid_int'] == -1).sum()}")

# --- 2) 陽性のみSMILES拡張（CID・Weight・SampleWeight も複製） ---
def aug_smiles(sm, k=5):
    m = Chem.MolFromSmiles(sm)
    if m is None:
        return []
    return [Chem.MolToSmiles(m, doRandom=True) for _ in range(k)]

rows = []
for sm, y, w_cls, sw, cidv in zip(df0["SMILES"], df0["y"], df0["weight"], df0["SampleWeight"], df0["_cid_int"]):
    sm = "" if pd.isna(sm) else str(sm)
    if not sm:
        continue
    cidv_int = int(cidv)  # -1 or int
    if y == 1:
        aug = aug_smiles(sm, k=5)
        for s in [sm] + aug:
            rows.append((s, y, w_cls, float(sw), cidv_int))
    else:
        rows.append((sm, y, w_cls, float(sw), cidv_int))

df = pd.DataFrame(rows, columns=["SMILES","Label","Weight","SampleWeight","CID"])
print(f"✅ Augmented size: {len(df)} (original {len(df0)})")
print(f"🔎 Aug後 CID欠損(-1)件数: {(df['CID'] == -1).sum()}")

# --- 3) RDKit変換＆記述子（無効SMILESは除外） ---
mols, keep, g7_raw, r10_raw = [], [], [], []
for i, (sm, _, _, _, _) in enumerate(df.itertuples(index=False)):
    m = Chem.MolFromSmiles(sm)
    if m is None or m.GetNumAtoms() == 0:
        continue
    mols.append(m); keep.append(i)
    g7_raw.append(gfeat7(m))
    r10_raw.append(rdesc10(m))
df = df.iloc[keep].reset_index(drop=True)

# --- 4) 標準化 ---
scaler_g = StandardScaler().fit(g7_raw)
scaler_r = StandardScaler().fit(r10_raw)
g7_scaled = scaler_g.transform(g7_raw)
r10_scaled = scaler_r.transform(r10_raw)

# --- 5) グラフ作成（bondからedgeを作り、双方向に展開） ---
data_list = []
for row_idx, m in enumerate(mols):
    y  = int(df.loc[row_idx, "Label"])
    w  = float(df.loc[row_idx, "Weight"])        # 既存の学習クラス重み（弱ラベルなど）
    sw = float(df.loc[row_idx, "SampleWeight"])  # ★ 新規: サンプル重要度重み（FDA=5.0 など）
    sm = df.loc[row_idx, "SMILES"]
    cid_val = int(df.loc[row_idx, "CID"])  # -1 or int

    # エッジ無しは今回はスキップ
    if m.GetNumBonds() == 0:
        continue

    # ノード
    x = torch.stack([atom_f(a) for a in m.GetAtoms()])  # [N,5]

    # エッジ（双方向）
    src, dst, eattr = [], [], []
    for b in m.GetBonds():
        i = b.GetBeginAtomIdx(); j = b.GetEndAtomIdx()
        f = bond_f(b)
        src += [i, j]
        dst += [j, i]
        eattr += [f, f]
    edge_index = torch.tensor([src, dst], dtype=torch.long)      # [2, 2E]
    edge_attr  = torch.stack(eattr)                              # [2E, 6]

    # PyG Data
    d = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        y=torch.tensor([y], dtype=torch.float32)
    )
    d.g = torch.tensor(g7_scaled[row_idx], dtype=torch.float32).unsqueeze(0)  # [1,7]
    d.r = torch.tensor(r10_scaled[row_idx], dtype=torch.float32).unsqueeze(0) # [1,10]
    d.w = torch.tensor([w], dtype=torch.float32)                 # 既存: 学習用重み
    d.sample_weight = torch.tensor([sw], dtype=torch.float32)    # ★ 追加: FDAなどのサンプル重み
    d.smiles = sm
    d.cid = int(cid_val)  # 欠損は -1

    data_list.append(d)

print(f"✅ グラフ数: {len(data_list)}")
lbl = [int(d.y.item()) for d in data_list]
print(f"🔍 ラベル分布: {pd.Series(lbl).value_counts().to_dict()}")

# --- 6) 保存（.pt と参照用インデックスCSV） ---
torch.save(data_list, SAVE_PATH)
print(f"✅ PyGデータ保存完了: {SAVE_PATH}")

# ★ 参照用インデックス（SMILES, Label, Weight, SampleWeight, CID）
idx_df = pd.DataFrame({
    "SMILES": [d.smiles for d in data_list],
    "Label":  [int(d.y.item()) for d in data_list],
    "Weight": [float(d.w.item()) for d in data_list],
    "SampleWeight": [float(d.sample_weight.item()) for d in data_list],
    "CID":    [int(getattr(d, "cid", -1)) for d in data_list],
})
idx_df.to_csv(INDEX_CSV, index=False)
print(f"✅ インデックスCSV保存: {INDEX_CSV}")

✅ CID優先順: CID → PubChem_CID
  - CID 列あり? True
  - PubChem_CID 列あり? True
🔎 CID ソース別件数:
 CID            6030
PubChem_CID      21
none              2
🔎 CID欠損(-1)件数: 2
✅ Augmented size: 7323 (original 6053)
🔎 Aug後 CID欠損(-1)件数: 12
✅ グラフ数: 7321
🔍 ラベル分布: {0: 5797, 1: 1524}
✅ PyGデータ保存完了: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt
✅ インデックスCSV保存: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles_index.csv


In [ ]:
# --- スケーラ保存 ---
import joblib, os
ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
os.makedirs(ROOT, exist_ok=True)
joblib.dump(scaler_g, f"{ROOT}/scaler_g.joblib")
joblib.dump(scaler_r, f"{ROOT}/scaler_r.joblib")
print("✅ Saved scalers:", f"{ROOT}/scaler_g.joblib", f"{ROOT}/scaler_r.joblib")

✅ Saved scalers: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/scaler_g.joblib /content/drive/MyDrive/Chemoinfo_MDR1_ver7/scaler_r.joblib


In [ ]:
# === Colab: g(7) + r(10) 特徴量辞書を作成・保存・表示（ワンセル） ===
# 出力先: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/metadata/
# 生成物: feature_dictionary_gr17.csv / feature_dictionary_gr17.md

import os, pandas as pd

# 1) Google Drive をマウント（初回のみ認証が出ます）
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 2) 出力先（必要なら変更）
ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
OUT_DIR = f"{ROOT}/metadata"
os.makedirs(OUT_DIR, exist_ok=True)

# 3) 特徴量定義（g=7, r=10）
rows = [
    # g(7)
    {"feature_id":"g1","group":"g","name":"Molecular Weight","rdkit_func":"Descriptors.MolWt","unit":"g/mol",
     "description":"分子全体の質量。大きいほど表面積や結合数が増えやすい。",
     "interpretation_hint":"一般に高分子量は膜透過性を低下させる傾向（P-gp基質性とは一概に直結しない）。",
     "typical_range":"100–800","notes":"極端に大きい/小さい場合は他特徴との相互作用に注意。"},
    {"feature_id":"g2","group":"g","name":"LogP (octanol/water)","rdkit_func":"Descriptors.MolLogP","unit":"—",
     "description":"疎水性の尺度（分配係数の対数）。",
     "interpretation_hint":"中〜高い疎水性は膜挿入・結合部位相互作用を強める可能性。極端に高いと溶解性低下。",
     "typical_range":"−2–7","notes":"イオン化状態や極性表面積と併せて評価。"},
    {"feature_id":"g3","group":"g","name":"H-bond Donors","rdkit_func":"Descriptors.NumHDonors","unit":"count",
     "description":"水素結合供与基の個数。",
     "interpretation_hint":"ドナーが多いと透過性は下がりやすいが、相互作用部位の形成には有利な場合も。",
     "typical_range":"0–6","notes":"Lipinski規則などの範囲感参照。"},
    {"feature_id":"g4","group":"g","name":"H-bond Acceptors","rdkit_func":"Descriptors.NumHAcceptors","unit":"count",
     "description":"水素結合受容基の個数。",
     "interpretation_hint":"受容基が多いと極性が増し、受動透過は低下しやすい。",
     "typical_range":"0–10","notes":"官能基の種類に依存。"},
    {"feature_id":"g5","group":"g","name":"Topological Polar Surface Area (TPSA)","rdkit_func":"Descriptors.TPSA","unit":"Å²",
     "description":"極性表面積の近似。水素結合可能原子に基づく。",
     "interpretation_hint":"高TPSAは膜透過性の低下と相関しやすい。",
     "typical_range":"0–200","notes":"<60–90 Å²で高い経口吸収が期待されることが多い。"},
    {"feature_id":"g6","group":"g","name":"Heavy Atom Count","rdkit_func":"Descriptors.HeavyAtomCount","unit":"count",
     "description":"水素以外の原子数。",
     "interpretation_hint":"分子のサイズ・複雑さの粗指標。",
     "typical_range":"5–80","notes":"分岐/環構造で同カウントでも形状差が出る。"},
    {"feature_id":"g7","group":"g","name":"Ring Count","rdkit_func":"Descriptors.RingCount","unit":"count",
     "description":"環の個数（芳香族/脂肪族を含む）。",
     "interpretation_hint":"環の増加は剛直性↑・疎水性↑につながりうる。",
     "typical_range":"0–6","notes":"縮合環や多環は立体的な影響が大きい。"},

    # r(10)
    {"feature_id":"r1","group":"r","name":"FpDensity Morgan1","rdkit_func":"Descriptors.FpDensityMorgan1","unit":"—",
     "description":"Morgan半径1のビット密度（構造多様性の尺度）。",
     "interpretation_hint":"部分構造の豊富さ・置換密度の粗指標。",
     "typical_range":"0–1","notes":"分子サイズに依存して変動。"},
    {"feature_id":"r2","group":"r","name":"FpDensity Morgan2","rdkit_func":"Descriptors.FpDensityMorgan2","unit":"—",
     "description":"Morgan半径2のビット密度。",
     "interpretation_hint":"近傍（~2結合）までの多様性。",
     "typical_range":"0–1","notes":"r1との同時解釈が有効。"},
    {"feature_id":"r3","group":"r","name":"FpDensity Morgan3","rdkit_func":"Descriptors.FpDensityMorgan3","unit":"—",
     "description":"Morgan半径3のビット密度。",
     "interpretation_hint":"より広い近傍の多様性（骨格・置換の複合）。",
     "typical_range":"0–1","notes":"新規性評価の補助。"},
    {"feature_id":"r4","group":"r","name":"Num Aliphatic Rings","rdkit_func":"Descriptors.NumAliphaticRings","unit":"count",
     "description":"脂肪族環の個数。",
     "interpretation_hint":"立体的嵩高さや柔軟性に影響。",
     "typical_range":"0–6","notes":"スピロ/架橋で性質が変わる。"},
    {"feature_id":"r5","group":"r","name":"Num Aromatic Rings","rdkit_func":"Descriptors.NumAromaticRings","unit":"count",
     "description":"芳香族環の個数。",
     "interpretation_hint":"π-π相互作用・疎水性に寄与。",
     "typical_range":"0–4","notes":"多環芳香族は代謝性や毒性の懸念も。"},
    {"feature_id":"r6","group":"r","name":"Num Rotatable Bonds","rdkit_func":"Descriptors.NumRotatableBonds","unit":"count",
     "description":"回転可能結合の個数。",
     "interpretation_hint":"自由度↑でエントロピー損失↑・透過性↓の傾向。",
     "typical_range":"0–15","notes":"Veber基準（回転結合≤10）など参照。"},
    {"feature_id":"r7","group":"r","name":"Num Valence Electrons","rdkit_func":"Descriptors.NumValenceElectrons","unit":"count",
     "description":"価電子の総数。",
     "interpretation_hint":"全体の電子供与/受容能の粗指標。",
     "typical_range":"—","notes":"原子組成に依存。"},
    {"feature_id":"r8","group":"r","name":"Balaban J Index","rdkit_func":"Descriptors.BalabanJ","unit":"—",
     "description":"分子グラフの接続性指数（位相的複雑さ）。",
     "interpretation_hint":"骨格の分岐/環構造の複雑さを反映。",
     "typical_range":"~0–10","notes":"スケールは分子により大きく変動。"},
    {"feature_id":"r9","group":"r","name":"Bertz CT","rdkit_func":"Descriptors.BertzCT","unit":"—",
     "description":"分子複雑性指数。",
     "interpretation_hint":"高値は構造多様性・置換の多さを示唆。",
     "typical_range":"~0–4000","notes":"サイズ依存性が強い。"},
    {"feature_id":"r10","group":"r","name":"Fraction Csp3","rdkit_func":"Descriptors.FractionCSP3","unit":"—",
     "description":"sp3炭素の比率（3D性の指標）。",
     "interpretation_hint":"sp3比率↑は3D性↑で溶解性・物性が変化。基質性に影響し得る。",
     "typical_range":"0–1","notes":"芳香族が多いと低下。"},
]

df = pd.DataFrame(rows, columns=[
    "feature_id","group","name","rdkit_func","unit","description","interpretation_hint","typical_range","notes"
])

# 4) 保存
csv_path = f"{OUT_DIR}/feature_dictionary_gr17.csv"
md_path  = f"{OUT_DIR}/feature_dictionary_gr17.md"
df.to_csv(csv_path, index=False)

with open(md_path, "w", encoding="utf-8") as f:
    f.write("| feature_id | group | name | rdkit_func | unit | description | interpretation_hint | typical_range | notes |\n")
    f.write("|---|---|---|---|---|---|---|---|---|\n")
    for _, r in df.iterrows():
        f.write(f"| {r['feature_id']} | {r['group']} | {r['name']} | {r['rdkit_func']} | {r['unit']} | {r['description']} | {r['interpretation_hint']} | {r['typical_range']} | {r['notes']} |\n")

# 5) 表示 & パス確認
from IPython.display import display, Markdown
display(df)
print("Saved CSV:", csv_path)
print("Saved MD :", md_path)


Mounted at /content/drive


,feature_id,group,name,rdkit_func,unit,description,interpretation_hint,typical_range,notes
0,g1,g,Molecular Weight,Descriptors.MolWt,g/mol,分子全体の質量。大きいほど表面積や結合数が増えやすい。,一般に高分子量は膜透過性を低下させる傾向（P-gp基質性とは一概に直結しない）。,100–800,極端に大きい/小さい場合は他特徴との相互作用に注意。
1,g2,g,LogP (octanol/water),Descriptors.MolLogP,—,疎水性の尺度（分配係数の対数）。,中〜高い疎水性は膜挿入・結合部位相互作用を強める可能性。極端に高いと溶解性低下。,−2–7,イオン化状態や極性表面積と併せて評価。
2,g3,g,H-bond Donors,Descriptors.NumHDonors,count,水素結合供与基の個数。,ドナーが多いと透過性は下がりやすいが、相互作用部位の形成には有利な場合も。,0–6,Lipinski規則などの範囲感参照。
3,g4,g,H-bond Acceptors,Descriptors.NumHAcceptors,count,水素結合受容基の個数。,受容基が多いと極性が増し、受動透過は低下しやすい。,0–10,官能基の種類に依存。
4,g5,g,Topological Polar Surface Area (TPSA),Descriptors.TPSA,Å²,極性表面積の近似。水素結合可能原子に基づく。,高TPSAは膜透過性の低下と相関しやすい。,0–200,<60–90 Å²で高い経口吸収が期待されることが多い。
5,g6,g,Heavy Atom Count,Descriptors.HeavyAtomCount,count,水素以外の原子数。,分子のサイズ・複雑さの粗指標。,5–80,分岐/環構造で同カウントでも形状差が出る。
6,g7,g,Ring Count,Descriptors.RingCount,count,環の個数（芳香族/脂肪族を含む）。,環の増加は剛直性↑・疎水性↑につながりうる。,0–6,縮合環や多環は立体的な影響が大きい。
7,r1,r,FpDensity Morgan1,Descriptors.FpDensityMorgan1,—,Morgan半径1のビット密度（構造多様性の尺度）。,部分構造の豊富さ・置換密度の粗指標。,0–1,分子サイズに依存して変動。
8,r2,r,FpDensity Morgan2,Descriptors.FpDensityMorgan2,—,Morgan半径2のビット密度。,近傍（~2結合）までの多様性。,0–1,r1との同時解釈が有効。
9,r3,r,FpDensity Morgan3,Descriptors.FpDensityMorgan3,—,Morgan半径3のビット密度。,より広い近傍の多様性（骨格・置換の複合）。,0–1,新規性評価の補助。


Saved CSV: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/metadata/feature_dictionary_gr17.csv
Saved MD : /content/drive/MyDrive/Chemoinfo_MDR1_ver7/metadata/feature_dictionary_gr17.md


In [ ]:
# =============================================================
# 4モデル比較 (GINE / GATv2 / NNConv / TransformerConv)
# d.x + d.g + d.r を結合し、edge_attr も使用
# 入力: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt
# 出力: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_gnn_dx_dg_dr_cv_edgeall/
# 変更点:
#  - サンプル重み d.w と d.sample_weight を損失に反映
#  - pos_weight（不均衡対策）と併用（reduction='none' + 重み和で正規化）
#  - 優先CIDは常に学習へ固定（検証には出さない）
# =============================================================

import os, random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.serialization import add_safe_globals
from torch_geometric.nn import (
    GINEConv, GATv2Conv, NNConv, TransformerConv, AttentionalAggregation
)
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, f1_score
import pandas as pd

# ====== 設定 ======
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED   = 42
KFOLD  = 3
BATCH  = 64
HID    = 256
LR     = 3e-4
PAT    = 5          # 早期停止 patience
TMAX   = 5          # CosineAnnealing の周期
CLIP   = 5
WEIGHT_DECAY = 1e-4

IN_PYGDATA = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt"
SAVE_BASE  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_gnn_dx_dg_dr_cv_edgeall"
os.makedirs(SAVE_BASE, exist_ok=True)
ARCHES = ["GINE", "GATv2", "NNConv", "Transformer"]
for arch in ARCHES:
    os.makedirs(os.path.join(SAVE_BASE, arch), exist_ok=True)

# ★必ず学習に含めたいCID
PRIORITY_CIDS = {3348, 3955, 2724385, 10315094, 68770, 13342, 441074}

# ====== 再現性 ======
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed(SEED)

# ====== データ読み込み（PyTorch 2.6対策付き） ======
def load_pyg_data(path):
    try:
        try:
            from torch_geometric.data.data import DataEdgeAttr
            add_safe_globals([DataEdgeAttr])
        except Exception as e:
            print(f"[WARN] add_safe_globals(DataEdgeAttr) 失敗: {e}")
        obj = torch.load(path)  # 既定: weights_only=True
        return obj
    except Exception as e:
        print(f"[WARN] 安全ロードに失敗（{e}）。weights_only=False にフォールバック。")
        obj = torch.load(path, weights_only=False)
        return obj

obj = load_pyg_data(IN_PYGDATA)
if isinstance(obj, list):
    data_list = obj
elif isinstance(obj, dict) and "data_list" in obj:
    data_list = obj["data_list"]
else:
    try:
        data_list = list(obj)
    except Exception as e:
        raise RuntimeError(f"未知のデータ形式です: {type(obj)}; list化に失敗: {e}")

if len(data_list) == 0:
    raise RuntimeError("data_list が空です。")

sample = data_list[0]
req = ["x","edge_index","edge_attr","g","r","y"]
missing = [k for k in req if not hasattr(sample, k)]
if missing:
    raise RuntimeError(f"Dataに必要属性がありません: {missing}")

def infer_feat_dim(t):
    if t is None: return 0
    if t.dim() == 1: return t.numel()
    if t.dim() in (2,3): return t.size(-1)
    raise RuntimeError(f"未知のテンソル次元: dim={t.dim()}")

in_dim   = sample.x.size(-1)
edge_dim = sample.edge_attr.size(-1)
g_dim    = infer_feat_dim(sample.g)
r_dim    = infer_feat_dim(sample.r)

# ラベル & CID を抽出
labels = [int(d.y.item()) for d in data_list]
pos = sum(labels); neg = len(labels) - pos
print(f"[INFO] Graphs: {len(data_list)} | x_dim={in_dim} | edge_dim={edge_dim} | g_dim={g_dim} | r_dim={r_dim}")
print(f"[INFO] Label balance: pos={pos} neg={neg} (pos_ratio={pos/len(labels):.3f})")

cids = [getattr(d, "cid", -1) for d in data_list]
has_cid_attr = any(getattr(d, "cid", None) is not None for d in data_list)
if not has_cid_attr:
    print("⚠ 注意: data_list のサンプルに d.cid が見つかりません。優先CIDの固定は実行されません。")
priority_idx = [i for i, c in enumerate(cids) if c in PRIORITY_CIDS] if has_cid_attr else []
priority_idx_set = set(priority_idx)
print(f"[INFO] Priority CIDs present: {len(priority_idx)} / {len(PRIORITY_CIDS)}")
if len(priority_idx) > 0:
    found = sorted({cids[i] for i in priority_idx})
    print(f"[INFO] Found priority CIDs in data: {found}")

# ====== 畳み込みユニット ======
def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hidden = max(64, min(256, in_attr_dim * 8))
    return nn.Sequential(
        nn.Linear(in_attr_dim, hidden), nn.ReLU(),
        nn.Linear(hidden, in_channels * out_channels)
    )

def get_conv(name, c_in, c_out, edge_dim):
    if name == "GINE":
        return GINEConv(nn.Sequential(
            nn.Linear(c_in, c_out), nn.ReLU(), nn.Linear(c_out, c_out)
        ), edge_dim=edge_dim)
    if name == "GATv2":
        return GATv2Conv(c_in, c_out, heads=4, concat=False,
                         edge_dim=edge_dim, fill_value='mean')
    if name == "NNConv":
        edge_nn = make_edge_mlp(edge_dim, c_out, c_in)
        return NNConv(c_in, c_out, edge_nn, aggr='mean')
    if name == "Transformer":
        return TransformerConv(c_in, c_out, heads=4, concat=False,
                               edge_dim=edge_dim, dropout=0.0, beta=False)
    raise ValueError(f"unknown arch: {name}")

# ====== 共通モデル（d.x → GNN → pool → [g,r]結合 → cls） ======
class Net(nn.Module):
    def __init__(self, arch, in_dim, edge_dim, g_dim, r_dim, hid=256, p_drop=0.2):
        super().__init__()
        self.arch = arch
        self.c1 = get_conv(arch, in_dim, hid, edge_dim)
        self.c2 = get_conv(arch, hid, hid, edge_dim)
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(hid, 1))
        self.drop = nn.Dropout(p_drop)
        self.fc   = nn.Linear(hid + g_dim + r_dim, 1)

    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.drop(x)
        x = self.pool(x, d.batch)

        g = d.g; r = d.r
        if g.dim() == 3: g = g.squeeze(1)
        elif g.dim() == 1: g = g.view(d.num_graphs, -1)
        if r.dim() == 3: r = r.squeeze(1)
        elif r.dim() == 1: r = r.view(d.num_graphs, -1)

        z = torch.cat([x.float(), g.float(), r.float()], dim=1)
        return self.fc(z).view(-1)

# ====== 学習関数（d.w・d.sample_weight・pos_weight を反映） ======
def train_fold(net, tr_loader, va_loader, pos_weight, save_path):
    bce = nn.BCEWithLogitsLoss(reduction='none')  # 後で重み付け
    opt = torch.optim.AdamW(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, TMAX)

    # ★ 重みサマリ（任意ログ）
    try:
        _w_list  = torch.cat([getattr(d, "w", torch.tensor([1.0])) for d in tr_loader.dataset]).float().cpu().numpy()
    except Exception:
        _w_list = np.ones(len(tr_loader.dataset), dtype=float)
    try:
        _sw_list = torch.cat([getattr(d, "sample_weight", torch.tensor([1.0])) for d in tr_loader.dataset]).float().cpu().numpy()
    except Exception:
        _sw_list = np.ones(len(tr_loader.dataset), dtype=float)
    print(f"[train] mean(w)={np.mean(_w_list):.2f}, mean(SampleWeight)={np.mean(_sw_list):.2f}")

    best_auc, wait, best_state = 0.0, 0, None
    max_epochs = 200

    for ep in range(1, max_epochs + 1):
        net.train()
        for bt in tr_loader:
            bt = bt.to(DEVICE)
            opt.zero_grad()

            logits = net(bt)
            y = bt.y.float().view(-1)

            # 基本ロス（サンプル毎）
            loss_vec = bce(logits, y)

            # クラス不均衡補正: 正例に pos_weight、負例に 1
            cls_w = torch.where(
                y > 0.5,
                torch.tensor(pos_weight, device=DEVICE, dtype=loss_vec.dtype),
                torch.tensor(1.0,       device=DEVICE, dtype=loss_vec.dtype),
            )

            # ★ サンプル重み： d.w × d.sample_weight（無ければ1）
            w1 = bt.w.view(-1) if hasattr(bt, "w") else torch.ones_like(y)
            w2 = getattr(bt, "sample_weight", torch.ones_like(y)).view(-1)
            samp_w = (w1 * w2).to(loss_vec.dtype)

            # ★ 重み付き平均（分母は重み和）
            loss = (loss_vec * cls_w * samp_w).sum() / (samp_w.sum() + 1e-12)

            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), CLIP)
            opt.step()
        sch.step()

        # 検証（重みは使わず素の確率で評価）
        net.eval(); ys, ps = [], []
        with torch.no_grad():
            for bt in va_loader:
                bt = bt.to(DEVICE)
                ys += bt.y.cpu().tolist()
                ps += torch.sigmoid(net(bt)).cpu().tolist()
        auc = roc_auc_score(ys, ps)

        if auc > best_auc:
            best_auc, wait = auc, 0
            best_state = net.state_dict()
            torch.save(best_state, save_path)
        else:
            wait += 1
            if wait >= PAT:
                break

    if best_state is not None:
        net.load_state_dict(best_state)
    return net

# ====== K-Fold 学習・評価 ======
skf = StratifiedKFold(n_splits=KFOLD, shuffle=True, random_state=SEED)
all_summary_rows = []

def enforce_priority_in_train(tr_idx, va_idx):
    """検証側に混入した priority を学習へ移し、可能なら同ラベルの非優先を学習→検証へスワップ"""
    if len(priority_idx) == 0:
        return tr_idx, va_idx, 0, 0
    tr_set, va_set = set(tr_idx.tolist()), set(va_idx.tolist())
    pri_in_val = [i for i in va_idx.tolist() if i in priority_idx_set]
    moved_to_train = 0
    swapped = 0
    if len(pri_in_val) == 0:
        return tr_idx, va_idx, moved_to_train, swapped

    tr_list = tr_idx.tolist()
    va_list = va_idx.tolist()

    for i_pri in pri_in_val:
        # まず validation から外す
        if i_pri in va_list:
            va_list.remove(i_pri)
        # 学習へ入れる
        if i_pri not in tr_list:
            tr_list.append(i_pri)
            moved_to_train += 1

        # サイズとバランスを軽く維持：同ラベルの非優先をスワップ（任意）
        y_pri = labels[i_pri]
        candidate = None
        for j in tr_list:
            if j in priority_idx_set:
                continue
            if labels[j] == y_pri and j not in va_list:
                candidate = j
                break
        if candidate is not None:
            tr_list.remove(candidate)
            va_list.append(candidate)
            swapped += 1

    return np.array(sorted(set(tr_list))), np.array(sorted(set(va_list))), moved_to_train, swapped

for arch in ARCHES:
    print(f"\n================ {arch} (d.x + d.g + d.r + edge_attr) ================")
    fold_rows = []
    arch_dir = os.path.join(SAVE_BASE, arch)

    for f, (tr_idx, va_idx) in enumerate(skf.split(np.zeros(len(labels)), labels)):
        # ★ 検証foldからpriorityを除外して学習foldへ
        tr_idx, va_idx, moved_cnt, swap_cnt = enforce_priority_in_train(tr_idx, va_idx)
        if moved_cnt > 0:
            print(f"[Fold {f}] moved priority from val→train: {moved_cnt}  | swapped (train→val): {swap_cnt}")

        tr_ds = [data_list[i] for i in tr_idx]
        va_ds = [data_list[i] for i in va_idx]

        # pos_weight は foldごとに算出（過大にならないよう上限）
        ytr = torch.cat([d.y.view(-1) for d in tr_ds]).float()
        neg = (ytr == 0).sum().item(); posc = (ytr == 1).sum().item()
        w = min((neg / max(posc, 1.0)), 10.0)

        tr_loader = DataLoader(tr_ds, batch_size=BATCH, shuffle=True,
                               generator=torch.Generator().manual_seed(SEED))
        va_loader = DataLoader(va_ds, batch_size=BATCH, shuffle=False)

        net = Net(arch, in_dim, edge_dim, g_dim, r_dim, hid=HID).to(DEVICE)
        save_model_path = os.path.join(arch_dir, f"fold{f}.pt")
        net = train_fold(net, tr_loader, va_loader, pos_weight=w, save_path=save_model_path)

        # fold内検証スコア
        net.eval(); ys, ps = [], []
        with torch.no_grad():
            for bt in va_loader:
                bt = bt.to(DEVICE)
                ys += bt.y.cpu().tolist()
                ps += torch.sigmoid(net(bt)).cpu().tolist()

        y_pred = [int(p >= 0.5) for p in ps]
        metrics = {
            "Fold": f,
            "ROC-AUC": roc_auc_score(ys, ps),
            "PR-AUC": average_precision_score(ys, ps),
            "Precision": precision_score(ys, y_pred, zero_division=0),
            "Recall": recall_score(ys, y_pred, zero_division=0),
            "F1-Score": f1_score(ys, y_pred, zero_division=0),
            "Val_Size": len(va_idx),
            "Train_Size": len(tr_idx),
        }
        print("Fold{}: ROC {:.4f} | PR {:.4f} | P {:.4f} | R {:.4f} | F1 {:.4f} | Train {} | Val {}".format(
            f, metrics["ROC-AUC"], metrics["PR-AUC"], metrics["Precision"], metrics["Recall"], metrics["F1-Score"],
            metrics["Train_Size"], metrics["Val_Size"]
        ))
        fold_rows.append(metrics)

    df_metrics = pd.DataFrame(fold_rows)
    df_metrics.to_csv(os.path.join(arch_dir, "metrics_foldwise.csv"), index=False)

    mean_row = df_metrics[["ROC-AUC","PR-AUC","Precision","Recall","F1-Score"]].mean()
    std_row  = df_metrics[["ROC-AUC","PR-AUC","Precision","Recall","F1-Score"]].std()

    df_summary = pd.DataFrame({
        "Metric": ["ROC-AUC","PR-AUC","Precision","Recall","F1-Score"],
        "Mean":   [mean_row[m] for m in mean_row.index],
        "SD":     [std_row[m]  for m in std_row.index],
    })
    df_summary.to_csv(os.path.join(arch_dir, "metrics_summary.csv"), index=False)

    print("\n[{}] 平均±SD".format(arch))
    for col in ["ROC-AUC", "PR-AUC", "Precision", "Recall", "F1-Score"]:
        print(f"{col}: {mean_row[col]:.4f} ± {std_row[col]:.4f}")

    all_summary_rows.append({
        "Arch": arch,
        **{f"{k}_Mean": mean_row[k] for k in mean_row.index},
        **{f"{k}_SD":   std_row[k]  for k in std_row.index},
    })

pd.DataFrame(all_summary_rows).to_csv(
    os.path.join(SAVE_BASE, "metrics_all_summary.csv"), index=False
)

print(f"\n✅ 完了: 保存先 {SAVE_BASE}")

[INFO] Graphs: 7321 | x_dim=5 | edge_dim=6 | g_dim=7 | r_dim=10
[INFO] Label balance: pos=1524 neg=5797 (pos_ratio=0.208)
[INFO] Priority CIDs present: 30 / 7
[INFO] Found priority CIDs in data: [3348, 3955, 13342, 68770, 10315094]

================ GINE (d.x + d.g + d.r + edge_attr) ================
[Fold 0] moved priority from val→train: 9  | swapped (train→val): 9
[train] mean(w)=1.00, mean(SampleWeight)=0.75
Fold0: ROC 0.7720 | PR 0.4413 | P 0.3255 | R 0.8406 | F1 0.4692 | Train 4880 | Val 2441
[Fold 1] moved priority from val→train: 14  | swapped (train→val): 14
[train] mean(w)=1.00, mean(SampleWeight)=0.75
Fold1: ROC 0.8703 | PR 0.6002 | P 0.4438 | R 0.8543 | F1 0.5841 | Train 4881 | Val 2440
[Fold 2] moved priority from val→train: 7  | swapped (train→val): 7
[train] mean(w)=1.00, mean(SampleWeight)=0.75
Fold2: ROC 0.8499 | PR 0.5394 | P 0.4157 | R 0.8543 | F1 0.5593 | Train 4881 | Val 2440

[GINE] 平均±SD
ROC-AUC: 0.8308 ± 0.0519
PR-AUC: 0.5270 ± 0.0802
Precision: 0.3950 ± 0.0618


In [ ]:
# =============================================================
# TransformerConv + NNConv アンサンブル（d.x + d.g + d.r, edge_attr）
# 5-fold CV / 学習曲線・ROC・PR曲線 / ロジット加重平均アンサンブル
# ★優先CID（7化合物）は常に学習セットへ固定（検証には出さない）
# 入力: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt
# 出力: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_ens_trans_nn_5fold/
# + SampleWeight対応（学習損失は必ず加重、検証指標は任意で加重）
# =============================================================

import os, torch, random
import torch.nn as nn
import torch.nn.functional as F
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from torch.serialization import add_safe_globals
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_score, recall_score, f1_score,
    roc_curve, precision_recall_curve, auc
)

# ----- 事前準備 & 追加設定（★） -----
SEED = 42
torch.manual_seed(SEED); random.seed(SEED); np.random.seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_PATH = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt"
SAVE_DIR  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_ens_trans_nn_5fold"
PLOT_DIR  = os.path.join(SAVE_DIR, "plots")
os.makedirs(SAVE_DIR, exist_ok=True)

# ★ 重み属性の候補（上から順に探索）
WEIGHT_ATTR_CANDIDATES = ["sample_weight","weight","w","SampleWeight","Weight"]
# ★ 検証でも重みを使うか（True推奨。外部比較で非加重にしたい場合は False）
APPLY_WEIGHT_IN_VAL = True

BATCH_TRAIN = 64
BATCH_VAL   = 128
HID = 256
LR  = 3e-4
WEIGHT_DECAY = 1e-4
PATIENCE = 10
TMAX = 10
CLIP = 5
MAX_EPOCHS = 200

# ----- 優先CID（常に学習へ固定）-----
PRIORITY_CIDS = {3348, 3955, 2724385, 10315094, 68770, 13342, 441074}

# ----- データ読み込み（PyTorch 2.6 安全ロード対応） -----
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        add_safe_globals([DataEdgeAttr, DataTensorAttr])
    except Exception as e:
        print(f"[WARN] add_safe_globals 失敗: {e}")
    try:
        obj = torch.load(path)  # weights_only=True 既定
    except Exception as e:
        print(f"[WARN] 安全ロード失敗（{e}）。weights_only=False で再試行します（信頼ファイル前提）。")
        obj = torch.load(path, weights_only=False)

    if isinstance(obj, list):
        return obj
    if isinstance(obj, dict) and "data_list" in obj:
        return obj["data_list"]
    try:
        return list(obj)
    except Exception as e:
        raise RuntimeError(f"未知の形式: {type(obj)}; list化失敗: {e}")

data = load_pyg_list(DATA_PATH)
if len(data) == 0:
    raise RuntimeError("data が空です。")

# ----- ラベル & CID を抽出 -----
labels = [int(d.y.item()) for d in data]
cids   = [int(getattr(d, "cid", -1)) for d in data]  # 追加済みcidを参照（欠損は -1）

# 優先CIDインデックスの抽出
priority_idx = [i for i, cid in enumerate(cids) if cid in PRIORITY_CIDS]
non_priority_idx = [i for i in range(len(data)) if i not in priority_idx]

print(f"[INFO] 全サンプル: {len(data)} | 優先CIDヒット件数: {len(priority_idx)}")
if len(priority_idx) == 0:
    print("[WARN] 優先CIDに該当するサンプルが見つかりません（cid==-1含む）。")
else:
    _, counts = np.unique([cids[i] for i in priority_idx], return_counts=True)
    unique_cids = sorted(set([cids[i] for i in priority_idx]))
    print("[INFO] 優先CID一覧:", unique_cids)
    print("[INFO] 優先CIDの合計グラフ数（拡張含む）:", int(sum(counts)))

# ----- 次元を自動取得 -----
sample = data[0]
def infer_feat_dim(t):
    if t is None: return 0
    if t.dim() == 1: return t.numel()
    if t.dim() in (2,3): return t.size(-1)
    raise RuntimeError(f"未知のテンソル次元: dim={t.dim()}")

X_DIM   = sample.x.size(-1)
E_DIM   = sample.edge_attr.size(-1)
G_DIM   = infer_feat_dim(sample.g)
R_DIM   = infer_feat_dim(sample.r)
print(f"[INFO] Dims: x={X_DIM}, edge={E_DIM}, g={G_DIM}, r={R_DIM}")

# ----- モデル定義 -----
class Transformer_DX_DG_DR(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = TransformerConv(X_DIM, HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.c2 = TransformerConv(HID,  HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x, g, r], dim=1)).view(-1)  # logits

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hidden = max(64, min(256, in_attr_dim * 8))
    return nn.Sequential(nn.Linear(in_attr_dim, hidden), nn.ReLU(),
                         nn.Linear(hidden, in_channels * out_channels))

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self):
        super().__init__()
        edge_nn1 = make_edge_mlp(E_DIM, HID, X_DIM)
        edge_nn2 = make_edge_mlp(E_DIM, HID, HID)
        self.c1 = NNConv(X_DIM, HID, edge_nn1, aggr='mean')
        self.c2 = NNConv(HID,  HID, edge_nn2, aggr='mean')
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x, g, r], dim=1)).view(-1)  # logits

# ----- 重みユーティリティ（★） -----
def _find_weight_attr(d):
    for name in WEIGHT_ATTR_CANDIDATES:
        if hasattr(d, name):
            return name
    return None

def _to_1d_weight(t):
    """
    DataLoader後の batch.<attr> は [N] or [N,1] の可能性があるので 1D化
    """
    if t is None: return None
    if t.dim() == 2 and t.size(1) == 1:
        return t.view(-1)
    return t.view(-1)

def get_batch_weights(batch, device=None):
    name = None
    for cand in WEIGHT_ATTR_CANDIDATES:
        if hasattr(batch, cand):
            name = cand; break
    if name is None:
        # 1.0で埋める
        n = batch.y.numel()
        return torch.ones(n, dtype=torch.float, device=device or batch.y.device), "CONST1"
    w = getattr(batch, name)
    w = _to_1d_weight(w).to(device or batch.y.device).float()
    return w, name

# ----- 補助: プロット保存 -----
def _ensure_dir(path): os.makedirs(path, exist_ok=True)

def plot_curve(xs, ys, title, xlabel, ylabel, save_path):
    plt.figure(); plt.plot(xs, ys)
    plt.title(title); plt.xlabel(xlabel); plt.ylabel(ylabel)
    plt.tight_layout(); plt.savefig(save_path); plt.close()

def save_roc_pr_curves(y_true, y_prob, title_prefix, out_dir, sample_weight=None):
    _ensure_dir(out_dir)
    fpr, tpr, _ = roc_curve(y_true, y_prob, sample_weight=sample_weight)
    roc_auc = auc(fpr, tpr)
    plt.figure(); plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1],[0,1],'--', linewidth=1)
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"{title_prefix} ROC")
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{title_prefix}_ROC.png")); plt.close()
    prec, rec, _ = precision_recall_curve(y_true, y_prob, sample_weight=sample_weight)
    ap = average_precision_score(y_true, y_prob, sample_weight=sample_weight)
    plt.figure(); plt.plot(rec, prec, label=f"AP={ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"{title_prefix} PR")
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{title_prefix}_PR.png")); plt.close()

def best_f1_threshold(y_true, y_prob, sample_weight=None):
    prec, rec, thr = precision_recall_curve(y_true, y_prob, sample_weight=sample_weight)
    thr = np.append(thr, 1.0)
    f1s = 2 * prec * rec / np.clip(prec + rec, 1e-9, None)
    i = np.nanargmax(f1s)
    return float(thr[i]), float(f1s[i])

# ----- 学習関数（★ 重み対応） -----
def train_fold(model, tr_loader, va_loader, fold, save_path, model_name, apply_weight_in_val=True):
    # ---- pos_weight を「重み付き実効比」から計算（最大10にクリップ） ----
    # tr_loader.dataset は list[Data] を想定
    y_list, w_list = [], []
    for d in tr_loader.dataset:
        y_list.append(int(d.y.item()))
        # サンプル毎の重みを取得（無ければ1）
        w = None
        for cand in WEIGHT_ATTR_CANDIDATES:
            if hasattr(d, cand):
                a = getattr(d, cand)
                # a は tensor([w]) または float を想定
                w = float(a.item()) if torch.is_tensor(a) else float(a)
                break
        if w is None: w = 1.0
        w_list.append(w)
    ytr = torch.tensor(y_list, dtype=torch.float)
    wtr = torch.tensor(w_list, dtype=torch.float)
    pos_w = (wtr[ytr==0].sum() / torch.clamp(wtr[ytr==1].sum(), min=1e-8)).item()
    pos_w = float(min(max(pos_w, 1.0), 10.0))

    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_w], device=DEVICE),
                                     reduction='none')  # ← per-sample
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=TMAX)

    history = {"epoch": [], "train_loss": [], "val_auc": [], "val_prauc": []}
    best_auc, best_state, wait = 0.0, None, 0

    print(f"[INFO] fold{fold} pos_weight(eff)={pos_w:.3f}")

    for ep in range(1, MAX_EPOCHS + 1):
        model.train(); epoch_losses = []
        for batch in tr_loader:
            batch = batch.to(DEVICE)
            logits = model(batch)
            # per-sample loss
            loss_vec = criterion(logits, batch.y.float())
            sw, used_attr = get_batch_weights(batch, DEVICE)
            # 加重平均（分母=重み合計）
            loss = (loss_vec * sw).sum() / (sw.sum() + 1e-8)

            opt.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CLIP)
            opt.step()
            epoch_losses.append(loss.item())
        sch.step()

        # 検証
        model.eval(); ys, ps, ws = [], [], []
        with torch.no_grad():
            for batch in va_loader:
                batch = batch.to(DEVICE)
                ys += batch.y.cpu().tolist()
                ps += torch.sigmoid(model(batch)).cpu().tolist()
                sw, _ = get_batch_weights(batch, DEVICE)
                ws += sw.detach().cpu().tolist()

        if apply_weight_in_val:
            auc_val = roc_auc_score(ys, ps, sample_weight=ws)
            pr_val  = average_precision_score(ys, ps, sample_weight=ws)
        else:
            auc_val = roc_auc_score(ys, ps)
            pr_val  = average_precision_score(ys, ps)

        history["epoch"].append(ep)
        history["train_loss"].append(float(np.mean(epoch_losses)))
        history["val_auc"].append(float(auc_val))
        history["val_prauc"].append(float(pr_val))

        if auc_val > best_auc:
            best_auc, wait = auc_val, 0
            best_state = model.state_dict()
            torch.save(best_state, save_path)
        else:
            wait += 1
            if wait >= PATIENCE:
                print(f"✅ EarlyStopped [{model_name}] (fold{fold}) at epoch {ep}")
                break

    model.load_state_dict(best_state)
    return model, history

# ----- Cross Validation（5-fold）：優先CIDを常に学習へ固定 -----
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
metrics_tr_list, metrics_nn_list, metrics_ens_list = [], [], []

# 分割対象は「非優先」サンプルのみで行い、各foldの学習側に優先サンプルを追加
non_priority_labels = np.array([labels[i] for i in non_priority_idx])
non_priority_idx    = np.array(non_priority_idx, dtype=int)
priority_idx        = np.array(priority_idx, dtype=int)

fold_counter = 0
for tr_np, va_np in kf.split(np.zeros(len(non_priority_idx)), non_priority_labels):
    # 非優先のfold分割結果 → 実インデックスへ変換
    tr_base = non_priority_idx[tr_np]
    va_base = non_priority_idx[va_np]

    # 学習 = 非優先の学習 + 全優先
    tr_idx = np.concatenate([tr_base, priority_idx]).astype(int)
    # 検証 = 非優先の検証（優先は一切含めない）
    va_idx = va_base.astype(int)

    # 念のためユニーク化＆衝突除去
    tr_idx = np.unique(tr_idx)
    va_idx = np.setdiff1d(np.unique(va_idx), priority_idx, assume_unique=False)

    # foldごとの情報表示（重み統計も表示）（★）
    def _stats(indices):
        ys = np.array([int(data[i].y.item()) for i in indices])
        # 重み取り出し
        ws = []
        for i in indices:
            w = None
            for cand in WEIGHT_ATTR_CANDIDATES:
                if hasattr(data[i], cand):
                    a = getattr(data[i], cand)
                    w = float(a.item()) if torch.is_tensor(a) else float(a)
                    break
            if w is None: w = 1.0
            ws.append(w)
        ws = np.array(ws, dtype=float)
        return ys.mean(), ws.mean(), ws.min(), ws.max()
    y_bar_tr, w_bar_tr, w_min_tr, w_max_tr = _stats(tr_idx)
    y_bar_va, w_bar_va, w_min_va, w_max_va = _stats(va_idx)

    print(f"\n📂 Fold {fold_counter} | train={len(tr_idx)} val={len(va_idx)} | "
          f"train_w(mean/min/max)={w_bar_tr:.2f}/{w_min_tr:.2f}/{w_max_tr:.2f} | "
          f"val_w(mean/min/max)={w_bar_va:.2f}/{w_min_va:.2f}/{w_max_va:.2f}")

    tr_data = [data[i] for i in tr_idx]
    va_data = [data[i] for i in va_idx]

    tr_loader = DataLoader(tr_data, batch_size=BATCH_TRAIN, shuffle=True, pin_memory=True)
    va_loader = DataLoader(va_data, batch_size=BATCH_VAL,   shuffle=False, pin_memory=True)

    fold_plot_dir = os.path.join(PLOT_DIR, f"fold{fold_counter}")
    _ensure_dir(fold_plot_dir)

    # ---- Transformer ----
    model_tr = Transformer_DX_DG_DR().to(DEVICE)
    path_tr  = f"{SAVE_DIR}/transformer_fold{fold_counter}.pt"
    model_tr, hist_tr = train_fold(model_tr, tr_loader, va_loader, fold_counter, path_tr, "Transformer",
                                   apply_weight_in_val=APPLY_WEIGHT_IN_VAL)

    pd.DataFrame(hist_tr).to_csv(f"{SAVE_DIR}/history_transformer_fold{fold_counter}.csv", index=False)
    plot_curve(hist_tr["epoch"], hist_tr["train_loss"], f"Transformer Fold{fold_counter} Train Loss", "Epoch", "Loss",
               os.path.join(fold_plot_dir, "Transformer_Loss.png"))
    plot_curve(hist_tr["epoch"], hist_tr["val_auc"], f"Transformer Fold{fold_counter} Val ROC-AUC", "Epoch", "ROC-AUC",
               os.path.join(fold_plot_dir, "Transformer_ValAUC.png"))
    plot_curve(hist_tr["epoch"], hist_tr["val_prauc"], f"Transformer Fold{fold_counter} Val PR-AUC", "Epoch", "PR-AUC",
               os.path.join(fold_plot_dir, "Transformer_ValPRAUC.png"))

    # ---- NNConv ----
    model_nn = NNConv_DX_DG_DR().to(DEVICE)
    path_nn  = f"{SAVE_DIR}/nnconv_fold{fold_counter}.pt"
    model_nn, hist_nn = train_fold(model_nn, tr_loader, va_loader, fold_counter, path_nn, "NNConv",
                                   apply_weight_in_val=APPLY_WEIGHT_IN_VAL)

    pd.DataFrame(hist_nn).to_csv(f"{SAVE_DIR}/history_nnconv_fold{fold_counter}.csv", index=False)
    plot_curve(hist_nn["epoch"], hist_nn["train_loss"], f"NNConv Fold{fold_counter} Train Loss", "Epoch", "Loss",
               os.path.join(fold_plot_dir, "NNConv_Loss.png"))
    plot_curve(hist_nn["epoch"], hist_nn["val_auc"], f"NNConv Fold{fold_counter} Val ROC-AUC", "Epoch", "ROC-AUC",
               os.path.join(fold_plot_dir, "NNConv_ValAUC.png"))
    plot_curve(hist_nn["epoch"], hist_nn["val_prauc"], f"NNConv Fold{fold_counter} Val PR-AUC", "Epoch", "PR-AUC",
               os.path.join(fold_plot_dir, "NNConv_ValPRAUC.png"))

    # ---- 評価（各モデル&アンサンブル; ロジット加重平均）----
    def eval_logits_and_weights(m):
        m.eval(); ys, logits, ws = [], [], []
        with torch.no_grad():
            for batch in va_loader:
                batch = batch.to(DEVICE)
                ys += batch.y.cpu().tolist()
                logits += m(batch).cpu().tolist()  # ロジット
                sw, _ = get_batch_weights(batch, DEVICE)
                ws += sw.detach().cpu().tolist()
        return np.array(ys), np.array(logits), np.array(ws, dtype=float)

    ys_tr, lg_tr, w_va = eval_logits_and_weights(model_tr)
    ys_nn, lg_nn, w_va2 = eval_logits_and_weights(model_nn)
    assert np.allclose(ys_tr, ys_nn)
    ys = ys_tr.tolist()

    # まず各モデルの確率でPR-AUCを算出 → その重みでロジット加重平均
    ps_tr = torch.sigmoid(torch.tensor(lg_tr)).numpy()
    ps_nn = torch.sigmoid(torch.tensor(lg_nn)).numpy()
    if APPLY_WEIGHT_IN_VAL:
        pr_tr = average_precision_score(ys, ps_tr, sample_weight=w_va)
        pr_nn = average_precision_score(ys, ps_nn, sample_weight=w_va)
    else:
        pr_tr = average_precision_score(ys, ps_tr)
        pr_nn = average_precision_score(ys, ps_nn)

    a_tr = pr_tr / (pr_tr + pr_nn + 1e-12)
    a_nn = pr_nn / (pr_tr + pr_nn + 1e-12)

    lg_ens = a_tr * lg_tr + a_nn * lg_nn
    ps_ens = torch.sigmoid(torch.tensor(lg_ens)).numpy()

    def pack_metrics(y, p, name, sw=None):
        thr_opt, f1_opt = best_f1_threshold(y, p, sample_weight=sw if APPLY_WEIGHT_IN_VAL else None)
        y_pred05 = (p >= 0.5).astype(int)
        y_predF  = (p >= thr_opt).astype(int)
        if APPLY_WEIGHT_IN_VAL:
            roc = roc_auc_score(y, p, sample_weight=sw)
            pr  = average_precision_score(y, p, sample_weight=sw)
            # ※ precision/recall/f1 は表示用に非加重（※必要なら sklearn に重み版は無いので手計算に変更可）
            prec05 = precision_score(y, y_pred05, zero_division=0)
            rec05  = recall_score(y, y_pred05, zero_division=0)
            f105   = f1_score(y, y_pred05, zero_division=0)
        else:
            roc = roc_auc_score(y, p)
            pr  = average_precision_score(y, p)
            prec05 = precision_score(y, y_pred05, zero_division=0)
            rec05  = recall_score(y, y_pred05, zero_division=0)
            f105   = f1_score(y, y_pred05, zero_division=0)
        return {
            "Fold": fold_counter, "Model": name,
            "ROC-AUC": roc, "PR-AUC": pr,
            "Precision@0.5": prec05, "Recall@0.5": rec05, "F1@0.5": f105,
            "BestThr(F1)": thr_opt, "F1@BestThr": f1_opt
        }

    mt = pack_metrics(ys, ps_tr,  "Transformer", w_va)
    mn = pack_metrics(ys, ps_nn,  "NNConv",      w_va)
    me = pack_metrics(ys, ps_ens, "Ensemble",    w_va)
    print(f"[Transformer] {mt}")
    print(f"[NNConv]      {mn}")
    print(f"[Ensemble]    {me} (alpha={a_tr:.3f}/{a_nn:.3f})")

    # ROC/PR カーブ保存（★重み適用オプション）
    fold_dir = os.path.join(fold_plot_dir)
    save_roc_pr_curves(ys, ps_tr,  "Transformer", fold_dir, sample_weight=w_va if APPLY_WEIGHT_IN_VAL else None)
    save_roc_pr_curves(ys, ps_nn,  "NNConv",      fold_dir, sample_weight=w_va if APPLY_WEIGHT_IN_VAL else None)
    save_roc_pr_curves(ys, ps_ens, "Ensemble",    fold_dir, sample_weight=w_va if APPLY_WEIGHT_IN_VAL else None)

    metrics_tr_list.append(mt)
    metrics_nn_list.append(mn)
    metrics_ens_list.append(me)

    fold_counter += 1

# ----- 結果保存 -----
def save_metrics(rows, csv_name):
    df = pd.DataFrame(rows)
    mean = df.select_dtypes(include=[np.number]).mean()
    std  = df.select_dtypes(include=[np.number]).std()
    df.loc["Mean"] = mean
    df.loc["Std"]  = std
    path = f"{SAVE_DIR}/{csv_name}"
    df.to_csv(path, index=False)
    print(f"📁 Saved: {path}")
    print("🎯 平均結果:"); print(mean)

save_metrics(metrics_tr_list,  "transformer_cv_results.csv")
save_metrics(metrics_nn_list,  "nnconv_cv_results.csv")
save_metrics(metrics_ens_list, "ensemble_cv_results.csv")

print("\n✅ 完了:", SAVE_DIR)
print(f"ℹ️ 検証で重み適用: {APPLY_WEIGHT_IN_VAL}")

[INFO] 全サンプル: 7321 | 優先CIDヒット件数: 30
[INFO] 優先CID一覧: [3348, 3955, 13342, 68770, 10315094]
[INFO] 優先CIDの合計グラフ数（拡張含む）: 30
[INFO] Dims: x=5, edge=6, g=7, r=10

📂 Fold 0 | train=5862 val=1459 | train_w(mean/min/max)=0.74/0.70/5.00 | val_w(mean/min/max)=0.73/0.70/5.00
[INFO] fold0 pos_weight(eff)=3.001
✅ EarlyStopped [Transformer] (fold0) at epoch 123
[INFO] fold0 pos_weight(eff)=3.001
✅ EarlyStopped [NNConv] (fold0) at epoch 122
[Transformer] {'Fold': 0, 'Model': 'Transformer', 'ROC-AUC': 0.9290930207799629, 'PR-AUC': 0.7607180366092011, 'Precision@0.5': 0.6155913978494624, 'Recall@0.5': 0.7658862876254181, 'F1@0.5': 0.6825633383010432, 'BestThr(F1)': 0.3620673165428875, 'F1@BestThr': 0.7299046330523874}
[NNConv]      {'Fold': 0, 'Model': 'NNConv', 'ROC-AUC': 0.9007353953179269, 'PR-AUC': 0.6945218779560177, 'Precision@0.5': 0.5343347639484979, 'Recall@0.5': 0.8327759197324415, 'F1@0.5': 0.6509803921568628, 'BestThr(F1)': 0.5473303160253076, 'F1@BestThr': 0.6893572919018572}
[Ensemble]    {

In [ ]:
# ================================================
# TransformerConv + NNConv (d.x + d.g + d.r, edge_attr)
# Scaffold 5-Fold CV + Ensemble (logit-weighted by Val PR-AUC)
# 学習曲線（Loss/Val AUC/Val PR-AUC）＆ROC/PR図 保存付き
# ★ 優先CIDの属するスキャフォールドは毎foldで必ず学習へ固定（検証/テストから除外）
# --- 以下、加筆 ---
# ★ SampleWeight対応：
#    ・学習損失は必ずサンプル重みで加重（BCEWithLogitsLoss(reduction='none') → 重み付き平均）
#    ・pos_weight は重み付き実効比（neg_sum/pos_sum、上限10）で計算
#    ・検証の ROC/PR・図・アンサンブル重み（Val PR-AUC）も、フラグで重み適用可能
# ★ 重み属性候補（上から順に探索）：["sample_weight","weight","w","SampleWeight","Weight"]（無ければ全て1.0）
# ★ フラグ：APPLY_WEIGHT_IN_VAL（検証に重み適用）、APPLY_WEIGHT_IN_TEST（テストに重み適用）
# ================================================
import os, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.serialization import add_safe_globals
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation

from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

from sklearn.metrics import (
    roc_auc_score, precision_recall_curve, f1_score, auc, roc_curve, average_precision_score
)
from collections import defaultdict

# ----- 設定 -----
PT_PATH   = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt"
SAVE_DIR  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_ens_trans_nn_scaffold5fold"
PLOT_DIR  = os.path.join(SAVE_DIR, "plots")
NUM_FOLDS = 5
BATCH_TR  = 64
BATCH_TE  = 128
LR        = 3e-4
WD        = 1e-4
TMAX      = 20
PATIENCE  = 15
MAX_EPOCH = 200
CLIP      = 5
SEED      = 42

# ★ 学習へ常時含める CID（該当スキャフォールドごと学習へ固定）
PRIORITY_CIDS = {3348, 3955, 2724385, 10315094, 68770, 13342, 441074}

# ★ SampleWeight 設定（加筆）
WEIGHT_ATTR_CANDIDATES = ["sample_weight","weight","w","SampleWeight","Weight"]
APPLY_WEIGHT_IN_VAL  = True   # 検証の AUC/AP/図/アンサンブル重みに重みを使う
APPLY_WEIGHT_IN_TEST = False  # テストの指標・図に重みを使う（必要なら True に）

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(SAVE_DIR, exist_ok=True); os.makedirs(PLOT_DIR, exist_ok=True)

# ----- ユーティリティ -----
def _ensure_dir(p): os.makedirs(p, exist_ok=True)

def plot_curve(xs, ys, title, xlabel, ylabel, save_path):
    plt.figure(); plt.plot(xs, ys)
    plt.title(title); plt.xlabel(xlabel); plt.ylabel(ylabel)
    plt.tight_layout(); plt.savefig(save_path); plt.close()

def save_roc_pr_curves(y_true, y_prob, title_prefix, out_dir, sample_weight=None):
    _ensure_dir(out_dir)
    # ROC
    fpr, tpr, _ = roc_curve(y_true, y_prob, sample_weight=sample_weight)
    roc_auc = auc(fpr, tpr)
    plt.figure(); plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1],[0,1],'--', linewidth=1)
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"{title_prefix} ROC")
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{title_prefix}_ROC.png")); plt.close()
    # PR
    prec, rec, _ = precision_recall_curve(y_true, y_prob, sample_weight=sample_weight)
    ap = average_precision_score(y_true, y_prob, sample_weight=sample_weight)
    plt.figure(); plt.plot(rec, prec, label=f"AP={ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"{title_prefix} PR")
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{title_prefix}_PR.png")); plt.close()

# ★ 重み取り出し（Data / batch 共通）
def _get_weight_from_data(d):
    for nm in WEIGHT_ATTR_CANDIDATES:
        if hasattr(d, nm):
            v = getattr(d, nm)
            return float(v.item()) if torch.is_tensor(v) else float(v)
    return 1.0

def _to_1d(w):
    if w is None: return None
    if torch.is_tensor(w):
        if w.dim() == 2 and w.size(1) == 1: return w.view(-1)
        return w.view(-1)
    return torch.tensor(w, dtype=torch.float32)

def get_batch_weights(batch, device=None):
    for nm in WEIGHT_ATTR_CANDIDATES:
        if hasattr(batch, nm):
            w = getattr(batch, nm)
            w = _to_1d(w).to(device or batch.y.device).float()
            return w, nm
    # 無ければ1
    return torch.ones(batch.y.numel(), device=device or batch.y.device, dtype=torch.float32), "CONST1"

# ----- データ読み込み（PyTorch 2.6 安全ロード対応） -----
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception as e:
        print(f"[WARN] add_safe_globals 失敗: {e}")
    try:
        obj = torch.load(path)  # PyTorch 2.6 default: weights_only=True
    except Exception as e:
        print(f"[WARN] 安全ロード失敗（{e}）。weights_only=False で再試行します（信頼ファイル前提）。")
        obj = torch.load(path, weights_only=False)

    if isinstance(obj, list):
        return obj
    if isinstance(obj, dict) and "data_list" in obj:
        return obj["data_list"]
    try:
        return list(obj)
    except Exception as e:
        raise RuntimeError(f"未知の形式: {type(obj)}; list化失敗: {e}")

data = load_pyg_list(PT_PATH)
print(f"✅ データ読み込み完了: {len(data)}件")
print("ラベル分布:", pd.Series([int(d.y) for d in data]).value_counts().to_dict())

# ----- スキャフォールド算出 & インデックス対応 -----
def smiles_to_scaffold(smi, fallback):
    m = Chem.MolFromSmiles(smi)
    if m:
        try:
            return MurckoScaffold.MurckoScaffoldSmiles(mol=m)
        except Exception:
            pass
    return fallback  # 失敗時はユニークなフォールバック

scaf_of = []
for i, d in enumerate(data):
    smi = getattr(d, "smiles", "")
    scaf = smiles_to_scaffold(smi, f"_NOSCAF_{i}")
    scaf_of.append(scaf)

# スキャフォールド→インデックス
scaf_to_indices = defaultdict(list)
for i, scaf in enumerate(scaf_of):
    scaf_to_indices[scaf].append(i)

# 優先CIDのインデックスと、それが属するスキャフォールド集合
cids = [int(getattr(d, "cid", -1)) for d in data]
priority_idx = [i for i, c in enumerate(cids) if c in PRIORITY_CIDS]
priority_scaffolds = set(scaf_of[i] for i in priority_idx)
print(f"[INFO] 優先CIDヒット: {len(priority_idx)} サンプル / 優先スキャフォールド: {len(priority_scaffolds)} 種")

# ----- スキャフォールド分割 -----
def scaffold_split_kfold(k=5, seed=42):
    scafs = list(scaf_to_indices.keys())
    random.seed(seed); random.shuffle(scafs)
    folds = [[] for _ in range(k)]
    for i, sc in enumerate(scafs):
        folds[i % k].extend(scaf_to_indices[sc])
    return folds

folds = scaffold_split_kfold(k=NUM_FOLDS, seed=SEED)

# ----- 次元取得 -----
sample = data[0]
def infer_dim(t):
    if t.dim() == 1: return t.numel()
    if t.dim() in (2,3): return t.size(-1)
    raise RuntimeError(f"未知のテンソル次元: {t.size()}")

X_DIM = sample.x.size(-1)
E_DIM = sample.edge_attr.size(-1)
G_DIM = infer_dim(sample.g)
R_DIM = infer_dim(sample.r)
HID   = 256
print(f"[INFO] Dims: x={X_DIM}, edge={E_DIM}, g={G_DIM}, r={R_DIM}")

# ----- モデル -----
class Transformer_DX_DG_DR(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1 = TransformerConv(X_DIM, HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.c2 = TransformerConv(HID,  HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x, g, r], dim=1)).view(-1)  # ロジット

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hidden = max(64, min(256, in_attr_dim * 8))
    return nn.Sequential(nn.Linear(in_attr_dim, hidden), nn.ReLU(),
                         nn.Linear(hidden, in_channels * out_channels))

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self):
        super().__init__()
        edge_nn1 = make_edge_mlp(E_DIM, HID, X_DIM)
        edge_nn2 = make_edge_mlp(E_DIM, HID, HID)
        self.c1 = NNConv(X_DIM, HID, edge_nn1, aggr='mean')
        self.c2 = NNConv(HID,  HID, edge_nn2, aggr='mean')
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x, g, r], dim=1)).view(-1)  # ロジット

# ----- 学習/評価ユーティリティ（SampleWeight対応に改修） -----
def pos_weight_from(dataset):
    # 重み付き実効比（neg_sum / pos_sum）、上限10
    ys = torch.tensor([float(d.y.item()) for d in dataset])
    ws = torch.tensor([_get_weight_from_data(d) for d in dataset], dtype=torch.float32)
    pos_sum = ws[ys == 1].sum()
    neg_sum = ws[ys == 0].sum()
    pw = (neg_sum / torch.clamp(pos_sum, min=1e-8)).item()
    return torch.tensor([min(max(pw, 1.0), 10.0)], dtype=torch.float32)

@torch.no_grad()
def eval_logits(model, ds, batch_size=BATCH_TE):
    model.eval(); ys, lg, ws = [], [], []
    for bt in DataLoader(ds, batch_size=batch_size, shuffle=False):
        bt = bt.to(DEVICE)
        ys.append(bt.y.detach().cpu())
        lg.append(model(bt).detach().cpu())  # ロジット
        w, _ = get_batch_weights(bt, DEVICE)
        ws.append(w.detach().cpu())
    y  = torch.cat(ys).numpy()
    l  = torch.cat(lg).numpy()
    sw = torch.cat(ws).numpy()
    return y, l, sw

def metrics_from(y, p, thr=0.5, sample_weight=None):
    precision, recall, _ = precision_recall_curve(y, p, sample_weight=sample_weight)
    pr_auc = auc(recall, precision)
    roc    = roc_auc_score(y, p, sample_weight=sample_weight)
    f1     = f1_score(y, (p >= thr).astype(int))  # ※F1は表示用に非加重（必要なら手計算に変更）
    return roc, pr_auc, f1

def train_one(model, train_set, val_set, save_path, tag, plot_dir):
    model.to(DEVICE)
    pw = pos_weight_from(train_set).to(DEVICE)
    crit = nn.BCEWithLogitsLoss(pos_weight=pw, reduction='none')  # ← per-sample
    opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    sch  = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=TMAX)

    history = {"epoch": [], "train_loss": [], "val_auc": [], "val_prauc": []}
    best_auc, wait = 0.0, 0

    print(f"[{tag}] pos_weight(eff)={float(pw.item()):.3f}")

    for ep in range(1, MAX_EPOCH + 1):
        model.train(); epoch_losses = []
        for bt in DataLoader(train_set, batch_size=BATCH_TR, shuffle=True):
            bt = bt.to(DEVICE)
            logits = model(bt)
            loss_vec = crit(logits, bt.y.float())
            sw, _nm = get_batch_weights(bt, DEVICE)
            loss = (loss_vec * sw).sum() / (sw.sum() + 1e-8)

            opt.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CLIP)
            opt.step()
            epoch_losses.append(loss.item())
        sch.step()

        yv, lv, wv = eval_logits(model, val_set)
        pv = torch.sigmoid(torch.tensor(lv)).numpy()
        if APPLY_WEIGHT_IN_VAL:
            roc_v, pr_v, f1_v = metrics_from(yv, pv, sample_weight=wv)
        else:
            roc_v, pr_v, f1_v = metrics_from(yv, pv)

        history["epoch"].append(ep)
        history["train_loss"].append(float(np.mean(epoch_losses)))
        history["val_auc"].append(float(roc_v))
        history["val_prauc"].append(float(pr_v))

        print(f"[{tag}] Epoch {ep:03d} | ROC-AUC {roc_v:.4f} | PR-AUC {pr_v:.4f} | F1 {f1_v:.4f}")

        if roc_v > best_auc:
            best_auc, wait = roc_v, 0
            torch.save(model.state_dict(), save_path)
        else:
            wait += 1
            if wait >= PATIENCE:
                print(f"✅ EarlyStopping [{tag}]")
                break

    # ベスト復元
    model.load_state_dict(torch.load(save_path, map_location=DEVICE))

    # 学習曲線保存
    df_hist = pd.DataFrame(history)
    df_hist.to_csv(save_path.replace(".pt", f"_history.csv"), index=False)
    _ensure_dir(plot_dir)
    plot_curve(df_hist["epoch"], df_hist["train_loss"], f"{tag} Train Loss", "Epoch", "Loss",
               os.path.join(plot_dir, f"{tag.replace('/','_')}_Loss.png"))
    plot_curve(df_hist["epoch"], df_hist["val_auc"], f"{tag} Val ROC-AUC", "Epoch", "ROC-AUC",
               os.path.join(plot_dir, f"{tag.replace('/','_')}_ValAUC.png"))
    plot_curve(df_hist["epoch"], df_hist["val_prauc"], f"{tag} Val PR-AUC", "Epoch", "PR-AUC",
               os.path.join(plot_dir, f"{tag.replace('/','_')}_ValPRAUC.png"))

    # ベストモデルでの検証PR-AUC（アンサンブル重み用）
    yv, lv, wv = eval_logits(model, val_set)
    pv = torch.sigmoid(torch.tensor(lv)).numpy()
    pr_v = average_precision_score(yv, pv, sample_weight=wv) if APPLY_WEIGHT_IN_VAL else average_precision_score(yv, pv)
    return model, float(pr_v)

# ----- クロスバリデーション（優先スキャフォールドは常に学習へ） -----
res_tr, res_nn, res_en = [], [], []

for k in range(NUM_FOLDS):
    test_idx = folds[k]
    val_idx  = folds[(k + 1) % NUM_FOLDS]
    train_idx = [i for j in range(NUM_FOLDS) if j not in [k, (k + 1) % NUM_FOLDS] for i in folds[j]]

    # ここが肝：優先CIDが属するスキャフォールドは検証・テストから除外し、学習へ移動
    def move_priority_scaffolds_to_train(idx_list):
        moved = []
        for i in list(idx_list):
            sc = scaf_of[i]
            if sc in priority_scaffolds:  # 優先スキャフォールド
                moved.extend(scaf_to_indices[sc])
        idx_set = set(idx_list) - set(moved)
        return list(sorted(idx_set)), list(sorted(set(moved)))

    val_idx,  moved_from_val  = move_priority_scaffolds_to_train(val_idx)
    test_idx, moved_from_test = move_priority_scaffolds_to_train(test_idx)

    # 学習へ追加（重複排除）
    train_idx = list(sorted(set(train_idx).union(moved_from_val).union(moved_from_test)))

    # サニティチェック：優先インデックスがval/testに残っていないこと
    pr_in_val  = len(set(priority_idx).intersection(val_idx))
    pr_in_test = len(set(priority_idx).intersection(test_idx))
    print(f"\n📂 Fold {k} | train={len(train_idx)} val={len(val_idx)} test={len(test_idx)} "
          f"| moved_from_val={len(moved_from_val)} moved_from_test={len(moved_from_test)} "
          f"| priority_in_val={pr_in_val} priority_in_test={pr_in_test}")

    train_set = [data[i] for i in train_idx]
    val_set   = [data[i] for i in val_idx]
    test_set  = [data[i] for i in test_idx]

    fold_plot_dir = os.path.join(PLOT_DIR, f"fold{k}"); _ensure_dir(fold_plot_dir)

    # --- 学習（検証PR-AUCを重みとして返却） ---
    m_tr = Transformer_DX_DG_DR().to(DEVICE)
    m_nn = NNConv_DX_DG_DR().to(DEVICE)
    path_tr = f"{SAVE_DIR}/transformer_fold{k}.pt"
    path_nn = f"{SAVE_DIR}/nnconv_fold{k}.pt"

    m_tr, pr_tr_val = train_one(m_tr, train_set, val_set, path_tr, f"Transformer/F{k}", fold_plot_dir)
    m_nn, pr_nn_val = train_one(m_nn, train_set, val_set, path_nn, f"NNConv/F{k}",      fold_plot_dir)

    # foldごとのアンサンブル重み（検証PR-AUCで正規化） ※検証PR-AUCはフラグに応じて重み付け済み
    w_tr, w_nn = pr_tr_val, pr_nn_val
    s = w_tr + w_nn + 1e-12
    a_tr, a_nn = w_tr/s, w_nn/s

    # --- テスト（ロジット加重平均） ---
    yt, lg_tr, wt = eval_logits(m_tr, test_set)
    _,  lg_nn, wt2 = eval_logits(m_nn, test_set)
    assert np.allclose(wt, wt2)  # 同一分割のはず

    ps_tr = torch.sigmoid(torch.tensor(lg_tr)).numpy()
    ps_nn = torch.sigmoid(torch.tensor(lg_nn)).numpy()
    lg_en = a_tr * lg_tr + a_nn * lg_nn
    ps_en = torch.sigmoid(torch.tensor(lg_en)).numpy()

    def metrics_from_pack(y, p, sw=None):
        return metrics_from(y, p, sample_weight=sw)

    if APPLY_WEIGHT_IN_TEST:
        sw_use = wt
    else:
        sw_use = None

    roc_tr, pr_tr, f1_tr = metrics_from_pack(yt, ps_tr, sw_use)
    roc_nn, pr_nn, f1_nn = metrics_from_pack(yt, ps_nn, sw_use)
    roc_en, pr_en, f1_en = metrics_from_pack(yt, ps_en, sw_use)

    print(f"✅ Fold{k} [Transformer] ROC {roc_tr:.4f} | PR {pr_tr:.4f} | F1 {f1_tr:.4f}")
    print(f"✅ Fold{k} [NNConv]      ROC {roc_nn:.4f} | PR {pr_nn:.4f} | F1 {f1_nn:.4f}")
    print(f"✅ Fold{k} [Ensemble]    ROC {roc_en:.4f} | PR {pr_en:.4f} | F1 {f1_en:.4f} (alpha={a_tr:.3f}/{a_nn:.3f})")

    # テストROC/PR図保存（重みは図にも反映可能）
    save_roc_pr_curves(yt, ps_tr,  f"Transformer_F{k}", fold_plot_dir, sample_weight=sw_use)
    save_roc_pr_curves(yt, ps_nn,  f"NNConv_F{k}",      fold_plot_dir, sample_weight=sw_use)
    save_roc_pr_curves(yt, ps_en,  f"Ensemble_F{k}",    fold_plot_dir, sample_weight=sw_use)

    res_tr.append([roc_tr, pr_tr, f1_tr])
    res_nn.append([roc_nn, pr_nn, f1_nn])
    res_en.append([roc_en, pr_en, f1_en])

# ----- 結果まとめ -----
def to_df(rows, name):
    arr = np.array(rows)
    df = pd.DataFrame({
        "Fold": [f"Fold{i}" for i in range(NUM_FOLDS)],
        "ROC-AUC": arr[:, 0],
        "PR-AUC":  arr[:, 1],
        "F1":      arr[:, 2]
    })
    df.index = df["Fold"]; df = df.drop(columns=["Fold"])
    df.loc["Mean"] = df.mean(numeric_only=True)
    df.loc["Std"]  = df.std(numeric_only=True)
    out = f"{SAVE_DIR}/cv_results_{name}.csv"
    df.to_csv(out, index=True)
    print(f"📁 Saved: {out}")
    print("🎯 平均結果:"); print(df.loc["Mean"])
    return df

df_tr = to_df(res_tr, "transformer")
df_nn = to_df(res_nn, "nnconv")
df_en = to_df(res_en, "ensemble")

rows_order = [f"Fold{i}" for i in range(NUM_FOLDS)] + ["Mean", "Std"]
def pick(df, col): return df.reindex(rows_order)[col].tolist()

df_comb = pd.DataFrame({
    "Fold": rows_order,
    "Transformer_ROC": pick(df_tr, "ROC-AUC"),
    "Transformer_PR":  pick(df_tr, "PR-AUC"),
    "Transformer_F1":  pick(df_tr, "F1"),
    "NNConv_ROC":      pick(df_nn, "ROC-AUC"),
    "NNConv_PR":       pick(df_nn, "PR-AUC"),
    "NNConv_F1":       pick(df_nn, "F1"),
    "Ensemble_ROC":    pick(df_en, "ROC-AUC"),
    "Ensemble_PR":     pick(df_en, "PR-AUC"),
    "Ensemble_F1":     pick(df_en, "F1"),
})
df_comb.to_csv(f"{SAVE_DIR}/cv_results_combined.csv", index=False)
print("\n✅ 完了:", SAVE_DIR)
print(f"ℹ️ 検証で重み適用: {APPLY_WEIGHT_IN_VAL} / テストで重み適用: {APPLY_WEIGHT_IN_TEST}")

✅ データ読み込み完了: 7321件
ラベル分布: {0: 5797, 1: 1524}
[INFO] 優先CIDヒット: 30 サンプル / 優先スキャフォールド: 5 種
[INFO] Dims: x=5, edge=6, g=7, r=10

📂 Fold 0 | train=4682 val=1327 test=1312 | moved_from_val=8 moved_from_test=7 | priority_in_val=0 priority_in_test=0
[Transformer/F0] pos_weight(eff)=2.674
[Transformer/F0] Epoch 001 | ROC-AUC 0.7379 | PR-AUC 0.3870 | F1 0.4255
[Transformer/F0] Epoch 002 | ROC-AUC 0.7217 | PR-AUC 0.3677 | F1 0.4484
[Transformer/F0] Epoch 003 | ROC-AUC 0.7409 | PR-AUC 0.3875 | F1 0.4620
[Transformer/F0] Epoch 004 | ROC-AUC 0.7603 | PR-AUC 0.4226 | F1 0.4387
[Transformer/F0] Epoch 005 | ROC-AUC 0.7478 | PR-AUC 0.3965 | F1 0.4620
[Transformer/F0] Epoch 006 | ROC-AUC 0.7640 | PR-AUC 0.4284 | F1 0.4728
[Transformer/F0] Epoch 007 | ROC-AUC 0.7665 | PR-AUC 0.4174 | F1 0.4983
[Transformer/F0] Epoch 008 | ROC-AUC 0.7642 | PR-AUC 0.4134 | F1 0.4718
[Transformer/F0] Epoch 009 | ROC-AUC 0.7528 | PR-AUC 0.3907 | F1 0.4533
[Transformer/F0] Epoch 010 | ROC-AUC 0.7595 | PR-AUC 0.4028 | F1 0.4724

In [ ]:
# ================================================
# LCO-CV (K=5) - TransformerConv & NNConv Ensemble (CID優先=元仕様)
#  - features: d.x + d.g + d.r (+ edge_attr)
#  - 陽性が少ない場合のみ SMILES ランダム化で上限付きオーバーサンプル
#  - 検証PR-AUC最大となる Ensemble 重み α を自動選択
#  - ★ 指定CIDのみ「必ず学習に含める」(val/testから除外) ＝ クラスタは固定しない
# --- 以下、加筆 ---
#  - ★ SampleWeight対応：
#      ・学習損失は必ずサンプル重みで加重（BCEWithLogitsLoss(reduction='none') → 重み付き平均）
#      ・pos_weight は重み付き実効比（neg_sum/pos_sum、上限10）で算出
#      ・検証/テストの ROC・PR・F1（図・数値）および α* 最適化（Val PR-AUC）にも重みを適用可
#      ・重み属性候補：["sample_weight","weight","w","SampleWeight","Weight"]（無ければ 1.0）
#      ・フラグ：APPLY_WEIGHT_IN_VAL（検証に重みを適用）、APPLY_WEIGHT_IN_TEST（テストに重みを適用）
# ================================================
import os, random, math, warnings as _warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit import RDLogger

from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, auc, average_precision_score, roc_curve
from torch.serialization import add_safe_globals

# --- quiet ---
RDLogger.DisableLog('rdApp.*')
_warnings.filterwarnings("ignore", category=DeprecationWarning)

# ----- 再現性 -----
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ----- 設定 -----
PT_PATH  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt"
SAVE_DIR = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_lco5_trans_nn_ens_posaug_CIDprior"
os.makedirs(SAVE_DIR, exist_ok=True)
DEVICE   = torch.device("cuda" if torch.cuda.is_available() else "cpu")

K          = 5
BATCH_TR   = 64
BATCH_TE   = 128
LR         = 3e-4
WD         = 1e-4
TMAX       = 20
PATIENCE   = 10
MAX_EPOCH  = 200
CLIP       = 5.0
ALPHAS     = np.linspace(0, 1, 11)   # 0,0.1,...,1.0

# --- 陽性オーバーサンプル方針 ---
AUG_TARGET_POS_RATE = 0.30
AUG_MAX_PER_POS     = 3

# ★ 学習へ常時含める CID（CID単位でtrainへ。クラスタ固定はしない）
PRIORITY_CIDS = {3348, 3955, 2724385, 10315094, 68770, 13342, 441074}

# ★ SampleWeight 設定（加筆）
WEIGHT_ATTR_CANDIDATES = ["sample_weight","weight","w","SampleWeight","Weight"]
APPLY_WEIGHT_IN_VAL  = True   # 検証の PR-AUC/ROC などに重み適用（α*最適化も重み付き）
APPLY_WEIGHT_IN_TEST = False  # テストの指標・図に重み適用（必要に応じて True）

# ===== Utils =====
def pr_auc_score(y, p, sample_weight=None):
    prec, rec, _ = precision_recall_curve(y, p, sample_weight=sample_weight)
    return auc(rec, prec)

def _get_weight_from_data(d):
    for nm in WEIGHT_ATTR_CANDIDATES:
        if hasattr(d, nm):
            v = getattr(d, nm)
            return float(v.item()) if torch.is_tensor(v) else float(v)
    return 1.0

def _to_1d(w):
    if torch.is_tensor(w):
        if w.dim() == 2 and w.size(1) == 1: return w.view(-1)
        return w.view(-1)
    return torch.tensor(w, dtype=torch.float32)

def get_batch_weights(batch, device=None):
    for nm in WEIGHT_ATTR_CANDIDATES:
        if hasattr(batch, nm):
            w = getattr(batch, nm)
            w = _to_1d(w).to(device or batch.y.device).float()
            return w, nm
    return torch.ones(batch.y.numel(), device=device or batch.y.device, dtype=torch.float32), "CONST1"

def pos_weight_from(dataset):
    ys = torch.tensor([float(d.y.item()) for d in dataset])
    ws = torch.tensor([_get_weight_from_data(d) for d in dataset], dtype=torch.float32)
    pos_sum = ws[ys == 1].sum()
    neg_sum = ws[ys == 0].sum()
    w = (neg_sum / torch.clamp(pos_sum, min=1e-8)).item()
    return torch.tensor([min(max(w, 1.0), 10.0)], dtype=torch.float32)

@torch.no_grad()
def eval_probs_model(m, ds, batch_size=BATCH_TE):
    if len(ds) == 0:
        return np.array([]), np.array([]), np.array([])
    m.eval(); ys, ps, ws = [], [], []
    for bt in DataLoader(ds, batch_size=batch_size, shuffle=False):
        bt = bt.to(DEVICE)
        ys.append(bt.y.detach().cpu())
        ps.append(torch.sigmoid(m(bt)).detach().cpu())
        w, _ = get_batch_weights(bt, DEVICE)
        ws.append(w.detach().cpu())
    return torch.cat(ys).numpy(), torch.cat(ps).numpy(), torch.cat(ws).numpy()

def best_alpha_for_prauc(y_val, p1, p2, alphas=ALPHAS, sample_weight=None):
    best_a, best_s = 0.5, -1.0
    for a in alphas:
        pv = a * p1 + (1 - a) * p2
        s = pr_auc_score(y_val, pv, sample_weight=sample_weight)
        if s > best_s:
            best_s, best_a = s, a
    return best_a, best_s

def infer_dim(t):
    if t.dim() == 1: return t.numel()
    if t.dim() in (2,3): return t.size(-1)
    raise RuntimeError(f"未知のテンソル次元: {t.size()}")

def ensure_priority_in_train(train_idx, val_idx, test_idx, data, priority_cids):
    """CID単位で優先サンプルをtrainへ強制移動（val/testから除外）"""
    pr_set = set(priority_cids)
    def is_pr(i): return int(getattr(data[i], "cid", -1)) in pr_set

    moved_from_val  = [i for i in val_idx  if is_pr(i)]
    moved_from_test = [i for i in test_idx if is_pr(i)]

    if moved_from_val or moved_from_test:
        print(f"[CID-PRIOR] move to train | from val: {len(moved_from_val)}, from test: {len(moved_from_test)}")

    train_idx = list(train_idx) + moved_from_val + moved_from_test
    val_idx   = [i for i in val_idx  if i not in moved_from_val]
    test_idx  = [i for i in test_idx if i not in moved_from_test]

    train_idx = sorted(set(train_idx))
    val_idx   = sorted(set(val_idx)   - set(train_idx))
    test_idx  = sorted(set(test_idx)  - set(train_idx) - set(val_idx))

    def contains_priority(indices):
        return any(int(getattr(data[i], "cid", -1)) in pr_set for i in indices)
    assert not contains_priority(val_idx),  "priority CID in VAL (should be TRAIN)"
    assert not contains_priority(test_idx), "priority CID in TEST (should be TRAIN)"

    return train_idx, val_idx, test_idx

# ----- データ読み込み（PyTorch 2.6 安全ロード対応） -----
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception as e:
        print(f"[WARN] add_safe_globals 失敗: {e}")
    try:
        obj = torch.load(path)  # default weights_only=True
    except Exception as e:
        print(f"[WARN] 安全ロード失敗（{e}）。weights_only=False で再試行（信頼ファイル前提）。")
        obj = torch.load(path, weights_only=False)

    if isinstance(obj, list):
        return obj
    if isinstance(obj, dict) and "data_list" in obj:
        return obj["data_list"]
    try:
        return list(obj)
    except Exception as e:
        raise RuntimeError(f"未知形式: {type(obj)}; list化失敗: {e}")

# ===== モデル =====
class Transformer_DX_DG_DR(nn.Module):
    def __init__(self, X_DIM, G_DIM, R_DIM, E_DIM, HID=256):
        super().__init__()
        self.c1 = TransformerConv(X_DIM, HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.c2 = TransformerConv(HID,  HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x.float(), g.float(), r.float()], dim=1)).view(-1)

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hidden = max(64, min(256, in_attr_dim * 8))
    return nn.Sequential(nn.Linear(in_attr_dim, hidden), nn.ReLU(),
                         nn.Linear(hidden, in_channels * out_channels))

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self, X_DIM, G_DIM, R_DIM, E_DIM, HID=256):
        super().__init__()
        edge_nn1 = make_edge_mlp(E_DIM, HID, X_DIM)
        edge_nn2 = make_edge_mlp(E_DIM, HID, HID)
        self.c1 = NNConv(X_DIM, HID, edge_nn1, aggr='mean')
        self.c2 = NNConv(HID,  HID, edge_nn2, aggr='mean')
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x.float(), g.float(), r.float()], dim=1)).view(-1)

# ===== メイン処理 =====
def main():
    # 1) データロード
    data = load_pyg_list(PT_PATH)
    if len(data) == 0:
        raise RuntimeError("data が空です。")
    print(f"✅ グラフ数: {len(data)}")

    # 2) 次元確認
    sample = data[0]
    X_DIM = sample.x.size(-1)
    if not hasattr(sample, "edge_attr") or sample.edge_attr is None:
        raise RuntimeError("edge_attr が必要です。")
    E_DIM = sample.edge_attr.size(-1)
    G_DIM = infer_dim(sample.g)
    R_DIM = infer_dim(sample.r)
    HID   = 256
    print(f"[INFO] Dims: x={X_DIM}, edge={E_DIM}, g={G_DIM}, r={R_DIM}")

    # 3) KMeansクラスタ（LCOベース）
    fps = []
    for d in data:
        m = Chem.MolFromSmiles(d.smiles)
        arr = np.zeros((2048,), dtype=np.float32)
        fp = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048) if m is not None else None
        if fp is None:
            fps.append(arr)
        else:
            DataStructs.ConvertToNumpyArray(fp, arr)
            fps.append(arr)

    km = KMeans(n_clusters=K, random_state=SEED, n_init=10).fit(fps)
    for d, c in zip(data, km.labels_):
        d.cluster = int(c)

    # 4) クラスタ → インデックス表
    cluster_to_indices = {cl: [] for cl in range(K)}
    for i, d in enumerate(data):
        cluster_to_indices[int(d.cluster)].append(i)

    # 5) LCO-CV（test=foldクラスタ、val=(fold+1)%K、train=残り）
    rows = []
    for fold in range(K):
        test_cls = fold
        val_cls  = (fold + 1) % K
        test_idx = list(cluster_to_indices[test_cls])
        val_idx  = list(cluster_to_indices[val_cls])
        train_idx = [i for cl, idxs in cluster_to_indices.items() if cl not in (test_cls, val_cls) for i in idxs]

        # --- ★ CID単位で優先サンプルをtrainへ強制移動（val/testから外す）
        train_idx, val_idx, test_idx = ensure_priority_in_train(train_idx, val_idx, test_idx, data, PRIORITY_CIDS)

        train_base = [data[i] for i in sorted(train_idx)]
        val_data   = [data[i] for i in sorted(val_idx)]
        test_data  = [data[i] for i in sorted(test_idx)]

        # ---- 陽性が少ないときのみオーバーサンプリング（trainのみ）----
        n_train = len(train_base)
        pos_cnt = sum(int(d.y) for d in train_base)
        neg_cnt = n_train - pos_cnt
        pos_rate = pos_cnt / max(1, n_train)
        target_pos = math.ceil(AUG_TARGET_POS_RATE * n_train)
        need_pos   = max(0, target_pos - pos_cnt)

        if pos_cnt > 0 and need_pos > 0:
            per_pos_aug = max(1, min(AUG_MAX_PER_POS, math.ceil(need_pos / pos_cnt)))
            train_aug = []
            for d in train_base:
                if int(d.y) == 1:
                    m = Chem.MolFromSmiles(d.smiles)
                    if m is None:
                        continue
                    for _ in range(per_pos_aug):
                        dd = d.clone()
                        # 重みは clone で引き継がれる想定（安全のためそのまま）
                        dd.smiles = Chem.MolToSmiles(m, doRandom=True)
                        train_aug.append(dd)
            train_data = train_base + train_aug
            print(f"🔁 Fold {fold}: pos-aug enabled | base_pos={pos_cnt}, neg={neg_cnt}, "
                  f"pos_rate={pos_rate:.3f} -> target={AUG_TARGET_POS_RATE:.2f}, "
                  f"per_pos_aug={per_pos_aug}, +aug={len(train_aug)}, new_train={len(train_data)}")
        else:
            train_data = train_base
            print(f"🔁 Fold {fold}: pos-aug skipped | base_pos={pos_cnt}, neg={neg_cnt}, pos_rate={pos_rate:.3f}")

        print(f"\n📂 Fold {fold} | train={len(train_data)} val={len(val_data)} test={len(test_data)}")

        # ---- 学習（1モデル） ----
        def train_one(model, train_set, val_set, save_path, tag):
            model.to(DEVICE)
            pw = pos_weight_from(train_set).to(DEVICE)
            crit = nn.BCEWithLogitsLoss(pos_weight=pw, reduction='none')  # per-sample
            opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
            sch  = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=TMAX)

            best_auc, wait = 0.0, 0
            for ep in range(1, MAX_EPOCH + 1):
                model.train()
                for bt in DataLoader(train_set, batch_size=BATCH_TR, shuffle=True):
                    bt = bt.to(DEVICE)
                    logits = model(bt)
                    loss_vec = crit(logits, bt.y.float())
                    sw, _nm = get_batch_weights(bt, DEVICE)
                    loss = (loss_vec * sw).sum() / (sw.sum() + 1e-8)

                    opt.zero_grad()
                    loss.backward()
                    nn.utils.clip_grad_norm_(model.parameters(), CLIP)
                    opt.step()
                sch.step()

                yv, pv, wv = eval_probs_model(model, val_set)
                if yv.size > 0:
                    if APPLY_WEIGHT_IN_VAL:
                        roc = roc_auc_score(yv, pv, sample_weight=wv)
                        prc = pr_auc_score(yv, pv, sample_weight=wv)
                    else:
                        roc = roc_auc_score(yv, pv)
                        prc = pr_auc_score(yv, pv)
                    f1  = f1_score(yv, (pv >= 0.5).astype(int))
                else:
                    roc = prc = f1 = 0.0
                print(f"[{tag}] Ep{ep:03d} | ROC={roc:.4f} PR={prc:.4f} F1={f1:.4f}")

                if roc > best_auc:
                    best_auc, wait = roc, 0
                    torch.save(model.state_dict(), save_path)
                else:
                    wait += 1
                    if wait >= PATIENCE:
                        print(f"✅ EarlyStopping [{tag}]")
                        break

            model.load_state_dict(torch.load(save_path, map_location=DEVICE))
            yv, pv, wv = eval_probs_model(model, val_set)
            return model, yv, pv, wv

        # --- 学習（Transformer / NNConv） ---
        m_tr = Transformer_DX_DG_DR(X_DIM, G_DIM, R_DIM, E_DIM, HID)
        m_nn = NNConv_DX_DG_DR(      X_DIM, G_DIM, R_DIM, E_DIM, HID)
        p_tr = os.path.join(SAVE_DIR, f"trans_fold{fold}.pt")
        p_nn = os.path.join(SAVE_DIR, f"nnconv_fold{fold}.pt")

        m_tr, yv, pv_tr, wv = train_one(m_tr, train_data, val_data, p_tr, f"Transformer/F{fold}")
        m_nn, _,  pv_nn, _  = train_one(m_nn, train_data, val_data, p_nn, f"NNConv/F{fold}")

        # --- 検証で α*: PR-AUC 最大（重み適用可） ---
        if yv.size == 0:
            alpha_star, pr_val = 0.5, 0.0
        else:
            alpha_star, pr_val = best_alpha_for_prauc(yv, pv_tr, pv_nn,
                                                      alphas=ALPHAS,
                                                      sample_weight=wv if APPLY_WEIGHT_IN_VAL else None)
        print(f"⭐ Fold{fold} best α={alpha_star:.2f} (Val PR-AUC={pr_val:.4f}{' [weighted]' if APPLY_WEIGHT_IN_VAL else ''})")

        # --- テスト ---
        yt, pt_tr, wt = eval_probs_model(m_tr, test_data)
        _,  pt_nn, wt2 = eval_probs_model(m_nn, test_data)
        if yt.size == 0:
            print(f"[WARN] Fold{fold}: test が空のため評価をスキップします。")
            continue
        assert np.allclose(wt, wt2)

        pt_en = alpha_star * pt_tr + (1 - alpha_star) * pt_nn

        def pack(prefix, y, p, sw=None):
            if sw is not None:
                return {
                    f"{prefix}_ROC": roc_auc_score(y, p, sample_weight=sw),
                    f"{prefix}_PR":  pr_auc_score(y, p, sample_weight=sw),
                    f"{prefix}_F1":  f1_score(y, (p >= 0.5).astype(int)),  # F1は表示用に非加重
                }
            else:
                return {
                    f"{prefix}_ROC": roc_auc_score(y, p),
                    f"{prefix}_PR":  pr_auc_score(y, p),
                    f"{prefix}_F1":  f1_score(y, (p >= 0.5).astype(int)),
                }

        sw_use = wt if APPLY_WEIGHT_IN_TEST else None
        rec = {"Fold": fold, "Alpha": float(alpha_star)}
        rec.update(pack("Transformer", yt, pt_tr, sw_use))
        rec.update(pack("NNConv",      yt, pt_nn, sw_use))
        rec.update(pack("Ensemble",    yt, pt_en, sw_use))
        rows.append(rec)

        print(f"✅ Fold{fold} [Transformer] ROC={rec['Transformer_ROC']:.4f} PR={rec['Transformer_PR']:.4f} F1={rec['Transformer_F1']:.4f}")
        print(f"✅ Fold{fold} [NNConv]      ROC={rec['NNConv_ROC']:.4f} PR={rec['NNConv_PR']:.4f} F1={rec['NNConv_F1']:.4f}")
        print(f"✅ Fold{fold} [Ensemble]    ROC={rec['Ensemble_ROC']:.4f} PR={rec['Ensemble_PR']:.4f} F1={rec['Ensemble_F1']:.4f} (α={alpha_star:.2f})")

    # 6) 保存・概要
    if len(rows) == 0:
        print("\n⚠️ すべてのfoldが評価スキップとなりました。")
        return

    df = pd.DataFrame(rows)
    out_csv = os.path.join(SAVE_DIR, "lco_cv_metrics_trans_nn_ensemble_CIDprior.csv")
    df.to_csv(out_csv, index=False)

    metrics_cols = [c for c in df.columns if c not in ["Fold", "Alpha"]]
    mean_row = df[metrics_cols].mean().to_dict()
    std_row  = df[metrics_cols].std().to_dict()
    print("\n🎉 LCO-CV 完了！")
    print("平均値:", {k: float(v) for k, v in mean_row.items()})
    print("標準偏差:", {k: float(v) for k, v in std_row.items()})
    print("α*", df["Alpha"].round(2).tolist())
    print(f"📁 Saved: {out_csv}")
    print(f"ℹ️ 検証で重み適用: {APPLY_WEIGHT_IN_VAL} / テストで重み適用: {APPLY_WEIGHT_IN_TEST}")

if __name__ == "__main__":
    main()

✅ グラフ数: 7321
[INFO] Dims: x=5, edge=6, g=7, r=10
🔁 Fold 0: pos-aug enabled | base_pos=1482, neg=4922, pos_rate=0.231 -> target=0.30, per_pos_aug=1, +aug=1482, new_train=7886

📂 Fold 0 | train=7886 val=632 test=285
[Transformer/F0] Ep001 | ROC=0.5966 PR=0.0766 F1=0.1379
[Transformer/F0] Ep002 | ROC=0.6960 PR=0.1032 F1=0.1805
[Transformer/F0] Ep003 | ROC=0.7103 PR=0.1116 F1=0.1287
[Transformer/F0] Ep004 | ROC=0.7057 PR=0.1166 F1=0.1524
[Transformer/F0] Ep005 | ROC=0.7071 PR=0.1147 F1=0.1720
[Transformer/F0] Ep006 | ROC=0.7371 PR=0.1231 F1=0.1441
[Transformer/F0] Ep007 | ROC=0.7483 PR=0.1344 F1=0.1784
[Transformer/F0] Ep008 | ROC=0.7625 PR=0.1366 F1=0.1853
[Transformer/F0] Ep009 | ROC=0.8113 PR=0.1337 F1=0.2025
[Transformer/F0] Ep010 | ROC=0.8243 PR=0.1367 F1=0.1942
[Transformer/F0] Ep011 | ROC=0.8336 PR=0.1393 F1=0.2335
[Transformer/F0] Ep012 | ROC=0.8558 PR=0.1496 F1=0.2532
[Transformer/F0] Ep013 | ROC=0.8717 PR=0.1642 F1=0.2419
[Transformer/F0] Ep014 | ROC=0.8385 PR=0.1369 F1=0.1923
[T

In [ ]:
# ================================================
# LCO-CV with Advanced Clustering & Fold Constraints
#  - clustering: Butina(Tanimoto) / K-Medoids(Tanimoto) / Murcko Scaffold(+subcluster)
#  - fold constraints: minimum positives/negatives per fold (with relaxation)
#  - models: TransformerConv & NNConv, PR-AUC-based early stopping
#  - ensemble alpha chosen to maximize validation PR-AUC
# --- 以下、加筆 ---
#  - ★ SampleWeight対応：
#      ・学習損失はサンプル重みで加重（BCEWithLogitsLoss(reduction='none') → 重み付き平均）
#      ・pos_weight は重み付き実効比（neg_sum/pos_sum、上限10）で算出
#      ・検証/テストの PR-AUC・ROC（数値）および α* 最適化（Val PR-AUC）にも重み適用可
#      ・重み属性候補：["sample_weight","weight","w","SampleWeight","Weight"]（無ければ 1.0）
#      ・フラグ：APPLY_WEIGHT_IN_VAL（検証に重み）、APPLY_WEIGHT_IN_TEST（テストに重み）
# ================================================
import os, random, math, gc
import numpy as np
import pandas as pd
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
from torch.serialization import add_safe_globals

from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score, auc

# RDKit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina

# ====== 基本設定 ======
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

PT_PATH   = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/data_graph_with_smiles.pt"
SAVE_DIR  = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/results_lco5_trans_nn_ens_posaug_advanced"
os.makedirs(SAVE_DIR, exist_ok=True)
DEVICE    = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- CV / 学習設定 ----
K                  = 5
BATCH_TR           = 64
BATCH_TE           = 128
LR                 = 3e-4
WD                 = 1e-4
PATIENCE           = 10
MAX_EPOCH          = 200
CLIP               = 5.0
ALPHAS             = np.linspace(0,1,11)   # 0.0, 0.1, ..., 1.0

# ---- オーバーサンプリング（陽性のみ複製）----
AUG_TARGET_POS_RATE = 0.30
AUG_MAX_PER_POS     = 3

# ---- クラスタリング方式 ----
# "butina" | "kmedoids" | "scaffold"
CLUSTER_METHOD   = "butina"
BUTINA_THRESH    = 0.7      # tanimoto >= 0.7 を同クラスタ
KMEDOIDS_ITERS   = 50
SUBCLUSTER_MIN   = 10
SUBCLUSTER_K_MAX = 5

# ---- Fold 充足制約 ----
MIN_POS_PER_FOLD = 10
MIN_NEG_PER_FOLD = 10
FOLD_ASSIGN_TRIALS = 200

# ★ SampleWeight 設定（加筆）
WEIGHT_ATTR_CANDIDATES = ["sample_weight","weight","w","SampleWeight","Weight"]
APPLY_WEIGHT_IN_VAL  = True   # Val指標・α*最適化を重み付きで
APPLY_WEIGHT_IN_TEST = False  # Test指標を重み付きで出す場合は True

# ====== ユーティリティ（重み対応） ======
def pr_auc_score(y, p, sample_weight=None):
    prec, rec, _ = precision_recall_curve(y, p, sample_weight=sample_weight)
    return auc(rec, prec)

def _get_weight_from_data(d):
    for nm in WEIGHT_ATTR_CANDIDATES:
        if hasattr(d, nm):
            v = getattr(d, nm)
            return float(v.item()) if torch.is_tensor(v) else float(v)
    return 1.0

def _to_1d(w):
    if torch.is_tensor(w):
        if w.dim() == 2 and w.size(1) == 1: return w.view(-1)
        return w.view(-1)
    return torch.tensor(w, dtype=torch.float32)

def get_batch_weights(batch, device=None):
    for nm in WEIGHT_ATTR_CANDIDATES:
        if hasattr(batch, nm):
            w = getattr(batch, nm)
            w = _to_1d(w).to(device or batch.y.device).float()
            return w, nm
    return torch.ones(batch.y.numel(), device=device or batch.y.device, dtype=torch.float32), "CONST1"

def pos_weight_from(dataset):
    ys = torch.tensor([float(d.y.item()) for d in dataset])
    ws = torch.tensor([_get_weight_from_data(d) for d in dataset], dtype=torch.float32)
    pos_sum = ws[ys == 1].sum()
    neg_sum = ws[ys == 0].sum()
    w = (neg_sum / torch.clamp(pos_sum, min=1e-8)).item()
    return torch.tensor([min(max(w, 1.0), 10.0)], dtype=torch.float32)

@torch.no_grad()
def eval_probs(model, ds, batch_size=BATCH_TE):
    model.eval(); ys, ps, ws = [], [], []
    for bt in DataLoader(ds, batch_size=batch_size, shuffle=False):
        bt = bt.to(DEVICE)
        ys.append(bt.y.detach().cpu())
        ps.append(torch.sigmoid(model(bt)).detach().cpu())
        w, _ = get_batch_weights(bt, DEVICE)
        ws.append(w.detach().cpu())
    return torch.cat(ys).numpy(), torch.cat(ps).numpy(), torch.cat(ws).numpy()

def infer_dim(t):
    if t.dim() == 1: return t.numel()
    if t.dim() in (2,3): return t.size(-1)
    raise RuntimeError(f"未知のテンソル次元: {t.size()}")

def best_alpha_for_prauc(y_val, p1, p2, alphas=ALPHAS, sample_weight=None):
    best_a, best_s = 0.5, -1.0
    for a in alphas:
        s = pr_auc_score(y_val, a*p1 + (1-a)*p2, sample_weight=sample_weight)
        if s > best_s:
            best_s, best_a = s, a
    return best_a, best_s

# ====== モデル定義 ======
def build_models_dims(sample):
    X_DIM = sample.x.size(-1)
    G_DIM = infer_dim(sample.g)
    R_DIM = infer_dim(sample.r)
    assert hasattr(sample, "edge_attr") and sample.edge_attr is not None, "edge_attr が必要です。"
    E_DIM = sample.edge_attr.size(-1)
    HID   = 256
    return X_DIM, G_DIM, R_DIM, E_DIM, HID

class Transformer_DX_DG_DR(nn.Module):
    def __init__(self, X_DIM, G_DIM, R_DIM, E_DIM, HID=256):
        super().__init__()
        self.c1 = TransformerConv(X_DIM, HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.c2 = TransformerConv(HID,  HID, heads=4, concat=False, edge_dim=E_DIM, dropout=0.0, beta=False)
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x, g, r], dim=1)).view(-1)

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hidden = max(64, min(256, in_attr_dim * 8))
    return nn.Sequential(nn.Linear(in_attr_dim, hidden), nn.ReLU(),
                         nn.Linear(hidden, in_channels * out_channels))

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self, X_DIM, G_DIM, R_DIM, E_DIM, HID=256):
        super().__init__()
        edge_nn1 = make_edge_mlp(E_DIM, HID, X_DIM)
        edge_nn2 = make_edge_mlp(E_DIM, HID, HID)
        self.c1 = NNConv(X_DIM, HID, edge_nn1, aggr='mean')
        self.c2 = NNConv(HID,  HID, edge_nn2, aggr='mean')
        self.pool = AttentionalAggregation(gate_nn=nn.Linear(HID, 1))
        self.drop = nn.Dropout(0.2)
        self.fc = nn.Linear(HID + G_DIM + R_DIM, 1)
    def forward(self, d):
        x = F.relu(self.c1(d.x, d.edge_index, d.edge_attr))
        x = F.relu(self.c2(x, d.edge_index, d.edge_attr))
        x = self.pool(self.drop(x), d.batch)
        g = d.g.squeeze(1) if d.g.dim() == 3 else d.g
        r = d.r.squeeze(1) if d.r.dim() == 3 else d.r
        return self.fc(torch.cat([x, g, r], dim=1)).view(-1)

def train_one(model, train_set, val_set, save_path, tag):
    model.to(DEVICE)
    crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight_from(train_set).to(DEVICE),
                                reduction='none')  # per-sample
    opt  = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    best_pr, wait = -1.0, 0
    for ep in range(1, MAX_EPOCH+1):
        model.train()
        for bt in DataLoader(train_set, batch_size=BATCH_TR, shuffle=True):
            bt = bt.to(DEVICE)
            logits  = model(bt)
            lossvec = crit(logits, bt.y.float())
            sw, _nm = get_batch_weights(bt, DEVICE)
            loss = (lossvec * sw).sum() / (sw.sum() + 1e-8)

            opt.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), CLIP)
            opt.step()

        yv, pv, wv = eval_probs(model, val_set)
        prc = pr_auc_score(yv, pv, sample_weight=wv if APPLY_WEIGHT_IN_VAL else None)
        roc = roc_auc_score(yv, pv, sample_weight=wv if APPLY_WEIGHT_IN_VAL else None)
        f1  = f1_score(yv, (pv>=0.5).astype(int))  # 表示用に非加重
        print(f"[{tag}] Ep{ep:03d} | PR={prc:.4f} ROC={roc:.4f} F1@0.5={f1:.4f}")

        if prc > best_pr:
            best_pr, wait = prc, 0
            torch.save(model.state_dict(), save_path)
        else:
            wait += 1
            if wait >= PATIENCE:
                print(f"✅ EarlyStopping (best Val PR-AUC={best_pr:.4f}) [{tag}]")
                break

    model.load_state_dict(torch.load(save_path, map_location=DEVICE))
    return model, *eval_probs(model, val_set)

# ====== ECFP4 & Tanimoto ======
def mol_from_smiles(s):
    try:
        return Chem.MolFromSmiles(s) if s is not None else None
    except:
        return None

def fp_from_mol(m):
    if m is None:
        # 長さ2048のゼロベクトルを代用（類似度=0）
        return AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(""), 2, nBits=2048)
    return AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)

def tanimoto(a, b):
    return DataStructs.TanimotoSimilarity(a, b)

# ====== クラスタリング ======
def cluster_butina(fps, threshold=0.7):
    dists = []
    for i in range(1, len(fps)):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i], fps[:i])
        dists.extend([1.0 - float(s) for s in sims])
    cs = Butina.ClusterData(dists, len(fps), 1.0 - float(threshold), isDistData=True)
    return [list(c) for c in cs]

def kmedoids_tanimoto(fps, k, max_iters=50, seed=SEED):
    rng = np.random.RandomState(seed)
    n = len(fps)
    medoids = rng.choice(n, size=min(k, n), replace=False).tolist()
    if len(medoids) < k: k = len(medoids)

    def assign(meds):
        assigns = np.empty(n, dtype=np.int64)
        for i in range(n):
            best_j, best_d = 0, 1e9
            for j, m in enumerate(meds):
                d = 1.0 - tanimoto(fps[i], fps[m])
                if d < best_d: best_d, best_j = d, j
            assigns[i] = best_j
        return assigns

    def total_cost(assigns, meds):
        return sum(1.0 - tanimoto(fps[i], meds[assigns[i]]) for i in range(n))

    assigns = assign(medoids)
    best_cost = total_cost(assigns, medoids)
    it, improved = 0, True
    while improved and it < max_iters:
        improved, it = False, it+1
        for mi in range(k):
            for cand in range(n):
                if cand in medoids: continue
                new_meds = medoids.copy(); new_meds[mi] = cand
                new_assigns = assign(new_meds)
                cost = total_cost(new_assigns, new_meds)
                if cost + 1e-9 < best_cost:
                    medoids, assigns, best_cost = new_meds, new_assigns, cost
                    improved = True
    clusters = [[] for _ in range(k)]
    for idx, a in enumerate(assigns): clusters[a].append(idx)
    return [c for c in clusters if len(c) > 0]

def scaffold_key(mol):
    try:
        scaf = MurckoScaffold.GetScaffoldForMol(mol)
        return Chem.MolToSmiles(scaf, isomericSmiles=False) if scaf is not None else None
    except:
        return None

def cluster_scaffold_then_subcluster(smiles_list, fps, sub_min=10, sub_kmax=5):
    sc_groups = defaultdict(list)
    for i, smi in enumerate(smiles_list):
        key = scaffold_key(mol_from_smiles(smi))
        if key is None: key = f"_NOSCAF_{i}"
        sc_groups[key].append(i)

    clusters = []
    for _, idxs in sc_groups.items():
        if len(idxs) >= sub_min:
            k = max(1, min(sub_kmax, int(math.ceil(len(idxs)/sub_min))))
            sub_fps = [fps[i] for i in idxs]
            sub_clusters = kmedoids_tanimoto(sub_fps, k=k, max_iters=KMEDOIDS_ITERS, seed=SEED)
            for sc in sub_clusters:
                clusters.append([idxs[t] for t in sc])
        else:
            clusters.append(idxs)
    return clusters

# ====== fold 割当（制約付き） ======
def summarize_cluster_labels(data, clusters):
    rows = []
    for cid, idxs in enumerate(clusters):
        ys = [int(data[i].y) for i in idxs]
        rows.append({"cid":cid, "size":len(idxs), "pos":sum(ys), "neg":len(ys)-sum(ys)})
    return pd.DataFrame(rows)

def try_pack_clusters(clusters, labels_df, k=K, min_pos=MIN_POS_PER_FOLD, min_neg=MIN_NEG_PER_FOLD, trials=FOLD_ASSIGN_TRIALS):
    sizes = labels_df.set_index("cid")["size"].to_dict()
    pos_c = labels_df.set_index("cid")["pos"].to_dict()
    neg_c = labels_df.set_index("cid")["neg"].to_dict()
    cluster_ids = labels_df.sort_values("size", ascending=False)["cid"].tolist()

    best_solution, best_score = None, (1e9, 1e9)  # (violations, size_var)

    for _ in range(trials):
        folds = [{"cids":[], "pos":0, "neg":0, "size":0} for _ in range(k)]
        for cid in cluster_ids:
            best_fold, best_tuple = None, (1e9, 1e9)
            for f in range(k):
                p = folds[f]["pos"]  + pos_c[cid]
                n = folds[f]["neg"]  + neg_c[cid]
                s = folds[f]["size"] + sizes[cid]
                viol = int(p < min_pos) + int(n < min_neg)
                score = (viol, s)
                if score < best_tuple:
                    best_tuple, best_fold = score, f
            f = best_fold
            folds[f]["cids"].append(cid)
            folds[f]["pos"]  += pos_c[cid]
            folds[f]["neg"]  += neg_c[cid]
            folds[f]["size"] += sizes[cid]

        viol = sum(int(fd["pos"] < min_pos or fd["neg"] < min_neg) for fd in folds)
        sz = np.array([fd["size"] for fd in folds], dtype=float)
        size_var = float(np.var(sz))
        score = (viol, size_var)
        if score < best_score:
            best_score, best_solution = score, folds
            if viol == 0: break
    return best_solution, best_score

def assign_folds_with_relaxation(data, clusters, k=K, min_pos=MIN_POS_PER_FOLD, min_neg=MIN_NEG_PER_FOLD):
    labels_df = summarize_cluster_labels(data, clusters)
    folds, score = try_pack_clusters(clusters, labels_df, k=k, min_pos=min_pos, min_neg=min_neg)
    viol, size_var = score

    relax_steps = 0
    mpos, mneg = min_pos, min_neg
    while viol > 0 and (mpos > 0 or mneg > 0):
        relax_steps += 1
        if mpos > 0: mpos -= 1
        if mneg > 0: mneg -= 1
        folds2, score2 = try_pack_clusters(clusters, labels_df, k=k, min_pos=mpos, min_neg=mneg)
        if score2 < score:
            folds, score = folds2, score2
            viol, size_var = score
        if viol == 0: break

    node2fold = {}
    for f, info in enumerate(folds):
        for cid in info["cids"]:
            for idx in clusters[cid]:
                node2fold[idx] = f

    recs = [{"Fold": f, "size": fd["size"], "pos": fd["pos"], "neg": fd["neg"]} for f, fd in enumerate(folds)]
    fold_summary = pd.DataFrame(recs)
    labels_df["fold"] = -1
    for f, info in enumerate(folds):
        for cid in info["cids"]:
            labels_df.loc[labels_df["cid"]==cid, "fold"] = f

    return node2fold, fold_summary, labels_df, {
        "violations": int(viol),
        "size_var": float(size_var),
        "relax_steps": int(relax_steps),
        "min_pos_final": int(mpos),
        "min_neg_final": int(mneg)
    }

# ====== 安全ロード ======
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception as e:
        print(f"[WARN] add_safe_globals 失敗: {e}")
    try:
        obj = torch.load(path)  # PyTorch 2.6: weights_only=True 既定
    except Exception as e:
        print(f"[WARN] 安全ロード失敗（{e}）。weights_only=Falseで再試行（信頼ファイル前提）。")
        obj = torch.load(path, weights_only=False)

    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    try: return list(obj)
    except Exception as e:
        raise RuntimeError(f"未知形式: {type(obj)}; list化失敗: {e}")

# ====== メイン ======
def main():
    # 1) データロード
    data = load_pyg_list(PT_PATH)
    print(f"✅ グラフ数: {len(data)}")
    sample = data[0]
    X_DIM, G_DIM, R_DIM, E_DIM, HID = build_models_dims(sample)

    # 2) SMILES / mol / fp
    smiles = [getattr(d, "smiles", None) for d in data]
    mols   = [mol_from_smiles(s) for s in smiles]
    fps    = [fp_from_mol(m) for m in mols]

    # 3) クラスタリング
    cm = CLUSTER_METHOD.lower()
    if cm == "butina":
        clusters = cluster_butina(fps, threshold=BUTINA_THRESH)
        method_tag = f"butina_t{BUTINA_THRESH}"
    elif cm == "kmedoids":
        clusters = kmedoids_tanimoto(fps, k=K, max_iters=KMEDOIDS_ITERS, seed=SEED)
        method_tag = f"kmedoids_k{K}_it{KMEDOIDS_ITERS}"
    elif cm == "scaffold":
        clusters = cluster_scaffold_then_subcluster(smiles, fps, sub_min=SUBCLUSTER_MIN, sub_kmax=SUBCLUSTER_K_MAX)
        method_tag = f"scaffold_sub{SUBCLUSTER_K_MAX}"
    else:
        raise ValueError("CLUSTER_METHOD は 'butina' | 'kmedoids' | 'scaffold'")

    # K未満ならマージ
    while len(clusters) < K:
        clusters = sorted(clusters, key=len)
        a, b = clusters.pop(0), clusters.pop(0)
        clusters.append(a + b)
    print(f"🔧 Clusters={len(clusters)} (method={method_tag})")

    # 4) 制約付き fold 割当
    node2fold, fold_summary, cluster_labels, extra = assign_folds_with_relaxation(
        data, clusters, k=K, min_pos=MIN_POS_PER_FOLD, min_neg=MIN_NEG_PER_FOLD
    )
    print("📊 Fold summary:\n", fold_summary)
    print("⚠️ Constraint:", extra)

    cluster_labels.to_csv(os.path.join(SAVE_DIR, f"clusters_{method_tag}.csv"), index=False)
    fold_summary.to_csv(os.path.join(SAVE_DIR, f"fold_summary_{method_tag}.csv"), index=False)

    for i, d in enumerate(data):
        d.cluster = int(node2fold[i])

    # 5) LCO-CV
    rows = []
    for fold in range(K):
        test_idx = [i for i,d in enumerate(data) if d.cluster==fold]
        rest_idx = [i for i,d in enumerate(data) if d.cluster!=fold]

        rng = np.random.default_rng(SEED + fold)
        rng.shuffle(rest_idx)

        n_val = max(1, int(0.1 * len(rest_idx)))
        val_idx   = rest_idx[:n_val]
        train_idx = rest_idx[n_val:]

        test_data  = [data[i] for i in test_idx]
        val_data   = [data[i] for i in val_idx]
        train_base = [data[i] for i in train_idx]

        # ---- 陽性オーバーサンプリング（学習のみ）----
        n_train = len(train_base)
        pos_cnt = sum(int(d.y) for d in train_base)
        neg_cnt = n_train - pos_cnt
        pos_rate = pos_cnt / max(1, n_train)
        target_pos = math.ceil(AUG_TARGET_POS_RATE * n_train)
        need_pos   = max(0, target_pos - pos_cnt)

        if pos_cnt > 0 and need_pos > 0:
            per_pos_aug = max(1, min(AUG_MAX_PER_POS, math.ceil(need_pos / pos_cnt)))
            train_aug = []
            for d in train_base:
                if int(d.y)==1:
                    for _ in range(per_pos_aug):
                        dd = d.clone()  # 重みは clone で引き継がれる想定
                        train_aug.append(dd)
            train_data = train_base + train_aug
            print(f"🔁 Fold {fold}: pos-oversample | base_pos={pos_cnt}, neg={neg_cnt}, "
                  f"pos_rate={pos_rate:.3f} -> target={AUG_TARGET_POS_RATE:.2f}, "
                  f"per_pos_aug={per_pos_aug}, +aug={len(train_aug)}, new_train={len(train_data)}")
        else:
            train_data = train_base
            print(f"🔁 Fold {fold}: oversample skipped | base_pos={pos_cnt}, neg={neg_cnt}, pos_rate={pos_rate:.3f}")

        print(f"\n📂 Fold {fold} | train={len(train_data)} val={len(val_data)} test={len(test_data)}")

        # ---- モデル ----
        m_tr = Transformer_DX_DG_DR(X_DIM, G_DIM, R_DIM, E_DIM, HID)
        m_nn = NNConv_DX_DG_DR(      X_DIM, G_DIM, R_DIM, E_DIM, HID)
        p_tr = os.path.join(SAVE_DIR, f"{method_tag}_trans_fold{fold}.pt")
        p_nn = os.path.join(SAVE_DIR, f"{method_tag}_nnconv_fold{fold}.pt")

        # ---- 学習（PR-AUCで保存；重み対応）----
        m_tr, yv, pv_tr, wv = train_one(m_tr, train_data, val_data, p_tr, f"Transformer/F{fold}")
        m_nn, _,  pv_nn, _  = train_one(m_nn, train_data, val_data, p_nn, f"NNConv/F{fold}")

        # ---- α*: Val PR 最大（重み適用可）----
        alpha_star, pr_val = best_alpha_for_prauc(
            yv, pv_tr, pv_nn, alphas=ALPHAS, sample_weight=wv if APPLY_WEIGHT_IN_VAL else None
        )
        print(f"⭐ Fold{fold} best α={alpha_star:.2f} (Val PR-AUC={pr_val:.4f}{' [weighted]' if APPLY_WEIGHT_IN_VAL else ''})")

        # ---- テスト（重み適用はフラグで）----
        yt, pt_tr, wt = eval_probs(m_tr, test_data)
        _,  pt_nn, wt2 = eval_probs(m_nn, test_data)
        pt_en = alpha_star * pt_tr + (1 - alpha_star) * pt_nn
        sw_use = wt if APPLY_WEIGHT_IN_TEST else None

        def pack(prefix, y, p, sw=None):
            return {
                f"{prefix}_ROC": roc_auc_score(y, p, sample_weight=sw) if sw is not None else roc_auc_score(y, p),
                f"{prefix}_PR":  pr_auc_score(y, p, sample_weight=sw),
                f"{prefix}_F1":  f1_score(y, (p>=0.5).astype(int)),  # 表示用に非加重
            }

        rec = {"Fold": fold, "Alpha": float(alpha_star), "Method": CLUSTER_METHOD}
        rec.update(pack("Transformer", yt, pt_tr, sw_use))
        rec.update(pack("NNConv",      yt, pt_nn, sw_use))
        rec.update(pack("Ensemble",    yt, pt_en, sw_use))
        rows.append(rec)

        print(f"✅ Fold{fold} [Transformer] ROC={rec['Transformer_ROC']:.4f} PR={rec['Transformer_PR']:.4f} F1={rec['Transformer_F1']:.4f}")
        print(f"✅ Fold{fold} [NNConv]      ROC={rec['NNConv_ROC']:.4f} PR={rec['NNConv_PR']:.4f} F1={rec['NNConv_F1']:.4f}")
        print(f"✅ Fold{fold} [Ensemble]    ROC={rec['Ensemble_ROC']:.4f} PR={rec['Ensemble_PR']:.4f} F1={rec['Ensemble_F1']:.4f}")

        del m_tr, m_nn; gc.collect()
        if torch.cuda.is_available(): torch.cuda.empty_cache()

    # 6) 保存・集計
    df = pd.DataFrame(rows)
    out_csv = os.path.join(SAVE_DIR, f"lco_cv_metrics_{method_tag}.csv")
    df.to_csv(out_csv, index=False)

    metrics_cols = [c for c in df.columns if c not in ["Fold", "Alpha", "Method"]]
    mean_row = df[metrics_cols].mean().to_dict()
    std_row  = df[metrics_cols].std().to_dict()
    print("\n🎉 LCO-CV 完了！")
    print("平均値:", {k: float(v) for k, v in mean_row.items()})
    print("標準偏差:", {k: float(v) for k, v in std_row.items()})
    print("α*", df["Alpha"].round(2).tolist())
    print("保存:", out_csv)

# ====== 実行 ======
if __name__ == "__main__":
    from rdkit import RDLogger, Chem as _Chem
    import warnings as _warnings
    RDLogger.DisableLog('rdApp.*')
    _warnings.filterwarnings("ignore", category=DeprecationWarning)
    main()

✅ グラフ数: 7321
🔧 Clusters=4906 (method=butina_t0.7)
📊 Fold summary:
    Fold  size  pos   neg
0     0  1465  246  1219
1     1  1464  324  1140
2     2  1464  318  1146
3     3  1464  318  1146
4     4  1464  318  1146
⚠️ Constraint: {'violations': 0, 'size_var': 0.16000000000000006, 'relax_steps': 7, 'min_pos_final': 3, 'min_neg_final': 3}
🔁 Fold 0: pos-oversample | base_pos=1163, neg=4108, pos_rate=0.221 -> target=0.30, per_pos_aug=1, +aug=1163, new_train=6434

📂 Fold 0 | train=6434 val=585 test=1465
[Transformer/F0] Ep001 | PR=0.4480 ROC=0.7865 F1@0.5=0.2727
[Transformer/F0] Ep002 | PR=0.4465 ROC=0.7883 F1@0.5=0.5232
[Transformer/F0] Ep003 | PR=0.4697 ROC=0.8044 F1@0.5=0.5000
[Transformer/F0] Ep004 | PR=0.5083 ROC=0.8238 F1@0.5=0.5000
[Transformer/F0] Ep005 | PR=0.5203 ROC=0.8230 F1@0.5=0.5263
[Transformer/F0] Ep006 | PR=0.5421 ROC=0.8347 F1@0.5=0.5040
[Transformer/F0] Ep007 | PR=0.5485 ROC=0.8434 F1@0.5=0.4952
[Transformer/F0] Ep008 | PR=0.5523 ROC=0.8497 F1@0.5=0.5225
[Transformer/F

モデル定義

| モデルID | 名称/目的                                            | 入力特徴                                 | 学習/分割                                                                                                                                                                              | 早期停止・最適化                                                                                                                           | アンサンブル重み                                                              | データ入出力                                                                                                                                    |
| ----- | ------------------------------------------------ | ------------------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------- |
| M1    | TransformerConv + NNConv アンサンブル（標準5-fold）        | `d.x + d.g + d.r` ＋ `edge_attr`（PyG） | Stratified 5-fold CV                                                                                                                                                               | BCEWithLogits(+`pos_weight`<10) / AdamW(LR=3e-4, WD=1e-4) / CosineAnnealing(T_max=10) / 勾配クリップ=5 / **Val ROC-AUC**で早停（patience=10） | **各foldの検証PR-AUCで正規化した係数**で**ロジット**加重（Transformer/NNConv）             | 入力：`/.../data_graph_with_smiles.pt` 出力：`/.../results_ens_trans_nn_5fold/`（各foldモデル・履歴CSV・ROC/PR図、集計CSV3種）                                 |
| M2    | TransformerConv + NNConv アンサンブル（Scaffold 5-fold） | 同上                                   | **Murcko Scaffold 5-fold**（独自split）                                                                                                                                                | AdamW(LR=3e-4, WD=1e-4) / Cosine(T_max=20) / クリップ=5 / **Val ROC-AUC**で早停（patience=15）                                              | **検証PR-AUCを重み**として**ロジット**加重（foldごとにwを正規化）                            | 入力：`/.../data_graph_with_smiles.pt` 出力：`/.../results_ens_trans_nn_scaffold5fold/`（fold別モデル・学習曲線・ROC/PR図・結果CSV）                            |
| M3    | TransformerConv + NNConv アンサンブル（**LCO-CV** 基本形）  | 同上                                   | **LCO-CV K=5**（KMeans(ECFP4)クラスタ）/ **陽性が少ない時のみ** SMILESランダム化で**陽性オーバーサンプル**（上限3/分子、目標陽性率0.30）                                                                                      | AdamW(LR=3e-4, WD=1e-4) / Cosine(T_max=20) / クリップ=5 / **Val ROC-AUC**で早停（patience=10）                                              | **Val PR-AUC最大化**となるα∈{0,0.1,…,1}を探索し、**確率**をαで線形結合（テストも**確率**を同αで合成） | 入力：`/.../data_graph_with_smiles.pt` 出力：`/.../results_lco5_trans_nn_ens_posaug/`（`lco_cv_metrics_trans_nn_ensemble.csv` ほか）                |
| M4    | **Advanced LCO-CV**（高次クラスタ＋制約付きfold割当）           | 同上                                   | **クラスタ方式を選択**：Butina(Tanimoto≥0.7) / K-Medoids(Tanimoto) / Murcko+サブクラスタ（min=10, k≤5）。**fold制約**：各foldの最小Pos/Neg（デフォ10/10）を満たすようクラスタ梱包＋段階的緩和。オーバーサンプルは**陽性の単純複製**（SMILESランダム化はしない） | AdamW(LR=3e-4, WD=1e-4) / **Val PR-AUCで早停**（patience=10） / クリップ=5                                                                  | **Val PR-AUC最大**となるαを探索し、**確率**をαで合成（テストも同α）                          | 入力：`/.../data_graph_with_smiles.pt` 出力：`/.../results_lco5_trans_nn_ens_posaug_advanced/`（クラスタ割当CSV・foldサマリ・`lco_cv_metrics_{method}.csv`） |

In [ ]:
# ===== External Evaluation (ver7; uses provided 'tier' column & drops unknown rows) =====
# - external10_union.csv の 'tier' と 'y' をそのまま使用（unknownは除外）
# - KEGG-only 側は従来通り対応（tier が無ければ weak_pos 推定）
# - 出力: external_eval_comparison_all.csv / external_eval_ranking.csv
# - 指標: overall + Strong-only + Weak-only（PR/ROC/F1/BestThr, Weak P@K）
# ----------------------------------------------------------------------------------------
import os, glob, itertools, numpy as np, pandas as pd, torch, torch.nn as nn, torch.nn.functional as F
from pathlib import Path
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from rdkit import RDLogger
RDLogger.DisableLog("rdApp.warning")

# ===== Paths =====
ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
M1_DIR = f"{ROOT}/results_ens_trans_nn_5fold"
M2_DIR = f"{ROOT}/results_ens_trans_nn_scaffold5fold"
M3_DIR = f"{ROOT}/results_lco5_trans_nn_ens_posaug"
M4_DIR = f"{ROOT}/results_lco5_trans_nn_ens_posaug_advanced"
TRAIN_CSV        = f"{ROOT}/train_split.csv"               # for g/r standardization
EXT_UNION_CSV    = f"{ROOT}/external10_union.csv"
EXT_KEGG_CSV     = f"{ROOT}/external10_kegg_only.csv"
EXT_UNION_GRAPHS = f"{ROOT}/external10_union_graphs.pt"
EXT_KEGG_GRAPHS  = f"{ROOT}/external10_kegg_only_graphs.pt"
BATCH = 256
P_AT_K = [10,25,50,100]
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(ROOT, exist_ok=True)

# ===== RDKit / PyG =====
try:
    from rdkit import Chem
    from rdkit.Chem import Descriptors
    from rdkit.Chem import rdMolDescriptors as rdMD
    from rdkit.Chem import rdchem
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "rdkit-pypi"])
    from rdkit import Chem
    from rdkit.Chem import Descriptors
    from rdkit.Chem import rdMolDescriptors as rdMD
    from rdkit import rdBase; rdBase.DisableLog('rdApp.error')
    from rdkit.Chem import rdchem

try:
    from torch_geometric.loader import DataLoader
    from torch_geometric.data import Data
    from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
    from torch.serialization import add_safe_globals
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch-geometric"])
    from torch_geometric.loader import DataLoader
    from torch_geometric.data import Data
    from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
    from torch.serialization import add_safe_globals

# ===== ver7 features =====
def gfeat7(mol):
    from rdkit.Chem import Descriptors
    return [
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol),
        Descriptors.TPSA(mol),
        Descriptors.HeavyAtomCount(mol),
        Descriptors.RingCount(mol),
    ]

def rdesc10(mol):
    from rdkit.Chem import Descriptors
    return [
        Descriptors.FpDensityMorgan1(mol),
        Descriptors.FpDensityMorgan2(mol),
        Descriptors.FpDensityMorgan3(mol),
        Descriptors.NumAliphaticRings(mol),
        Descriptors.NumAromaticRings(mol),
        Descriptors.NumRotatableBonds(mol),
        Descriptors.NumValenceElectrons(mol),
        Descriptors.BalabanJ(mol),
        Descriptors.BertzCT(mol),
        Descriptors.FractionCSP3(mol),
    ]

from rdkit.Chem import rdchem
def atom_f(atom: rdchem.Atom):
    return torch.tensor([
        atom.GetAtomicNum(),
        atom.GetTotalDegree(),
        atom.GetFormalCharge(),
        float(atom.GetIsAromatic()),
        atom.GetTotalNumHs(includeNeighbors=True),
    ], dtype=torch.float32)

def bond_f(bond: rdchem.Bond):
    return torch.tensor([
        bond.GetBondTypeAsDouble(),
        float(bond.GetIsConjugated()),
        float(bond.IsInRing()),
        int(bond.GetStereo()),
        bond.GetBeginAtom().GetAtomicNum(),
        bond.GetEndAtom().GetAtomicNum(),
    ], dtype=torch.float32)

# ===== StandardScaler (g/r) fitted on train_split.csv =====
def fit_gr_scalers(train_csv):
    if not Path(train_csv).exists():
        print(f"[WARN] train_split.csv not found. Skip standardization for g/r.")
        return None, None
    tdf = pd.read_csv(train_csv)
    if "SMILES" not in tdf.columns:
        print("[WARN] train_split.csv lacks SMILES. Skip g/r standardization.")
        return None, None
    tdf = tdf[(tdf["SMILES"].notna()) & (tdf["SMILES"]!="")]
    g_raw, r_raw = [], []
    for sm in tdf["SMILES"]:
        m = Chem.MolFromSmiles(str(sm))
        if m is None or m.GetNumAtoms()==0: continue
        g_raw.append(gfeat7(m)); r_raw.append(rdesc10(m))
    if not g_raw or not r_raw:
        print("[WARN] No valid g/r to fit. Skip standardization.")
        return None, None
    sg = StandardScaler().fit(np.asarray(g_raw, dtype=np.float32))
    sr = StandardScaler().fit(np.asarray(r_raw, dtype=np.float32))
    print(f"[INFO] StandardScaler fitted: g(7), r(10) from train_split.csv (n={len(g_raw)})")
    return sg, sr

_SCALER_G, _SCALER_R = fit_gr_scalers(TRAIN_CSV)

# ===== CSV→PyG（provided tier を使用; unknown は y を -1 にして後で除外） =====
def _norm_tier(val):
    if val is None: return ""
    s = str(val).strip().lower()
    mapping = {
        "strongpos":"strong_pos","strong+":"strong_pos","s_pos":"strong_pos",
        "strongneg":"strong_neg","strong-":"strong_neg","s_neg":"strong_neg",
        "weakpos":"weak_pos","weak+":"weak_pos","w_pos":"weak_pos",
        "unknown":"unknown","unk":"unknown","": ""
    }
    return mapping.get(s, s)

def _pick_smiles_from_row(row):
    for key in ("SMILES","SMILES_aid","SMILES_kegg"):
        if key in row and pd.notna(row[key]):
            sm = str(row[key]).strip()
            if sm and sm.lower() not in {"nan","none","null"}:
                return sm
    for k,v in row.items():
        if "smiles" in str(k).lower():
            sm = str(v).strip()
            if sm and sm.lower() not in {"nan","none","null"}:
                return sm
    return None

def _csv_role_from_name(path):
    name = os.path.basename(path).lower()
    return "kegg_only" if ("kegg" in name and "only" in name) else "union"

def _tier_for_row(row, role):
    if "tier" in row and pd.notna(row["tier"]):
        return _norm_tier(row["tier"])
    # 予備: KEGG-only は weak_pos とする
    if role == "kegg_only":
        return "weak_pos"
    return "unknown"

def smiles_to_data_ver7(smi, y_val, tier_val):
    m = Chem.MolFromSmiles(smi)
    if m is None or m.GetNumAtoms()==0: return None
    if m.GetNumBonds()==0: return None

    x = torch.stack([atom_f(a) for a in m.GetAtoms()])
    src, dst, eattr = [], [], []
    for b in m.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        f = bond_f(b)
        src += [i, j]; dst += [j, i]; eattr += [f, f]
    edge_index = torch.tensor([src, dst], dtype=torch.long)
    edge_attr  = torch.stack(eattr)

    g_vec = np.asarray(gfeat7(m), dtype=np.float32).reshape(1, -1)
    r_vec = np.asarray(rdesc10(m), dtype=np.float32).reshape(1, -1)
    if _SCALER_G is not None and _SCALER_R is not None:
        g_vec = _SCALER_G.transform(g_vec); r_vec = _SCALER_R.transform(r_vec)
    g = torch.tensor(g_vec, dtype=torch.float32)
    r = torch.tensor(r_vec, dtype=torch.float32)

    # y: 0/1 はそのまま。欠損や unknown は -1（後段で除外）
    y = -1
    if y_val is not None:
        try:
            y_i = int(float(y_val))
            if y_i in (0,1): y = y_i
        except Exception:
            pass

    d = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=torch.tensor(y, dtype=torch.long))
    d.g = g; d.r = r; d.smiles = smi; d.tier = _norm_tier(tier_val)
    return d

def csv_to_graphs_ver7(csv_path, out_pt_path):
    role = _csv_role_from_name(csv_path)
    df = pd.read_csv(csv_path)
    if "y" not in df.columns: df["y"] = np.nan
    if "tier" not in df.columns: df["tier"] = np.nan

    graphs = []; miss = 0
    for _, row in df.iterrows():
        smi = _pick_smiles_from_row(row)
        if not smi: miss += 1; continue
        tier = _tier_for_row(row, role)
        d = smiles_to_data_ver7(smi, row["y"], tier)
        if d is not None: graphs.append(d)

    torch.save(graphs, out_pt_path)
    print(f"✅ Saved graphs: {out_pt_path}  (n={len(graphs)})")

    # 件数サマリ（unknown/y=-1 を把握）
    ys = np.array([int(getattr(d,"y",-1)) for d in graphs])
    ts = np.array([getattr(d,"tier","") for d in graphs], dtype=object)
    from collections import Counter
    print(f"[SUMMARY] {os.path.basename(csv_path)} | y=1:{(ys==1).sum()}, y=0:{(ys==0).sum()}, y=-1:{(ys==-1).sum()}")
    print(f"[SUMMARY] {os.path.basename(csv_path)} | tier: {dict(Counter(ts))}")
    return graphs

def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception:
        pass
    obj=torch.load(path,map_location="cpu")
    if isinstance(obj,list): return obj
    if isinstance(obj,dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

def load_or_build(csv_path, pt_path, force_rebuild=False):
    if force_rebuild or (not Path(pt_path).exists()):
        return csv_to_graphs_ver7(csv_path, pt_path)
    return load_pyg_list(pt_path)

# ===== Extractors (unknownはここで除外) =====
def valid_mask(data_list):
    ys = np.array([int(getattr(d,"y",-1)) for d in data_list])
    return (ys == 0) | (ys == 1)

def extract_labels(data_list, mask):
    ys=[]
    for i,d in enumerate(data_list):
        if not mask[i]: continue
        ys.append(int(getattr(d,"y",-1)))
    return np.array(ys, dtype=int)

def extract_tiers(data_list, mask):
    out=[]
    for i,d in enumerate(data_list):
        if not mask[i]: continue
        out.append(str(getattr(d,"tier","")).lower())
    return np.array(out, dtype=object)

# ===== Models =====
def make_edge_mlp(in_attr_dim,out_channels,in_channels):
    hidden=max(64,min(256,in_attr_dim*8))
    return nn.Sequential(nn.Linear(in_attr_dim,hidden), nn.ReLU(),
                         nn.Linear(hidden,in_channels*out_channels))

class Transformer_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        self.c1=TransformerConv(X_DIM,HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.c2=TransformerConv(HID, HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1)); self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr)); x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g; r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        edge_nn1=make_edge_mlp(E_DIM,HID,X_DIM); edge_nn2=make_edge_mlp(E_DIM,HID,HID)
        self.c1=NNConv(X_DIM,HID,edge_nn1,aggr='mean'); self.c2=NNConv(HID, HID,edge_nn2,aggr='mean')
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1)); self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr)); x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g; r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

# ===== Weights & CV-PR weight =====
def list_weight_files(folder, patterns):
    files=[]
    for pat in patterns: files += sorted(glob.glob(os.path.join(folder, pat)))
    return files

def family_defs():
    return {
      "M1":{"dir":M1_DIR,"transformer":list_weight_files(M1_DIR,["transformer_fold*.pt"]),
            "nnconv":list_weight_files(M1_DIR,["nnconv_fold*.pt"]),
            "val_csvs":list_weight_files(M1_DIR,["*cv*results*.csv","*combined*.csv"])},
      "M2":{"dir":M2_DIR,"transformer":list_weight_files(M2_DIR,["transformer_fold*.pt","*trans*fold*.pt"]),
            "nnconv":list_weight_files(M2_DIR,["nnconv_fold*.pt","*nn*fold*.pt"]),
            "val_csvs":list_weight_files(M2_DIR,["cv_results_*.csv","*combined*.csv","*results*.csv"])},
      "M3":{"dir":M3_DIR,"transformer":list_weight_files(M3_DIR,["trans_fold*.pt","*trans*fold*.pt","transformer_fold*.pt"]),
            "nnconv":list_weight_files(M3_DIR,["nnconv_fold*.pt","*nn*fold*.pt"]),
            "val_csvs":list_weight_files(M3_DIR,["lco_cv_metrics_*.csv","*ensemble*.csv","*cv*.csv"])},
      "M4":{"dir":M4_DIR,"transformer":list_weight_files(M4_DIR,["*trans*fold*.pt","transformer_fold*.pt"]),
            "nnconv":list_weight_files(M4_DIR,["*nn*fold*.pt","nnconv_fold*.pt"]),
            "val_csvs":list_weight_files(M4_DIR,["lco_cv_metrics_*.csv","*cv*.csv"])},
    }

def infer_weights_from_val_csv(csv_paths):
    if not csv_paths: return None
    try:
        prs=[]
        for p in csv_paths:
            df=pd.read_csv(p)
            for c in ["PR-AUC","Ensemble_PR","PR","Val PR-AUC","Ensemble_PR:"]:
                if c in df.columns:
                    s=pd.to_numeric(df[c], errors="coerce")
                    prs += s[~s.isna()].tolist()
        if prs: return float(np.nanmean(prs))
    except Exception: pass
    return None

def fold_key_from_path(path):
    import re, os
    bn=os.path.basename(path)
    for tok in bn.replace(".pt","").replace("-","_").split("_"):
        if tok.lower().startswith("fold"):
            try: return int(tok.lower().replace("fold",""))
            except: pass
    m=re.findall(r"(\d+)", bn)
    return int(m[-1]) if m else -1

@torch.no_grad()
def predict_family_probs(fid, dims, data_list):
    fam=family_defs()[fid]; X_DIM,G_DIM,R_DIM,E_DIM = dims["x"],dims["g"],dims["r"],dims["edge"]
    tr_files=fam["transformer"]; nn_files=fam["nnconv"]
    tr_map={fold_key_from_path(p):p for p in tr_files}
    nn_map={fold_key_from_path(p):p for p in nn_files}
    fold_ids=sorted(set(list(tr_map.keys())+list(nn_map.keys())))
    if not fold_ids: raise RuntimeError(f"No weight files for {fid} in {fam['dir']}")
    dl=DataLoader(data_list, batch_size=BATCH, shuffle=False)
    fold_probs=[]
    for f in fold_ids:
        probs=[]
        if f in tr_map:
            m=Transformer_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE); m.load_state_dict(torch.load(tr_map[f], map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if f in nn_map:
            m=NNConv_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE); m.load_state_dict(torch.load(nn_map[f], map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if probs: fold_probs.append(np.mean(np.stack(probs,axis=0),axis=0))
    fam_probs=np.mean(np.stack(fold_probs,axis=0),axis=0)
    fam_weight=infer_weights_from_val_csv(fam["val_csvs"]) or 1.0
    return fam_probs, fam_weight

# ===== Metrics =====
def roc_auc(y,p):
    try: return float(roc_auc_score(y,p))
    except Exception: return float("nan")

def pr_auc(y,p):
    try:
        prec,rec,_=precision_recall_curve(y,p); return float(auc(rec,prec))
    except Exception: return float("nan")

def f1_at(y,p,thr):
    try: return float(f1_score(y,(p>=thr).astype(int)))
    except Exception: return float("nan")

def best_f1(y,p):
    prec,rec,thr=precision_recall_curve(y,p); thr=np.append(thr,1.0)
    f1=2*prec*rec/np.clip(prec+rec,1e-9,None); i=int(np.nanargmax(f1))
    return float(thr[i]), float(f1[i])

def precision_at_k(y,p,k):
    if len(p)==0: return float("nan")
    idx=np.argsort(-p)[:min(k,len(p))]
    if len(idx)==0: return float("nan")
    return float(np.mean(np.array(y)[idx]))

# ===== Helpers =====
def infer_dims(data_list):
    for d in data_list:
        if hasattr(d,"x") and hasattr(d,"edge_attr") and hasattr(d,"g") and hasattr(d,"r"):
            return {"x":d.x.size(-1), "edge":d.edge_attr.size(-1),
                    "g":(d.g.size(-1) if d.g.dim() in (2,3) else d.g.numel()),
                    "r":(d.r.size(-1) if d.r.dim() in (2,3) else d.r.numel())}
    raise RuntimeError("infer_dims failed")

def combine_probs(combo, table, mode, fam_weights):
    probs=[table[f] for f in combo]
    if mode=="equal": w=np.ones(len(combo))/len(combo)
    else:
        ws=np.array([max(1e-9,fam_weights[f]) for f in combo]); w=ws/ws.sum()
    return np.sum(np.stack([w[i]*probs[i] for i in range(len(combo))],axis=0),axis=0), w

def strat_masks(tiers):
    tiers = np.array([str(t).lower() for t in tiers], dtype=object)
    strong_mask = np.array([t.startswith("strong_") for t in tiers], dtype=bool)
    weak_mask   = (tiers == "weak_pos")
    return strong_mask, weak_mask

def metrics_block(y_true, p_pred):
    out = {
        "ROC": roc_auc(y_true, p_pred),
        "PR":  pr_auc(y_true, p_pred),
        "F1@0.5": f1_at(y_true, p_pred, 0.5)
    }
    bt, bf1 = best_f1(y_true, p_pred)
    out["BestThr"] = bt; out["F1@Best"] = bf1
    return out

def metrics_with_strata(prefix, y, p, tiers, p_at_k_list):
    rows={}
    m_all = metrics_block(y, p)
    rows.update({f"{prefix}_ROC":m_all["ROC"], f"{prefix}_PR":m_all["PR"],
                 f"{prefix}_F1@0.5":m_all["F1@0.5"], f"{prefix}_BestThr":m_all["BestThr"],
                 f"{prefix}_F1@Best":m_all["F1@Best"]})
    mask_s, mask_w = strat_masks(tiers)
    # strong-only
    if mask_s.any():
        m_s = metrics_block(y[mask_s], p[mask_s])
        rows.update({f"{prefix}_Strong_ROC":m_s["ROC"], f"{prefix}_Strong_PR":m_s["PR"],
                     f"{prefix}_Strong_F1@0.5":m_s["F1@0.5"], f"{prefix}_Strong_BestThr":m_s["BestThr"],
                     f"{prefix}_Strong_F1@Best":m_s["F1@Best"]})
    else:
        rows.update({f"{prefix}_Strong_ROC":np.nan, f"{prefix}_Strong_PR":np.nan,
                     f"{prefix}_Strong_F1@0.5":np.nan, f"{prefix}_Strong_BestThr":np.nan,
                     f"{prefix}_Strong_F1@Best":np.nan})
    # weak-only
    if mask_w.any():
        m_w = metrics_block(y[mask_w], p[mask_w])
        rows.update({f"{prefix}_Weak_ROC":m_w["ROC"], f"{prefix}_Weak_PR":m_w["PR"],
                     f"{prefix}_Weak_F1@0.5":m_w["F1@0.5"], f"{prefix}_Weak_BestThr":m_w["BestThr"],
                     f"{prefix}_Weak_F1@Best":m_w["F1@Best"]})
        for K in p_at_k_list:
            rows[f"{prefix}_Weak_P@{K}"] = precision_at_k(y[mask_w], p[mask_w], K)
    else:
        rows.update({f"{prefix}_Weak_ROC":np.nan, f"{prefix}_Weak_PR":np.nan,
                     f"{prefix}_Weak_F1@0.5":np.nan, f"{prefix}_Weak_BestThr":np.nan,
                     f"{prefix}_Weak_F1@Best":np.nan})
        for K in p_at_k_list:
            rows[f"{prefix}_Weak_P@{K}"] = np.nan
    return rows

# ===== Main =====
def main():
    assert Path(EXT_UNION_CSV).exists(), f"not found: {EXT_UNION_CSV}"
    # グラフは常に再構築（CSVのtier/y変更が確実に反映されるように）
    if Path(EXT_UNION_GRAPHS).exists(): os.remove(EXT_UNION_GRAPHS)
    data_union_all = load_or_build(EXT_UNION_CSV, EXT_UNION_GRAPHS, force_rebuild=True)

    # unknown（y==-1）は除外
    m_union = valid_mask(data_union_all)
    data_union = [d for i,d in enumerate(data_union_all) if m_union[i]]

    # KEGG-only は存在しない場合もあるので任意
    data_kegg = []
    if Path(EXT_KEGG_CSV).exists():
        if Path(EXT_KEGG_GRAPHS).exists(): os.remove(EXT_KEGG_GRAPHS)
        data_kegg_all = load_or_build(EXT_KEGG_CSV, EXT_KEGG_GRAPHS, force_rebuild=True)
        m_kegg = valid_mask(data_kegg_all)
        data_kegg = [d for i,d in enumerate(data_kegg_all) if m_kegg[i]]

    # 次元
    dims = None
    for d in data_union:
        if hasattr(d,"x") and hasattr(d,"edge_attr") and hasattr(d,"g") and hasattr(d,"r"):
            dims = {"x":d.x.size(-1), "edge":d.edge_attr.size(-1),
                    "g":(d.g.size(-1) if d.g.dim() in (2,3) else d.g.numel()),
                    "r":(d.r.size(-1) if d.r.dim() in (2,3) else d.r.numel())}
            break
    print("[INFO] dims:", dims)

    fam_ids=["M1","M2","M3","M4"]
    fam_probs_union={}; fam_probs_kegg={}; fam_weights={}
    for fid in fam_ids:
        print(f"\n== {fid} on UNION ==")
        pu,w=predict_family_probs(fid,dims,data_union); fam_probs_union[fid]=pu; fam_weights[fid]=w; print("  weight:",w)
        if data_kegg:
            print(f"== {fid} on KEGG ==")
            pk,_=predict_family_probs(fid,dims,data_kegg);  fam_probs_kegg[fid]=pk

    singles=[(f,) for f in fam_ids]
    pairs  =list(itertools.combinations(fam_ids,2))
    trips  =list(itertools.combinations(fam_ids,3))
    combos =singles+pairs+trips

    y_union   = extract_labels(data_union, np.ones(len(data_union), dtype=bool))
    tiers_union = extract_tiers(data_union, np.ones(len(data_union), dtype=bool))

    if data_kegg:
        y_kegg   = extract_labels(data_kegg, np.ones(len(data_kegg), dtype=bool))
        tiers_kegg = extract_tiers(data_kegg, np.ones(len(data_kegg), dtype=bool))
    else:
        y_kegg = tiers_kegg = None

    rows=[]
    for mode in ["equal","cv_pr"]:
        for combo in combos:
            name="+".join(combo)
            p_u,_=combine_probs(combo, fam_probs_union, mode, fam_weights)

            rec={"Mode":mode,"Combo":name}
            rec.update(metrics_with_strata("Union", y_union, p_u, tiers_union, P_AT_K))

            if data_kegg and (combo[0] in fam_probs_kegg or len(combo)>1):
                # KEGG 側の確率がある組合せのみ計算
                # （無い場合はスキップまたは NaN 埋め）
                try:
                    p_k,_=combine_probs(combo, fam_probs_kegg,  mode, fam_weights)
                    rec.update(metrics_with_strata("KEGG", y_kegg, p_k, tiers_kegg, P_AT_K))
                except Exception:
                    for key in ["ROC","PR","F1@0.5","BestThr","F1@Best"]:
                        rec[f"KEGG_{key}"]=np.nan; rec[f"KEGG_Strong_{key}"]=np.nan; rec[f"KEGG_Weak_{key}"]=np.nan
                    for K in P_AT_K: rec[f"KEGG_Weak_P@{K}"]=np.nan
            else:
                for key in ["ROC","PR","F1@0.5","BestThr","F1@Best"]:
                    rec[f"KEGG_{key}"]=np.nan; rec[f"KEGG_Strong_{key}"]=np.nan; rec[f"KEGG_Weak_{key}"]=np.nan
                for K in P_AT_K: rec[f"KEGG_Weak_P@{K}"]=np.nan

            rows.append(rec)

    df=pd.DataFrame(rows)
    out_all=f"{ROOT}/external_eval_comparison_all.csv"; df.to_csv(out_all, index=False); print("📁", out_all)

    # ランキング（Union_PR→Union_ROC）
    df_rank=df.copy()
    df_rank["RankScore"] = -df_rank["Union_PR"].fillna(-1e9) - 1e-3*df_rank["Union_ROC"].fillna(-1e9)
    df_rank=df_rank.sort_values(["RankScore"], ascending=True).reset_index(drop=True)
    out_rank=f"{ROOT}/external_eval_ranking.csv"; df_rank.to_csv(out_rank, index=False); print("🏁", out_rank)

    print("\n=== Top 10 by Union_PR then Union_ROC ===")
    cols = ["Mode","Combo","Union_PR","Union_ROC","Union_Weak_PR","Union_Strong_PR"]
    if "KEGG_PR" in df_rank.columns: cols += ["KEGG_PR"]
    print(df_rank[cols].head(10).to_string(index=False))

# ===== Run =====
if __name__=="__main__":
    main()

[INFO] StandardScaler fitted: g(7), r(10) from train_split.csv (n=6081)
✅ Saved graphs: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/external10_union_graphs.pt  (n=1478)
[SUMMARY] external10_union.csv | y=1:22, y=0:1453, y=-1:3
[SUMMARY] external10_union.csv | tier: {'aid_weak_neg': 1453, 'unknown': 3, 'aid_weak_pos': 16, 'kegg_pos': 6}
✅ Saved graphs: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/external10_kegg_only_graphs.pt  (n=0)
[SUMMARY] external10_kegg_only.csv | y=1:0, y=0:0, y=-1:0
[SUMMARY] external10_kegg_only.csv | tier: {}
[INFO] dims: {'x': 5, 'edge': 6, 'g': 7, 'r': 10}

== M1 on UNION ==
  weight: 0.6255231451006052

== M2 on UNION ==
  weight: 0.40228706089133903

== M3 on UNION ==
  weight: 0.2918669126356172

== M4 on UNION ==
  weight: 0.6185174288319562
📁 /content/drive/MyDrive/Chemoinfo_MDR1_ver7/external_eval_comparison_all.csv
🏁 /content/drive/MyDrive/Chemoinfo_MDR1_ver7/external_eval_ranking.csv

=== Top 10 by Union_PR then Union_ROC ===
 Mode    Combo  Union_PR  U

In [ ]:
# ==== External Evaluation for M4 only with ROC curves (UNION & KEGG-only) ====
# - Loads external10_union_graphs.pt / external10_kegg_only_graphs.pt
# - Loads only M4 weights, predicts per-family probs
# - Reports: PR-AUC, ROC-AUC, Precision/Recall/F1 @ fixed thr, Best-F1
# - Saves ROC curves (PNG) and ROC points (CSV) for UNION & KEGG-only (when defined)
# -----------------------------------------------------------------------------
import sys, subprocess

def ensure_sklearn_stack():
    try:
        import sklearn  # noqa
        from sklearn.metrics import precision_recall_curve  # noqa
        return
    except Exception as e:
        print("[INFO] fixing sklearn/scipy stack due to:", repr(e))
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                                   "scipy>=1.13.0", "scikit-learn>=1.4.0"])
        except Exception as e2:
            print("[WARN] upgrade path failed:", e2)
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                                   "numpy<2", "scipy<1.11", "scikit-learn<1.4"])
        import importlib; importlib.invalidate_caches()
ensure_sklearn_stack()

import os, glob, numpy as np, torch, torch.nn as nn, torch.nn.functional as F
from pathlib import Path
from sklearn.metrics import (precision_recall_curve, precision_score, recall_score, f1_score,
                             roc_auc_score, auc, roc_curve)
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
from torch.serialization import add_safe_globals
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# matplotlib（色は指定しない／単独プロット）
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
EXT_UNION_PT = f"{ROOT}/external10_union_graphs.pt"
EXT_KEGG_PT  = f"{ROOT}/external10_kegg_only_graphs.pt"

# ==== M4 only ====
M4_DIR = f"{ROOT}/results_lco5_trans_nn_ens_posaug_advanced"

OUT_DIR = f"{ROOT}/figs_m4_roc"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH = 256
FIXED_THR = 0.5

# ---------- helpers ----------
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception:
        pass
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

def dims_from_sample(d):
    x_dim = d.x.size(-1)
    e_dim = d.edge_attr.size(-1)
    g_dim = d.g.size(-1) if d.g.dim() in (2,3) else d.g.numel()
    r_dim = d.r.size(-1) if d.r.dim() in (2,3) else d.r.numel()
    return x_dim, e_dim, g_dim, r_dim

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hid = max(64, min(256, in_attr_dim*8))
    return nn.Sequential(nn.Linear(in_attr_dim,hid), nn.ReLU(),
                         nn.Linear(hid, in_channels*out_channels))

class Transformer_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        self.c1=TransformerConv(X_DIM,HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.c2=TransformerConv(HID, HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1))
        self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr))
        x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g
        r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        e1=make_edge_mlp(E_DIM,HID,X_DIM); e2=make_edge_mlp(E_DIM,HID,HID)
        self.c1=NNConv(X_DIM,HID,e1,aggr='mean'); self.c2=NNConv(HID,HID,e2,aggr='mean')
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1))
        self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr))
        x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g
        r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

def fold_id(p):
    import re, os
    bn=os.path.basename(p).lower()
    m=re.findall(r"fold(\d+)", bn)
    return int(m[-1]) if m else -1

def infer_cv_pr_weight(csv_paths):
    if not csv_paths: return None
    vals=[]
    for p in csv_paths:
        try:
            import pandas as pd
            df=pd.read_csv(p)
            for c in ["PR-AUC","Ensemble_PR","PR","Val PR-AUC","Ensemble_PR:"]:
                if c in df.columns:
                    s=pd.to_numeric(df[c], errors="coerce")
                    vals += s[~s.isna()].tolist()
        except Exception:
            pass
    return float(np.nanmean(vals)) if vals else None

@torch.no_grad()
def family_probs(data_list, family_dir, dims):
    """Return mean probs across folds/models for a family, and optional CV-PR weight (unused here)."""
    X_DIM,E_DIM,G_DIM,R_DIM = dims
    tr_files=sorted(glob.glob(os.path.join(family_dir,"transformer_fold*.pt")))
    nn_files=sorted(glob.glob(os.path.join(family_dir,"nnconv_fold*.pt")))
    if not tr_files: tr_files=sorted(glob.glob(os.path.join(family_dir,"*trans*fold*.pt")))
    if not nn_files: nn_files=sorted(glob.glob(os.path.join(family_dir,"*nn*fold*.pt")))
    tr_map={fold_id(p):p for p in tr_files}; nn_map={fold_id(p):p for p in nn_files}
    folds=sorted(set(list(tr_map.keys())+list(nn_map.keys())))
    assert folds, f"No weight files found in: {family_dir}"

    # optional weight (not used for single-family reporting)
    val_csvs = sorted(glob.glob(os.path.join(family_dir,"*.csv")))
    fam_w = infer_cv_pr_weight(val_csvs) or 1.0

    dl=DataLoader(data_list,batch_size=BATCH,shuffle=False)
    fold_probs=[]
    for f in folds:
        probs=[]
        if f in tr_map:
            m=Transformer_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE)
            m.load_state_dict(torch.load(tr_map[f],map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if f in nn_map:
            m=NNConv_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE)
            m.load_state_dict(torch.load(nn_map[f],map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if probs:
            fold_probs.append(np.mean(np.stack(probs,axis=0),axis=0))
    fam_probs=np.mean(np.stack(fold_probs,axis=0),axis=0)
    return fam_probs, float(fam_w)

def evaluable_subset_or_none(data_list):
    idx=[i for i,d in enumerate(data_list) if int(getattr(d,"y",-1)) in (0,1)]
    if not idx:
        return None, None
    y=np.array([int(getattr(data_list[i],'y')) for i in idx], dtype=int)
    return idx, y

def report_block(name, y, p, thr_fixed=0.5):
    prev = y.mean()
    print(f"\n=== {name} ===")
    print(f"N={len(y)} | positives={y.sum()} ({prev*100:.2f}%)")
    prec, rec, thr = precision_recall_curve(y, p)
    pr = auc(rec, prec)
    try:
        roc = roc_auc_score(y, p)
    except Exception:
        roc = float("nan")
    print(f"PR-AUC={pr:.6f} | ROC-AUC={roc:.6f}")

    predF = (p>=thr_fixed).astype(int)
    prF = precision_score(y,predF,zero_division=0)
    rcF = recall_score(y,predF,zero_division=0)
    f1F = f1_score(y,predF,zero_division=0)
    print(f"@thr={thr_fixed:.4f}  Precision={prF:.4f}  Recall={rcF:.4f}  F1={f1F:.4f}  PosPred={predF.sum()}")

    thr_all = np.append(thr, 1.0)
    f1_all = 2*prec*rec/np.clip(prec+rec,1e-9,None)
    i = int(np.nanargmax(f1_all))
    print(f"[Best-F1] thr={thr_all[i]:.4f}  F1={f1_all[i]:.4f}  (Prec={prec[i]:.4f}, Rec={rec[i]:.4f})")

def save_roc_curve(out_png, out_csv, y, p, title):
    """Save ROC curve image and CSV (fpr,tpr,thresholds). Skip if undefined."""
    if y is None or p is None or len(y)==0:
        print(f"[SKIP] ROC undefined for {title} (no evaluable samples).")
        return False
    if (np.unique(y).size < 2):
        print(f"[SKIP] ROC undefined for {title} (single-class labels).")
        return False
    fpr, tpr, thr = roc_curve(y, p)
    roc = roc_auc_score(y, p)
    import pandas as pd
    pd.DataFrame({"fpr":fpr, "tpr":tpr, "threshold":thr}).to_csv(out_csv, index=False)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC={roc:.4f}")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_png, dpi=160); plt.close()
    print(f"[ROC] Saved: {out_png} | {out_csv}")
    return True

# ---------- run ----------
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

union_list = load_pyg_list(EXT_UNION_PT)
kegg_list  = load_pyg_list(EXT_KEGG_PT) if Path(EXT_KEGG_PT).exists() else []

idx_u, y_u = evaluable_subset_or_none(union_list)
if idx_u is None:
    raise RuntimeError("UNION: No evaluable samples (y in {0,1}).")

idx_k, y_k = evaluable_subset_or_none(kegg_list) if len(kegg_list)>0 else (None, None)

X_DIM,E_DIM,G_DIM,R_DIM = dims_from_sample(union_list[idx_u[0]])
dims = (X_DIM,E_DIM,G_DIM,R_DIM)
print("[INFO] dims:", {"x":X_DIM,"edge":E_DIM,"g":G_DIM,"r":R_DIM})

# per-family probs (M4 only)
pM4_u, wM4 = family_probs([union_list[i] for i in idx_u], M4_DIR, dims)
if idx_k is not None:
    pM4_k, _  = family_probs([kegg_list[i] for i in idx_k], M4_DIR, dims)
else:
    pM4_k = None

print(f"[INFO] M4 CV-PR weight (unused for single-family): {wM4:.3f}")

# reports (text)
report_block("M4 on UNION", y_u, pM4_u, thr_fixed=FIXED_THR)
if idx_k is not None:
    report_block("M4 on KEGG-only", y_k, pM4_k, thr_fixed=FIXED_THR)
else:
    print("\n[INFO] KEGG-only set is empty or has no evaluable samples — skip KEGG reports.")

# ROC curves (PNG & CSV)
save_roc_curve(f"{OUT_DIR}/roc_union_m4.png",
               f"{OUT_DIR}/roc_union_m4.csv",
               y_u, pM4_u, "M4 ROC - UNION")

if idx_k is not None:
    save_roc_curve(f"{OUT_DIR}/roc_kegg_m4.png",
                   f"{OUT_DIR}/roc_kegg_m4.csv",
                   y_k, pM4_k, "M4 ROC - KEGG-only")

print(f"\nOutputs in: {OUT_DIR}")

[INFO] dims: {'x': 5, 'edge': 6, 'g': 7, 'r': 10}
[INFO] M4 CV-PR weight (unused for single-family): 0.619

=== M4 on UNION ===
N=1475 | positives=22 (1.49%)
PR-AUC=0.279945 | ROC-AUC=0.898580
@thr=0.5000  Precision=0.2727  Recall=0.5455  F1=0.3636  PosPred=44
[Best-F1] thr=0.5774  F1=0.4314  (Prec=0.3793, Rec=0.5000)

[INFO] KEGG-only set is empty or has no evaluable samples — skip KEGG reports.
[ROC] Saved: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc/roc_union_m4.png | /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc/roc_union_m4.csv

Outputs in: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc


In [ ]:
# ==== External Evaluation for M4 only with ROC/PR curves (UNION & KEGG-only) ====
# - Loads external10_union_graphs.pt / external10_kegg_only_graphs.pt
# - Loads only M4 weights, predicts per-family probs
# - Reports: PR-AUC, ROC-AUC, Precision/Recall/F1 @ fixed thr, Best-F1
# - Saves ROC/PR curves (PNG) and ROC/PR points (CSV) for UNION & KEGG-only (when defined)
# -----------------------------------------------------------------------------
import sys, subprocess

def ensure_sklearn_stack():
    try:
        import sklearn  # noqa
        from sklearn.metrics import precision_recall_curve  # noqa
        return
    except Exception as e:
        print("[INFO] fixing sklearn/scipy stack due to:", repr(e))
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                                   "scipy>=1.13.0", "scikit-learn>=1.4.0"])
        except Exception as e2:
            print("[WARN] upgrade path failed:", e2)
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                                   "numpy<2", "scipy<1.11", "scikit-learn<1.4"])
        import importlib; importlib.invalidate_caches()
ensure_sklearn_stack()

import os, glob, numpy as np, torch, torch.nn as nn, torch.nn.functional as F
from pathlib import Path
from sklearn.metrics import (precision_recall_curve, precision_score, recall_score, f1_score,
                             roc_auc_score, auc, roc_curve)
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
from torch.serialization import add_safe_globals
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# matplotlib（色は指定しない／単独プロット）
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
EXT_UNION_PT = f"{ROOT}/external10_union_graphs.pt"
EXT_KEGG_PT  = f"{ROOT}/external10_kegg_only_graphs.pt"

# ==== M4 only ====
M4_DIR = f"{ROOT}/results_lco5_trans_nn_ens_posaug_advanced"

OUT_DIR = f"{ROOT}/figs_m4_roc"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH = 256
FIXED_THR = 0.5

# ---------- helpers ----------
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception:
        pass
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

def dims_from_sample(d):
    x_dim = d.x.size(-1)
    e_dim = d.edge_attr.size(-1)
    g_dim = d.g.size(-1) if d.g.dim() in (2,3) else d.g.numel()
    r_dim = d.r.size(-1) if d.r.dim() in (2,3) else d.r.numel()
    return x_dim, e_dim, g_dim, r_dim

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hid = max(64, min(256, in_attr_dim*8))
    return nn.Sequential(nn.Linear(in_attr_dim,hid), nn.ReLU(),
                         nn.Linear(hid, in_channels*out_channels))

class Transformer_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        self.c1=TransformerConv(X_DIM,HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.c2=TransformerConv(HID, HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1))
        self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr))
        x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g
        r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        e1=make_edge_mlp(E_DIM,HID,X_DIM); e2=make_edge_mlp(E_DIM,HID,HID)
        self.c1=NNConv(X_DIM,HID,e1,aggr='mean'); self.c2=NNConv(HID,HID,e2,aggr='mean')
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1))
        self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr))
        x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g
        r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

def fold_id(p):
    import re, os
    bn=os.path.basename(p).lower()
    m=re.findall(r"fold(\d+)", bn)
    return int(m[-1]) if m else -1

def infer_cv_pr_weight(csv_paths):
    if not csv_paths: return None
    vals=[]
    for p in csv_paths:
        try:
            import pandas as pd
            df=pd.read_csv(p)
            for c in ["PR-AUC","Ensemble_PR","PR","Val PR-AUC","Ensemble_PR:"]:
                if c in df.columns:
                    s=pd.to_numeric(df[c], errors="coerce")
                    vals += s[~s.isna()].tolist()
        except Exception:
            pass
    return float(np.nanmean(vals)) if vals else None

@torch.no_grad()
def family_probs(data_list, family_dir, dims):
    """Return mean probs across folds/models for a family, and optional CV-PR weight (unused here)."""
    X_DIM,E_DIM,G_DIM,R_DIM = dims
    tr_files=sorted(glob.glob(os.path.join(family_dir,"transformer_fold*.pt")))
    nn_files=sorted(glob.glob(os.path.join(family_dir,"nnconv_fold*.pt")))
    if not tr_files: tr_files=sorted(glob.glob(os.path.join(family_dir,"*trans*fold*.pt")))
    if not nn_files: nn_files=sorted(glob.glob(os.path.join(family_dir,"*nn*fold*.pt")))
    tr_map={fold_id(p):p for p in tr_files}; nn_map={fold_id(p):p for p in nn_files}
    folds=sorted(set(list(tr_map.keys())+list(nn_map.keys())))
    assert folds, f"No weight files found in: {family_dir}"

    # optional weight (not used for single-family reporting)
    val_csvs = sorted(glob.glob(os.path.join(family_dir,"*.csv")))
    fam_w = infer_cv_pr_weight(val_csvs) or 1.0

    dl=DataLoader(data_list,batch_size=BATCH,shuffle=False)
    fold_probs=[]
    for f in folds:
        probs=[]
        if f in tr_map:
            m=Transformer_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE)
            m.load_state_dict(torch.load(tr_map[f],map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if f in nn_map:
            m=NNConv_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE)
            m.load_state_dict(torch.load(nn_map[f],map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if probs:
            fold_probs.append(np.mean(np.stack(probs,axis=0),axis=0))
    fam_probs=np.mean(np.stack(fold_probs,axis=0),axis=0)
    return fam_probs, float(fam_w)

def evaluable_subset_or_none(data_list):
    idx=[i for i,d in enumerate(data_list) if int(getattr(d,"y",-1)) in (0,1)]
    if not idx:
        return None, None
    y=np.array([int(getattr(data_list[i],'y')) for i in idx], dtype=int)
    return idx, y

def report_block(name, y, p, thr_fixed=0.5):
    prev = y.mean()
    print(f"\n=== {name} ===")
    print(f"N={len(y)} | positives={y.sum()} ({prev*100:.2f}%)")
    prec, rec, thr = precision_recall_curve(y, p)
    pr = auc(rec, prec)
    try:
        roc = roc_auc_score(y, p)
    except Exception:
        roc = float("nan")
    print(f"PR-AUC={pr:.6f} | ROC-AUC={roc:.6f}")

    predF = (p>=thr_fixed).astype(int)
    prF = precision_score(y,predF,zero_division=0)
    rcF = recall_score(y,predF,zero_division=0)
    f1F = f1_score(y,predF,zero_division=0)
    print(f"@thr={thr_fixed:.4f}  Precision={prF:.4f}  Recall={rcF:.4f}  F1={f1F:.4f}  PosPred={predF.sum()}")

    thr_all = np.append(thr, 1.0)
    f1_all = 2*prec*rec/np.clip(prec+rec,1e-9,None)
    i = int(np.nanargmax(f1_all))
    print(f"[Best-F1] thr={thr_all[i]:.4f}  F1={f1_all[i]:.4f}  (Prec={prec[i]:.4f}, Rec={rec[i]:.4f})")

def save_roc_curve(out_png, out_csv, y, p, title):
    """Save ROC curve image and CSV (fpr,tpr,thresholds). Skip if undefined."""
    if y is None or p is None or len(y)==0:
        print(f"[SKIP] ROC undefined for {title} (no evaluable samples).")
        return False
    if (np.unique(y).size < 2):
        print(f"[SKIP] ROC undefined for {title} (single-class labels).")
        return False
    fpr, tpr, thr = roc_curve(y, p)
    roc = roc_auc_score(y, p)
    import pandas as pd
    pd.DataFrame({"fpr":fpr, "tpr":tpr, "threshold":thr}).to_csv(out_csv, index=False)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC={roc:.4f}")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_png, dpi=160); plt.close()
    print(f"[ROC] Saved: {out_png} | {out_csv}")
    return True

def save_pr_curve(out_png, out_csv, y, p, title):
    """Save PR curve image and CSV (recall,precision,threshold). Skip if undefined."""
    if y is None or p is None or len(y)==0:
        print(f"[SKIP] PR undefined for {title} (no evaluable samples).")
        return False
    if (np.unique(y).size < 2):
        print(f"[SKIP] PR undefined for {title} (single-class labels).")
        return False
    prec, rec, thr = precision_recall_curve(y, p)
    pr_auc = auc(rec, prec)
    # thresholds: len = len(prec)-1; append 1.0 to align with prec/rec
    thr_all = np.append(thr, 1.0)

    import pandas as pd
    pd.DataFrame({"recall": rec, "precision": prec, "threshold": thr_all}).to_csv(out_csv, index=False)

    plt.figure()
    plt.plot(rec, prec, label=f"PR-AUC={pr_auc:.4f}")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_png, dpi=160); plt.close()
    print(f"[PR]  Saved: {out_png} | {out_csv}")
    return True

# ---------- run ----------
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

union_list = load_pyg_list(EXT_UNION_PT)
kegg_list  = load_pyg_list(EXT_KEGG_PT) if Path(EXT_KEGG_PT).exists() else []

idx_u, y_u = evaluable_subset_or_none(union_list)
if idx_u is None:
    raise RuntimeError("UNION: No evaluable samples (y in {0,1}).")

idx_k, y_k = evaluable_subset_or_none(kegg_list) if len(kegg_list)>0 else (None, None)

X_DIM,E_DIM,G_DIM,R_DIM = dims_from_sample(union_list[idx_u[0]])
dims = (X_DIM,E_DIM,G_DIM,R_DIM)
print("[INFO] dims:", {"x":X_DIM,"edge":E_DIM,"g":G_DIM,"r":R_DIM})

# per-family probs (M4 only)
pM4_u, wM4 = family_probs([union_list[i] for i in idx_u], M4_DIR, dims)
if idx_k is not None:
    pM4_k, _  = family_probs([kegg_list[i] for i in idx_k], M4_DIR, dims)
else:
    pM4_k = None

print(f"[INFO] M4 CV-PR weight (unused for single-family): {wM4:.3f}")

# reports (text)
report_block("M4 on UNION", y_u, pM4_u, thr_fixed=FIXED_THR)
if idx_k is not None:
    report_block("M4 on KEGG-only", y_k, pM4_k, thr_fixed=FIXED_THR)
else:
    print("\n[INFO] KEGG-only set is empty or has no evaluable samples — skip KEGG reports.")

# ROC curves (PNG & CSV)
save_roc_curve(f"{OUT_DIR}/roc_union_m4.png",
               f"{OUT_DIR}/roc_union_m4.csv",
               y_u, pM4_u, "M4 ROC - UNION")

if idx_k is not None:
    save_roc_curve(f"{OUT_DIR}/roc_kegg_m4.png",
                   f"{OUT_DIR}/roc_kegg_m4.csv",
                   y_k, pM4_k, "M4 ROC - KEGG-only")

# PR curves (PNG & CSV)
save_pr_curve(f"{OUT_DIR}/pr_union_m4.png",
              f"{OUT_DIR}/pr_union_m4.csv",
              y_u, pM4_u, "M4 PR - UNION")

if idx_k is not None:
    save_pr_curve(f"{OUT_DIR}/pr_kegg_m4.png",
                  f"{OUT_DIR}/pr_kegg_m4.csv",
                  y_k, pM4_k, "M4 PR - KEGG-only")

print(f"\nOutputs in: {OUT_DIR}")

[INFO] dims: {'x': 5, 'edge': 6, 'g': 7, 'r': 10}
[INFO] M4 CV-PR weight (unused for single-family): 0.619

=== M4 on UNION ===
N=1475 | positives=22 (1.49%)
PR-AUC=0.279945 | ROC-AUC=0.898580
@thr=0.5000  Precision=0.2727  Recall=0.5455  F1=0.3636  PosPred=44
[Best-F1] thr=0.5774  F1=0.4314  (Prec=0.3793, Rec=0.5000)

[INFO] KEGG-only set is empty or has no evaluable samples — skip KEGG reports.
[ROC] Saved: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc/roc_union_m4.png | /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc/roc_union_m4.csv
[PR]  Saved: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc/pr_union_m4.png | /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc/pr_union_m4.csv

Outputs in: /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_m4_roc


In [ ]:
# ==== MDR1-M4-HYB-v1: Internal evaluation with 95% CIs, ROC/PR curves, Confusion table ====
# - Input: internal graphs (train/val) at ROOT/data_graph_with_smiles.pt
# - Model: M4 only (results_lco5_trans_nn_ens_posaug_advanced)
# - Outputs:
#   * stats csv with point estimates + 95% bootstrap CIs (ROC-AUC, PR-AUC, F1@0.5, F1@Best)
#   * confusion tables (TP/FP/TN/FN) @0.5 and @Best-F1 threshold
#   * ROC/PR curves (PNG + CSV)
#   * per-sample predictions CSV  (p_ens は互換目的で p_M4 と同値)
# --------------------------------------------------------------------------------------------------
import sys, subprocess

# --- ensure sklearn/scipy stack works with current numpy ---
def ensure_sklearn_stack():
    try:
        import sklearn  # noqa
        from sklearn.metrics import precision_recall_curve  # noqa
        return
    except Exception as e:
        print("[INFO] fixing sklearn/scipy stack due to:", repr(e))
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                                   "scipy>=1.13.0", "scikit-learn>=1.4.0"])
        except Exception as e2:
            print("[WARN] upgrade path failed:", e2)
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                                   "numpy<2", "scipy<1.11", "scikit-learn<1.4"])
        import importlib; importlib.invalidate_caches()
ensure_sklearn_stack()

import os, glob, math, json, numpy as np, pandas as pd, torch, torch.nn as nn, torch.nn.functional as F
from pathlib import Path
from sklearn.metrics import (
    precision_recall_curve, roc_curve, auc, roc_auc_score,
    precision_score, recall_score, f1_score
)
from sklearn.exceptions import UndefinedMetricWarning
from torch_geometric.loader import DataLoader
from torch_geometric.nn import TransformerConv, NNConv, AttentionalAggregation
from torch.serialization import add_safe_globals
import warnings
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# matplotlib backend for headless
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ====== CONFIG ======
ROOT   = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PT = f"{ROOT}/data_graph_with_smiles.pt"   # 内部データ（学習/検証）グラフ

# --- M4 のみ ---
M4_DIR  = f"{ROOT}/results_lco5_trans_nn_ens_posaug_advanced"

OUT_DIR = f"{ROOT}/reports_internal_MDR1-M4-HYB-v1"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH = 256
FIXED_THR = 0.5           # 混同行列/固定F1用の閾値
N_BOOT = 2000             # ブートストラップ反復回数（95%CI）
SEED = 42                 # 再現性のため

# ====== PyG helpers ======
def load_pyg_list(path):
    try:
        from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
        from torch_geometric.data.storage import GlobalStorage
        add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    except Exception:
        pass
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

def dims_from_sample(d):
    x_dim = d.x.size(-1)
    e_dim = d.edge_attr.size(-1)
    g_dim = d.g.size(-1) if d.g.dim() in (2,3) else d.g.numel()
    r_dim = d.r.size(-1) if d.r.dim() in (2,3) else d.r.numel()
    return x_dim, e_dim, g_dim, r_dim

def make_edge_mlp(in_attr_dim, out_channels, in_channels):
    hid = max(64, min(256, in_attr_dim*8))
    return nn.Sequential(nn.Linear(in_attr_dim,hid), nn.ReLU(),
                         nn.Linear(hid, in_channels*out_channels))

class Transformer_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        self.c1=TransformerConv(X_DIM,HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.c2=TransformerConv(HID, HID,heads=4,concat=False,edge_dim=E_DIM,dropout=0.0,beta=False)
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1))
        self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr))
        x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g
        r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

class NNConv_DX_DG_DR(nn.Module):
    def __init__(self,X_DIM,G_DIM,R_DIM,E_DIM,HID=256):
        super().__init__()
        e1=make_edge_mlp(E_DIM,HID,X_DIM); e2=make_edge_mlp(E_DIM,HID,HID)
        self.c1=NNConv(X_DIM,HID,e1,aggr='mean'); self.c2=NNConv(HID,HID,e2,aggr='mean')
        self.pool=AttentionalAggregation(gate_nn=nn.Linear(HID,1))
        self.drop=nn.Dropout(0.2)
        self.fc=nn.Linear(HID+G_DIM+R_DIM,1)
    def forward(self,d):
        x=F.relu(self.c1(d.x,d.edge_index,d.edge_attr))
        x=F.relu(self.c2(x,d.edge_index,d.edge_attr))
        x=self.pool(self.drop(x), d.batch)
        g=d.g.squeeze(1) if d.g.dim()==3 else d.g
        r=d.r.squeeze(1) if d.r.dim()==3 else d.r
        return self.fc(torch.cat([x,g,r],dim=1)).view(-1)

def fold_id(p):
    import re, os
    bn=os.path.basename(p).lower()
    m=re.findall(r"fold(\d+)", bn)
    return int(m[-1]) if m else -1

def infer_cv_pr_weight(csv_paths):
    if not csv_paths: return None
    vals=[]
    for p in csv_paths:
        try:
            df=pd.read_csv(p)
            for c in ["PR-AUC","Ensemble_PR","PR","Val PR-AUC","Ensemble_PR:"]:
                if c in df.columns:
                    s=pd.to_numeric(df[c], errors="coerce")
                    vals += s[~s.isna()].tolist()
        except Exception:
            pass
    return float(np.nanmean(vals)) if vals else None

@torch.no_grad()
def family_probs(data_list, family_dir, dims):
    """Return mean probs across folds/models for M4, and optional CV-PR weight (info only)."""
    X_DIM,E_DIM,G_DIM,R_DIM = dims
    tr_files=sorted(glob.glob(os.path.join(family_dir,"transformer_fold*.pt")))
    nn_files=sorted(glob.glob(os.path.join(family_dir,"nnconv_fold*.pt")))
    if not tr_files: tr_files=sorted(glob.glob(os.path.join(family_dir,"*trans*fold*.pt")))
    if not nn_files: nn_files=sorted(glob.glob(os.path.join(family_dir,"*nn*fold*.pt")))
    tr_map={fold_id(p):p for p in tr_files}; nn_map={fold_id(p):p for p in nn_files}
    folds=sorted(set(list(tr_map.keys())+list(nn_map.keys())))
    assert folds, f"No weight files found in: {family_dir}"

    val_csvs = sorted(glob.glob(os.path.join(family_dir,"*.csv")))
    fam_w = infer_cv_pr_weight(val_csvs) or 1.0

    dl=DataLoader(data_list,batch_size=BATCH,shuffle=False)
    fold_probs=[]
    for f in folds:
        probs=[]
        if f in tr_map:
            m=Transformer_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE)
            m.load_state_dict(torch.load(tr_map[f],map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if f in nn_map:
            m=NNConv_DX_DG_DR(X_DIM,G_DIM,R_DIM,E_DIM).to(DEVICE)
            m.load_state_dict(torch.load(nn_map[f],map_location=DEVICE)); m.eval()
            ps=[]
            for bt in dl: bt=bt.to(DEVICE); ps.append(torch.sigmoid(m(bt)).detach().cpu())
            probs.append(torch.cat(ps).numpy()); del m
        if probs:
            fold_probs.append(np.mean(np.stack(probs,axis=0),axis=0))
    fam_probs=np.mean(np.stack(fold_probs,axis=0),axis=0)
    return fam_probs, float(fam_w)

def evaluable_subset(data_list):
    """Return indices and labels for samples with y in {0,1}."""
    idx=[i for i,d in enumerate(data_list) if int(getattr(d,"y",-1)) in (0,1)]
    if not idx:
        raise RuntimeError("No evaluable samples (y in {0,1}).")
    y=np.array([int(getattr(data_list[i],'y')) for i in idx], dtype=int)
    smiles=[]
    for i in idx:
        sm=getattr(data_list[i],"smiles", None)
        smiles.append(sm if sm is not None else "")
    return idx, y, smiles

# ====== metrics & CI ======
def pr_auc(y,p):
    prec, rec, _ = precision_recall_curve(y, p)
    return float(auc(rec, prec))

def roc_auc(y,p):
    try: return float(roc_auc_score(y,p))
    except Exception: return float("nan")

def f1_at(y,p,thr):
    return float(f1_score(y,(p>=thr).astype(int),zero_division=0))

def best_f1(y,p):
    prec,rec,thr=precision_recall_curve(y,p)
    thr=np.append(thr,1.0)
    f1=2*prec*rec/np.clip(prec+rec,1e-9,None)
    i=int(np.nanargmax(f1))
    return float(thr[i]), float(f1[i]), float(prec[i]), float(rec[i])

_compute_best_f1 = best_f1

def confusion_counts(y,p,thr):
    pred=(p>=thr).astype(int)
    tp=int(((pred==1)&(y==1)).sum())
    fp=int(((pred==1)&(y==0)).sum())
    tn=int(((pred==0)&(y==0)).sum())
    fn=int(((pred==0)&(y==1)).sum())
    return tp,fp,tn,fn

def bootstrap_ci(metric_fn, y, p, n_boot=2000, seed=42, stratified=True):
    """Return (point, lo95, hi95) using bootstrap. metric_fn(y, p) -> float"""
    rng = np.random.default_rng(seed)
    n=len(y)
    if stratified:
        pos_idx=np.where(y==1)[0]; neg_idx=np.where(y==0)[0]
        n_pos=len(pos_idx); n_neg=len(neg_idx)
    vals=[]
    for _ in range(n_boot):
        if stratified and n_pos>0 and n_neg>0:
            samp_pos = rng.choice(pos_idx, size=n_pos, replace=True)
            samp_neg = rng.choice(neg_idx, size=n_neg, replace=True)
            samp = np.concatenate([samp_pos, samp_neg])
        else:
            samp = rng.choice(np.arange(n), size=n, replace=True)
        yy=y[samp]; pp=p[samp]
        try:
            vals.append(float(metric_fn(yy,pp)))
        except Exception:
            continue
    vals=np.array(vals, dtype=float)
    point=float(metric_fn(y,p))
    lo=float(np.nanpercentile(vals, 2.5)) if len(vals)>0 else float("nan")
    hi=float(np.nanpercentile(vals,97.5)) if len(vals)>0 else float("nan")
    return point, lo, hi

# ====== curves saving ======
def save_roc(out_png, out_csv, y, p, title):
    if np.unique(y).size<2:
        print(f"[SKIP] ROC undefined (single-class labels). {title}")
        return None
    fpr,tpr,thr = roc_curve(y,p)
    roc = roc_auc_score(y,p)
    pd.DataFrame({"fpr":fpr,"tpr":tpr,"threshold":thr}).to_csv(out_csv, index=False)
    plt.figure()
    plt.plot(fpr,tpr,label=f"AUC={roc:.4f}")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_png, dpi=160); plt.close()
    print(f"[ROC] {title} -> {out_png} | {out_csv}")
    return roc

def save_pr(out_png, out_csv, y, p, title):
    prec,rec,thr = precision_recall_curve(y,p)
    pr = auc(rec,prec)
    pd.DataFrame({"recall":rec, "precision":prec, "threshold":np.append(thr,np.nan)}).to_csv(out_csv, index=False)
    plt.figure()
    plt.plot(rec,prec,label=f"AUC={pr:.4f}")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(title); plt.legend(); plt.tight_layout()
    plt.savefig(out_png, dpi=160); plt.close()
    print(f"[PR ] {title} -> {out_png} | {out_csv}")
    return pr

# ====== run ======
def main():
    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

    # Load internal data
    data_list = load_pyg_list(DATA_PT)
    idx, y, smiles = evaluable_subset(data_list)
    data_eval = [data_list[i] for i in idx]
    y = np.asarray(y, dtype=int)
    print(f"[INFO] Internal evaluable N={len(y)} | positives={y.sum()} | negatives={(y==0).sum()}")

    # Dims
    X_DIM,E_DIM,G_DIM,R_DIM = dims_from_sample(data_eval[0])
    dims = (X_DIM,E_DIM,G_DIM,R_DIM)
    print("[INFO] dims:", {"x":X_DIM,"edge":E_DIM,"g":G_DIM,"r":R_DIM})

    # --- M4 の確率 ---
    pM4, wM4 = family_probs(data_eval, M4_DIR, dims)
    p = pM4.copy()  # そのまま最終確率として使用

    print(f"[INFO] Using M4 only | optional CV-PR weight (info): {wM4:.3f}")

    # Save per-sample predictions（互換のため p_ens も保存）
    df_pred = pd.DataFrame({
        "SMILES": smiles,
        "y_true": y,
        "p_M4": pM4,
        "p_ens": p,   # 下流スクリプト互換: p_ens = M4
    })
    pred_csv = f"{OUT_DIR}/predictions_internal_MDR1-M4-HYB-v1.csv"
    df_pred.to_csv(pred_csv, index=False)
    print("[SAVE] predictions ->", pred_csv)

    # Curves
    roc_png = f"{OUT_DIR}/roc_internal_m4.png"
    roc_csv = f"{OUT_DIR}/roc_internal_m4.csv"
    pr_png  = f"{OUT_DIR}/pr_internal_m4.png"
    pr_csv  = f"{OUT_DIR}/pr_internal_m4.csv"
    roc_auc_val = save_roc(roc_png, roc_csv, y, p, "MDR1-M4-HYB-v1 - INTERNAL ROC")
    pr_auc_val  = save_pr (pr_png , pr_csv , y, p, "MDR1-M4-HYB-v1 - INTERNAL PR")

    # Point metrics + 95% CIs (bootstrap, stratified)
    roc_point, roc_lo, roc_hi = bootstrap_ci(roc_auc, y, p, n_boot=N_BOOT, seed=SEED, stratified=True)
    pr_point , pr_lo , pr_hi  = bootstrap_ci(pr_auc , y, p, n_boot=N_BOOT, seed=SEED, stratified=True)
    # F1 @ fixed threshold
    f1_fixed_point, f1_fixed_lo, f1_fixed_hi = bootstrap_ci(lambda yy,pp: f1_at(yy,pp,FIXED_THR),
                                                            y, p, n_boot=N_BOOT, seed=SEED, stratified=True)
    # Best-F1（各ブートでしきい値再最適化）
    def f1_best_metric(yy,pp):
        _, f1b, _, _ = _compute_best_f1(yy,pp); return f1b
    f1_best_point, f1_best_lo, f1_best_hi = bootstrap_ci(f1_best_metric, y, p, n_boot=N_BOOT, seed=SEED, stratified=True)

    # Confusion tables @0.5 and @Best-F1 threshold（母集団で決めたBestThr）
    best_thr, best_f1_val, best_prec, best_rec = _compute_best_f1(y,p)
    tp,fp,tn,fn = confusion_counts(y,p,FIXED_THR)
    tp_b,fp_b,tn_b,fn_b = confusion_counts(y,p,best_thr)

    # Save metrics table
    rows = [
        ["ROC-AUC", roc_point, roc_lo, roc_hi, ""],
        ["PR-AUC",  pr_point,  pr_lo,  pr_hi,  ""],
        [f"F1@{FIXED_THR:.2f}", f1_fixed_point, f1_fixed_lo, f1_fixed_hi, f"thr={FIXED_THR}"],
        ["F1@Best", f1_best_point, f1_best_lo, f1_best_hi, f"thr≈{best_thr:.4f}, Prec≈{best_prec:.4f}, Rec≈{best_rec:.4f}"],
    ]
    df_stats = pd.DataFrame(rows, columns=["metric","point","ci_2.5","ci_97.5","note"])
    stats_csv = f"{OUT_DIR}/stats_internal_MDR1-M4-HYB-v1.csv"
    df_stats.to_csv(stats_csv, index=False)
    print("[SAVE] stats ->", stats_csv)

    # Save confusion tables
    df_cm = pd.DataFrame([
        ["@fixed", FIXED_THR, tp,fp,tn,fn],
        ["@bestF1", best_thr, tp_b,fp_b,tn_b,fn_b],
    ], columns=["mode","threshold","TP","FP","TN","FN"])
    cm_csv = f"{OUT_DIR}/confusion_internal_MDR1-M4-HYB-v1.csv"
    df_cm.to_csv(cm_csv, index=False)
    print("[SAVE] confusion ->", cm_csv)

    print("\n=== SUMMARY ===")
    print(df_stats.to_string(index=False))
    print("\nConfusion (counts):")
    print(df_cm.to_string(index=False))
    print(f"\nCurves saved to: {roc_png} , {pr_png}")
    print(f"Predictions: {pred_csv}")

if __name__=="__main__":
    main()

[INFO] Internal evaluable N=7321 | positives=1524 | negatives=5797
[INFO] dims: {'x': 5, 'edge': 6, 'g': 7, 'r': 10}
[INFO] Using M4 only | optional CV-PR weight (info): 0.619
[SAVE] predictions -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/predictions_internal_MDR1-M4-HYB-v1.csv
[ROC] MDR1-M4-HYB-v1 - INTERNAL ROC -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/roc_internal_m4.png | /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/roc_internal_m4.csv
[PR ] MDR1-M4-HYB-v1 - INTERNAL PR -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/pr_internal_m4.png | /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/pr_internal_m4.csv
[SAVE] stats -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/stats_internal_MDR1-M4-HYB-v1.csv
[SAVE] confusion -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/reports_internal_MDR1-M4-HYB-v1/confusio

In [ ]:
# === One-cell Figure Generator for MDR1-M4-HYB-v1 (Index-safe, RDKit-robust) ===
# Generates: (A) UMAP, (B) Novelty vs PR-AUC, (C) Reliability (Brier/ECE),
#            (D) Top-K Precision, (E) Attribution (ECFP surrogate),
#            (F) Misclassification substructure heatmap.
# Inputs:
#   ROOT/data_graph_with_smiles.pt
#   ROOT/reports_internal_MDR1-M4-HYB-v1/predictions_internal_MDR1-M4-HYB-v1.csv
# Outputs under ROOT/figs_mdr1_final_m4/

import os, sys, subprocess, json, numpy as np, pandas as pd, warnings
warnings.filterwarnings("ignore")

# RDKitのMorganGenerator系Deprecationを黙らせる
warnings.filterwarnings("ignore", message=".*MorganGenerator.*", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from rdkit import RDLogger
from rdkit import rdBase
RDLogger.DisableLog("rdApp.*")
rdBase.DisableLog("rdApp.warning")

# --------- CONFIG ---------
ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PT = f"{ROOT}/data_graph_with_smiles.pt"
PRED_CSV = f"{ROOT}/reports_internal_MDR1-M4-HYB-v1/predictions_internal_MDR1-M4-HYB-v1.csv"
OUT_DIR = f"{ROOT}/figs_mdr1_final_m4"
os.makedirs(OUT_DIR, exist_ok=True)

# --------- Helper: quiet pip ---------
def _pip_install(cmd_list):
    try:
        subprocess.check_call(cmd_list, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except Exception:
        return False

def _ensure(pkg, pip_name=None):
    try:
        __import__(pkg)
        return True
    except Exception:
        return _pip_install([sys.executable, "-m", "pip", "install", "-q", pip_name or pkg])

# sklearn / umap / matplotlib
_ensure("sklearn", "scikit-learn")
_ensure("umap", "umap-learn")
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression

# torch / pyg
_ensure("torch", "torch")
try:
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage
except Exception:
    _pip_install([sys.executable, "-m", "pip", "install", "-q", "torch-geometric"])
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage
import torch

# RDKit（任意）
HAVE_RDKIT = True
try:
    import rdkit  # quick probe
except Exception:
    if not _pip_install([sys.executable, "-m", "pip", "install", "-q", "rdkit==2023.9.5"]):
        if not _pip_install([sys.executable, "-m", "pip", "install", "-q", "rdkit"]):
            HAVE_RDKIT = False

if HAVE_RDKIT:
    from rdkit import Chem
    from rdkit.Chem import AllChem, Draw
    from rdkit import DataStructs

# --------- Load internal data ---------
def load_pyg_list(path):
    add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

assert os.path.exists(DATA_PT), f"Missing: {DATA_PT}"
assert os.path.exists(PRED_CSV), f"Missing: {PRED_CSV}"

dl = load_pyg_list(DATA_PT)
pred = pd.read_csv(PRED_CSV)

# ---- 安全な結合：各行に gr_vec を紐づけてから merge する ----
records = []
for d in dl:
    smiles = getattr(d, "smiles", "")
    yv = int(getattr(d, "y", -1))
    yv = yv if yv in (0,1) else -1
    g = np.asarray(getattr(d, "g")).reshape(-1)
    r = np.asarray(getattr(d, "r")).reshape(-1)
    gr_vec = np.concatenate([g, r], axis=0)  # 17-dim (g7 + r10)
    records.append({"SMILES": smiles, "y_true": yv, "gr_vec": gr_vec})
base = pd.DataFrame(records)

# 期待する列（M4のみ）
needed_cols = {"SMILES","p_M4","p_ens"}
missing = sorted(list(needed_cols - set(pred.columns)))
assert not missing, f"Prediction CSV missing columns: {missing}"

# SMILESで結合（同一SMILESが複数ある場合は重複行ができるが gr_vec も複製される）
df = base.merge(pred[["SMILES","p_M4","p_ens"]], on="SMILES", how="inner")
df = df[df["y_true"].isin([0,1])].reset_index(drop=True)

# UMAP用の行列は df["gr_vec"] から生成（常に行と一致）
gr_mat = np.vstack(df["gr_vec"].to_numpy())

# --------- (A) UMAP on [g;r] ---------
try:
    import umap
    reducer = umap.UMAP(n_neighbors=30, min_dist=0.1, metric="euclidean", random_state=42)
    X2 = reducer.fit_transform(gr_mat)
    plt.figure()
    pos = df["y_true"].values==1
    neg = df["y_true"].values==0
    plt.scatter(X2[neg,0], X2[neg,1], s=8, alpha=0.7, label="Negative")
    plt.scatter(X2[pos,0], X2[pos,1], s=8, alpha=0.7, marker="x", label="Positive")
    plt.xlabel("UMAP-1"); plt.ylabel("UMAP-2"); plt.title("A) UMAP of molecular descriptors (g+r)")
    plt.legend(); plt.tight_layout()
    A_PATH = os.path.join(OUT_DIR, "A_umap_gr.png")
    plt.savefig(A_PATH, dpi=160); plt.close()
except Exception as e:
    A_PATH = None
    print("[WARN] UMAP failed:", e)

# --------- (B) Novelty proxy vs PR-AUC（RDKitがある時のみ） ---------
B_PATH = None
if HAVE_RDKIT:
    def morgan_fp(smiles, radius=2, nBits=2048):
        m = Chem.MolFromSmiles(smiles)
        if m is None: return None
        return AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=nBits)

    fps = [morgan_fp(s) for s in df["SMILES"].tolist()]
    valid_idx = [i for i,f in enumerate(fps) if f is not None]

    max_sim = np.zeros(len(df), dtype=float)
    for i in valid_idx:
        fi = fps[i]
        best = 0.0
        for j in valid_idx:
            if i==j: continue
            best = max(best, DataStructs.TanimotoSimilarity(fi, fps[j]))
        max_sim[i] = best
    df["max_sim_rest"] = max_sim
    bins = [0.0, 0.4, 0.6, 0.7, 0.8, 0.9, 0.95, 1.01]
    df["sim_bin"] = pd.cut(df["max_sim_rest"], bins=bins, right=False)

    rows_b = []
    for b in df["sim_bin"].dropna().unique().sort_values():
        sub = df[df["sim_bin"]==b]
        if sub["y_true"].nunique()<2:
            pr = np.nan
        else:
            prec, rec, _ = precision_recall_curve(sub["y_true"].values, sub["p_ens"].values)
            pr = auc(rec, prec)
        rows_b.append({"bin": str(b), "n": len(sub), "pr_auc": pr})
    df_b = pd.DataFrame(rows_b)

    plt.figure()
    plt.plot(np.arange(len(df_b)), df_b["pr_auc"].values, marker="o")
    plt.xticks(np.arange(len(df_b)), df_b["bin"].tolist(), rotation=45, ha="right")
    plt.ylabel("PR-AUC"); plt.xlabel("Max similarity to others (bin)")
    plt.title("B) Novelty proxy vs PR-AUC")
    plt.tight_layout()
    B_PATH = os.path.join(OUT_DIR, "B_novelty_vs_prauc.png")
    plt.savefig(B_PATH, dpi=160); plt.close()
else:
    print("[SKIP] RDKit unavailable -> skip panel (B).")

# --------- (C) Reliability curve (Brier/ECE) ---------
frac_pos, mean_pred = calibration_curve(df["y_true"].values, df["p_ens"].values, n_bins=10, strategy="quantile")
brier = np.mean((df["p_ens"].values - df["y_true"].values)**2)
ece = np.mean(np.abs(frac_pos - mean_pred))
plt.figure()
plt.plot([0,1],[0,1], linestyle="--")
plt.plot(mean_pred, frac_pos, marker="o")
plt.xlabel("Predicted probability"); plt.ylabel("Observed frequency")
plt.title(f"C) Reliability (Brier={brier:.3f}, ECE={ece:.3f})")
plt.tight_layout()
C_PATH = os.path.join(OUT_DIR, "C_reliability.png")
plt.savefig(C_PATH, dpi=160); plt.close()

# --------- (D) Top-K precision ---------
def precision_at_k(y, p, k):
    k = max(1, min(k, len(p)))
    idx = np.argsort(-p)[:k]
    return float(np.mean(y[idx]))
Ks = [10,25,50,100,200,500,1000]
p_at = [precision_at_k(df["y_true"].values, df["p_ens"].values, k) for k in Ks]
plt.figure()
plt.plot(Ks, p_at, marker="o")
plt.xlabel("K"); plt.ylabel("Precision@K")
plt.title("D) Top-K Precision curve")
plt.tight_layout()
D_PATH = os.path.join(OUT_DIR, "D_topk_precision.png")
plt.savefig(D_PATH, dpi=160); plt.close()

# --------- (E) Attribution via ECFP surrogate（RDKitがある時のみ） ---------
E_PATHS = []
if HAVE_RDKIT:
    bit_infos, bit_mats = [], []
    ys = df["y_true"].values
    for s in df["SMILES"].tolist():
        m = Chem.MolFromSmiles(s)
        if m is None:
            bit_infos.append({})
            bit_mats.append(np.zeros((2048,), dtype=np.uint8))
            continue
        info = {}
        bv = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048, bitInfo=info)
        arr = np.zeros((2048,), dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(bv, arr)
        bit_infos.append(info); bit_mats.append(arr)
    X = np.vstack(bit_mats)
    try:
        clf = LogisticRegression(max_iter=200, n_jobs=1)
    except TypeError:
        clf = LogisticRegression(max_iter=200)
    clf.fit(X, ys)
    coef = clf.coef_[0]

    p = df["p_ens"].values
    pred = (p>=0.5).astype(int)
    TP_idx = np.where((pred==1)&(ys==1))[0][:2]
    FP_idx = np.where((pred==1)&(ys==0))[0][:2]
    FN_idx = np.where((pred==0)&(ys==1))[0][:2]

    def atom_importance_from_bits(mol, bit_info, coefs):
        n = mol.GetNumAtoms()
        scores = np.zeros(n, dtype=float)
        for b, envs in bit_info.items():
            w = coefs[b]
            for (atom_idx, radius) in envs:
                amap = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom_idx)
                atoms = {atom_idx}
                for bond_idx in amap:
                    bnd = mol.GetBondWithIdx(bond_idx)
                    atoms.add(bnd.GetBeginAtomIdx()); atoms.add(bnd.GetEndAtomIdx())
                for a in atoms: scores[a] += w
        if (scores.max() - scores.min())>1e-9:
            scores = (scores - scores.min())/(scores.max()-scores.min())
        return scores

    def draw_with_atom_scores(smiles, scores, path, title):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return False
        atom_cols = {i:(float(scores[i]), 0.0, 0.0) for i in range(mol.GetNumAtoms())}
        d2d = Draw.MolDraw2DCairo(500, 400)
        Draw.PrepareAndDrawMolecule(d2d, mol, highlightAtoms=list(atom_cols.keys()), highlightAtomColors=atom_cols, legend=title)
        d2d.FinishDrawing()
        with open(path, "wb") as f: f.write(d2d.GetDrawingText())
        return True

    def make_attr_panels(indices, label):
        outs=[]
        for i in indices:
            s = df.loc[i,"SMILES"]
            m = Chem.MolFromSmiles(s)
            if m is None: continue
            scores = atom_importance_from_bits(m, bit_infos[i], coef)
            pth = os.path.join(OUT_DIR, f"E_attr_{label}_{i}.png")
            draw_with_atom_scores(s, scores, pth, title=f"{label} #{i}  p={p[i]:.2f}  y={ys[i]}")
            outs.append(pth)
        return outs

    E_PATHS += make_attr_panels(TP_idx, "TP")
    E_PATHS += make_attr_panels(FP_idx, "FP")
    E_PATHS += make_attr_panels(FN_idx, "FN")
else:
    print("[SKIP] RDKit unavailable -> skip panel (E).")

# --------- (F) Misclassification substructure heatmap（RDKitがある時のみ） ---------
F_PATH = None
if HAVE_RDKIT:
    SMARTS = {
        "AromaticRing": "a1aaaaa1",
        "AliphaticRing": "[R;!a]",
        "Halogen": "[F,Cl,Br,I]",
        "QuaternaryAmmonium": "[N+](C)(C)(C)C",
        "TertiaryAmine": "[NX3;H0;!$(NC=O)]",
        "CarboxylicAcid": "C(=O)[OH]",
        "Amide": "C(=O)N",
        "Ether": "COC",
        "Sulfonamide": "S(=O)(=O)N",
        "Nitrile": "C#N",
    }
    ps = {k: Chem.MolFromSmarts(v) for k,v in SMARTS.items()}
    ys = df["y_true"].values
    p = df["p_ens"].values
    pred = (p>=0.5).astype(int)
    groups = {
        "TP": np.where((pred==1)&(ys==1))[0],
        "FP": np.where((pred==1)&(ys==0))[0],
        "TN": np.where((pred==0)&(ys==0))[0],
        "FN": np.where((pred==0)&(ys==1))[0],
    }
    counts = pd.DataFrame(0, index=list(SMARTS.keys()), columns=list(groups.keys()))
    totals = {g: len(idx) for g, idx in groups.items()}

    for name, patt in ps.items():
        for g, idxs in groups.items():
            c = 0
            for i in idxs:
                m = Chem.MolFromSmiles(df.loc[i,"SMILES"])
                if m is None: continue
                if m.HasSubstructMatch(patt): c += 1
            counts.loc[name, g] = c

    rates = counts.copy().astype(float)
    for g in groups.keys():
        rates[g] = rates[g] / max(1, totals[g])  # 0割回避、0〜1の割合に

    # ★ 低い=青 → 高い=赤、凡例あり
    plt.figure(figsize=(7, 5))
    im = plt.imshow(
        rates.values,
        aspect="auto",
        cmap="bwr",     # blue-white-red（低:青 → 高:赤）
        vmin=0.0,
        vmax=1.0        # 比率なので0〜1に固定
    )
    plt.yticks(np.arange(len(rates.index)), rates.index.tolist())
    plt.xticks(np.arange(len(rates.columns)), rates.columns.tolist())
    plt.title("F) Misclassification substructure heatmap (rate)")
    cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
    cbar.set_label("Rate", rotation=90)
    plt.tight_layout()

    F_PATH = os.path.join(OUT_DIR, "F_misclf_substruct_heatmap.png")
    plt.savefig(F_PATH, dpi=160); plt.close()
else:
    print("[SKIP] RDKit unavailable -> skip panel (F).")

# --------- Save working table & report paths ---------
WORK_TBL = os.path.join(OUT_DIR, "working_table.csv")
# RDKitが無い場合、max_simやsim_binが無いのでNaNで用意
if "max_sim_rest" not in df.columns:
    df["max_sim_rest"] = np.nan
    df["sim_bin"] = np.nan
df[["SMILES","y_true","p_ens","max_sim_rest","sim_bin"]].to_csv(WORK_TBL, index=False)

outputs = {
    "A_umap_gr.png": A_PATH,
    "B_novelty_vs_prauc.png": B_PATH,
    "C_reliability.png": C_PATH,
    "D_topk_precision.png": D_PATH,
    "E_attr_paths": E_PATHS,
    "F_misclf_heatmap.png": F_PATH,
    "working_table.csv": WORK_TBL,
    "note": "Panels B/E/F require RDKit. Index mismatch fixed by carrying gr_vec inside the merged DataFrame. p_ens is M4 probability."
}
print(json.dumps(outputs, indent=2))

{
  "A_umap_gr.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/A_umap_gr.png",
  "B_novelty_vs_prauc.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/B_novelty_vs_prauc.png",
  "C_reliability.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/C_reliability.png",
  "D_topk_precision.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/D_topk_precision.png",
  "E_attr_paths": [
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/E_attr_TP_79.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/E_attr_TP_80.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/E_attr_FP_19.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/E_attr_FP_30.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/E_attr_FN_225.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m4/E_attr_FN_226.png"
  ],
  "F_misclf_heatmap.png"

In [ ]:
# === One-cell Figure Generator for MDR1-M1M4-cvpr (Index-safe, RDKit-robust) ===
# Generates: (A) UMAP, (B) Novelty vs PR-AUC, (C) Reliability (Brier/ECE),
#            (D) Top-K Precision, (E) Attribution (ECFP surrogate),
#            (F) Misclassification substructure heatmap.
# Inputs:
#   ROOT/data_graph_with_smiles.pt
#   ROOT/reports_internal_MDR1-M1M4-cvpr/predictions_internal_MDR1-M1M4-cvpr.csv
# Outputs under ROOT/figs_mdr1_final_m1m4_cvpr/

import os, sys, subprocess, json, numpy as np, pandas as pd, warnings
warnings.filterwarnings("ignore")

# RDKitのMorganGenerator系Deprecationを黙らせる
warnings.filterwarnings("ignore", message=".*MorganGenerator.*", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

from rdkit import RDLogger
from rdkit import rdBase
RDLogger.DisableLog("rdApp.*")
rdBase.DisableLog("rdApp.warning")

# --------- CONFIG ---------
ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PT = f"{ROOT}/data_graph_with_smiles.pt"
PRED_CSV = f"{ROOT}/reports_internal_MDR1-M1M4-cvpr/predictions_internal_MDR1-M1M4-cvpr.csv"
OUT_DIR = f"{ROOT}/figs_mdr1_final_m1m4_cvpr"
os.makedirs(OUT_DIR, exist_ok=True)

# --------- Helper: quiet pip ---------
def _pip_install(cmd_list):
    try:
        subprocess.check_call(cmd_list, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return True
    except Exception:
        return False

def _ensure(pkg, pip_name=None):
    try:
        __import__(pkg)
        return True
    except Exception:
        return _pip_install([sys.executable, "-m", "pip", "install", "-q", pip_name or pkg])

# sklearn / umap / matplotlib
_ensure("sklearn", "scikit-learn")
_ensure("umap", "umap-learn")
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression

# torch / pyg
_ensure("torch", "torch")
try:
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage
except Exception:
    _pip_install([sys.executable, "-m", "pip", "install", "-q", "torch-geometric"])
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage
import torch

# RDKit（任意）
HAVE_RDKIT = True
try:
    import rdkit  # quick probe
except Exception:
    if not _pip_install([sys.executable, "-m", "pip", "install", "-q", "rdkit==2023.9.5"]):
        if not _pip_install([sys.executable, "-m", "pip", "install", "-q", "rdkit"]):
            HAVE_RDKIT = False

if HAVE_RDKIT:
    from rdkit import Chem
    from rdkit.Chem import AllChem, Draw
    from rdkit import DataStructs

# --------- Load internal data ---------
def load_pyg_list(path):
    add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

assert os.path.exists(DATA_PT), f"Missing: {DATA_PT}"
assert os.path.exists(PRED_CSV), f"Missing: {PRED_CSV}"

dl = load_pyg_list(DATA_PT)
pred = pd.read_csv(PRED_CSV)

# ---- 安全な結合：各行に gr_vec を紐づけてから merge する ----
records = []
for d in dl:
    smiles = getattr(d, "smiles", "")
    yv = int(getattr(d, "y", -1))
    yv = yv if yv in (0,1) else -1
    g = np.asarray(getattr(d, "g")).reshape(-1)
    r = np.asarray(getattr(d, "r")).reshape(-1)
    gr_vec = np.concatenate([g, r], axis=0)  # 17-dim (g7 + r10)
    records.append({"SMILES": smiles, "y_true": yv, "gr_vec": gr_vec})
base = pd.DataFrame(records)

# 期待する列（cv_pr の M1+M4）
needed_cols = {"SMILES","p_M1","p_M4","p_ens"}  # 解析は p_ens を使用
missing = sorted(list(needed_cols - set(pred.columns)))
assert not missing, f"Prediction CSV missing columns: {missing}"

# SMILESで結合（同一SMILESが複数ある場合は重複行ができるが gr_vec も複製される）
df = base.merge(pred[["SMILES","p_M1","p_M4","p_ens"]], on="SMILES", how="inner")
df = df[df["y_true"].isin([0,1])].reset_index(drop=True)

# UMAP用の行列は df["gr_vec"] から生成（常に行と一致）
gr_mat = np.vstack(df["gr_vec"].to_numpy())

# --------- (A) UMAP on [g;r] ---------
try:
    import umap
    reducer = umap.UMAP(n_neighbors=30, min_dist=0.1, metric="euclidean", random_state=42)
    X2 = reducer.fit_transform(gr_mat)
    plt.figure()
    pos = df["y_true"].values==1
    neg = df["y_true"].values==0
    plt.scatter(X2[neg,0], X2[neg,1], s=8, alpha=0.7, label="Negative")
    plt.scatter(X2[pos,0], X2[pos,1], s=8, alpha=0.7, marker="x", label="Positive")
    plt.xlabel("UMAP-1"); plt.ylabel("UMAP-2"); plt.title("A) UMAP of molecular descriptors (g+r)")
    plt.legend(); plt.tight_layout()
    A_PATH = os.path.join(OUT_DIR, "A_umap_gr.png")
    plt.savefig(A_PATH, dpi=160); plt.close()
except Exception as e:
    A_PATH = None
    print("[WARN] UMAP failed:", e)

# --------- (B) Novelty proxy vs PR-AUC（RDKitがある時のみ） ---------
B_PATH = None
if HAVE_RDKIT:
    def morgan_fp(smiles, radius=2, nBits=2048):
        m = Chem.MolFromSmiles(smiles)
        if m is None: return None
        return AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=nBits)

    fps = [morgan_fp(s) for s in df["SMILES"].tolist()]
    valid_idx = [i for i,f in enumerate(fps) if f is not None]

    max_sim = np.zeros(len(df), dtype=float)
    for i in valid_idx:
        fi = fps[i]
        best = 0.0
        for j in valid_idx:
            if i==j: continue
            best = max(best, DataStructs.TanimotoSimilarity(fi, fps[j]))
        max_sim[i] = best
    df["max_sim_rest"] = max_sim
    bins = [0.0, 0.4, 0.6, 0.7, 0.8, 0.9, 0.95, 1.01]
    df["sim_bin"] = pd.cut(df["max_sim_rest"], bins=bins, right=False)

    rows_b = []
    for b in df["sim_bin"].dropna().unique().sort_values():
        sub = df[df["sim_bin"]==b]
        if sub["y_true"].nunique()<2:
            pr = np.nan
        else:
            prec, rec, _ = precision_recall_curve(sub["y_true"].values, sub["p_ens"].values)
            pr = auc(rec, prec)
        rows_b.append({"bin": str(b), "n": len(sub), "pr_auc": pr})
    df_b = pd.DataFrame(rows_b)

    plt.figure()
    plt.plot(np.arange(len(df_b)), df_b["pr_auc"].values, marker="o")
    plt.xticks(np.arange(len(df_b)), df_b["bin"].tolist(), rotation=45, ha="right")
    plt.ylabel("PR-AUC"); plt.xlabel("Max similarity to others (bin)")
    plt.title("B) Novelty proxy vs PR-AUC")
    plt.tight_layout()
    B_PATH = os.path.join(OUT_DIR, "B_novelty_vs_prauc.png")
    plt.savefig(B_PATH, dpi=160); plt.close()
else:
    print("[SKIP] RDKit unavailable -> skip panel (B).")

# --------- (C) Reliability curve (Brier/ECE) ---------
frac_pos, mean_pred = calibration_curve(df["y_true"].values, df["p_ens"].values, n_bins=10, strategy="quantile")
brier = np.mean((df["p_ens"].values - df["y_true"].values)**2)
ece = np.mean(np.abs(frac_pos - mean_pred))
plt.figure()
plt.plot([0,1],[0,1], linestyle="--")
plt.plot(mean_pred, frac_pos, marker="o")
plt.xlabel("Predicted probability"); plt.ylabel("Observed frequency")
plt.title(f"C) Reliability (Brier={brier:.3f}, ECE={ece:.3f})")
plt.tight_layout()
C_PATH = os.path.join(OUT_DIR, "C_reliability.png")
plt.savefig(C_PATH, dpi=160); plt.close()

# --------- (D) Top-K precision ---------
def precision_at_k(y, p, k):
    k = max(1, min(k, len(p)))
    idx = np.argsort(-p)[:k]
    return float(np.mean(y[idx]))
Ks = [10,25,50,100,200,500,1000]
p_at = [precision_at_k(df["y_true"].values, df["p_ens"].values, k) for k in Ks]
plt.figure()
plt.plot(Ks, p_at, marker="o")
plt.xlabel("K"); plt.ylabel("Precision@K")
plt.title("D) Top-K Precision curve")
plt.tight_layout()
D_PATH = os.path.join(OUT_DIR, "D_topk_precision.png")
plt.savefig(D_PATH, dpi=160); plt.close()

# --------- (E) Attribution via ECFP surrogate（RDKitがある時のみ） ---------
E_PATHS = []
if HAVE_RDKIT:
    bit_infos, bit_mats = [], []
    ys = df["y_true"].values
    for s in df["SMILES"].tolist():
        m = Chem.MolFromSmiles(s)
        if m is None:
            bit_infos.append({})
            bit_mats.append(np.zeros((2048,), dtype=np.uint8))
            continue
        info = {}
        bv = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048, bitInfo=info)
        arr = np.zeros((2048,), dtype=np.uint8)
        DataStructs.ConvertToNumpyArray(bv, arr)
        bit_infos.append(info); bit_mats.append(arr)
    X = np.vstack(bit_mats)
    try:
        clf = LogisticRegression(max_iter=200, n_jobs=1)
    except TypeError:
        clf = LogisticRegression(max_iter=200)
    clf.fit(X, ys)
    coef = clf.coef_[0]

    p = df["p_ens"].values
    pred = (p>=0.5).astype(int)
    TP_idx = np.where((pred==1)&(ys==1))[0][:2]
    FP_idx = np.where((pred==1)&(ys==0))[0][:2]
    FN_idx = np.where((pred==0)&(ys==1))[0][:2]

    def atom_importance_from_bits(mol, bit_info, coefs):
        n = mol.GetNumAtoms()
        scores = np.zeros(n, dtype=float)
        for b, envs in bit_info.items():
            w = coefs[b]
            for (atom_idx, radius) in envs:
                amap = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom_idx)
                atoms = {atom_idx}
                for bond_idx in amap:
                    bnd = mol.GetBondWithIdx(bond_idx)
                    atoms.add(bnd.GetBeginAtomIdx()); atoms.add(bnd.GetEndAtomIdx())
                for a in atoms: scores[a] += w
        if (scores.max() - scores.min())>1e-9:
            scores = (scores - scores.min())/(scores.max()-scores.min())
        return scores

    def draw_with_atom_scores(smiles, scores, path, title):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None: return False
        atom_cols = {i:(float(scores[i]), 0.0, 0.0) for i in range(mol.GetNumAtoms())}
        d2d = Draw.MolDraw2DCairo(500, 400)
        Draw.PrepareAndDrawMolecule(d2d, mol, highlightAtoms=list(atom_cols.keys()), highlightAtomColors=atom_cols, legend=title)
        d2d.FinishDrawing()
        with open(path, "wb") as f: f.write(d2d.GetDrawingText())
        return True

    def make_attr_panels(indices, label):
        outs=[]
        for i in indices:
            s = df.loc[i,"SMILES"]
            m = Chem.MolFromSmiles(s)
            if m is None: continue
            scores = atom_importance_from_bits(m, bit_infos[i], coef)
            pth = os.path.join(OUT_DIR, f"E_attr_{label}_{i}.png")
            draw_with_atom_scores(s, scores, pth, title=f"{label} #{i}  p={p[i]:.2f}  y={ys[i]}")
            outs.append(pth)
        return outs

    E_PATHS += make_attr_panels(TP_idx, "TP")
    E_PATHS += make_attr_panels(FP_idx, "FP")
    E_PATHS += make_attr_panels(FN_idx, "FN")
else:
    print("[SKIP] RDKit unavailable -> skip panel (E).")

# --------- (F) Misclassification substructure heatmap（RDKitがある時のみ） ---------
F_PATH = None
if HAVE_RDKIT:
    SMARTS = {
        "AromaticRing": "a1aaaaa1",
        "AliphaticRing": "[R;!a]",
        "Halogen": "[F,Cl,Br,I]",
        "QuaternaryAmmonium": "[N+](C)(C)(C)C",
        "TertiaryAmine": "[NX3;H0;!$(NC=O)]",
        "CarboxylicAcid": "C(=O)[OH]",
        "Amide": "C(=O)N",
        "Ether": "COC",
        "Sulfonamide": "S(=O)(=O)N",
        "Nitrile": "C#N",
    }
    ps = {k: Chem.MolFromSmarts(v) for k,v in SMARTS.items()}
    ys = df["y_true"].values
    p = df["p_ens"].values
    pred = (p>=0.5).astype(int)
    groups = {
        "TP": np.where((pred==1)&(ys==1))[0],
        "FP": np.where((pred==1)&(ys==0))[0],
        "TN": np.where((pred==0)&(ys==0))[0],
        "FN": np.where((pred==0)&(ys==1))[0],
    }
    counts = pd.DataFrame(0, index=list(SMARTS.keys()), columns=list(groups.keys()))
    totals = {g: len(idx) for g, idx in groups.items()}

    for name, patt in ps.items():
        for g, idxs in groups.items():
            c = 0
            for i in idxs:
                m = Chem.MolFromSmiles(df.loc[i,"SMILES"])
                if m is None: continue
                if m.HasSubstructMatch(patt): c += 1
            counts.loc[name, g] = c

    rates = counts.copy().astype(float)
    for g in groups.keys():
        rates[g] = rates[g] / max(1, totals[g])  # 0割回避、0〜1の割合に

    # ★ 低い=青 → 高い=赤、凡例あり
    plt.figure(figsize=(7, 5))
    im = plt.imshow(
        rates.values,
        aspect="auto",
        cmap="bwr",     # blue-white-red（低:青 → 高:赤）
        vmin=0.0,
        vmax=1.0        # 比率なので0〜1に固定
    )
    plt.yticks(np.arange(len(rates.index)), rates.index.tolist())
    plt.xticks(np.arange(len(rates.columns)), rates.columns.tolist())
    plt.title("F) Misclassification substructure heatmap (rate)")
    cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
    cbar.set_label("Rate", rotation=90)
    plt.tight_layout()

    F_PATH = os.path.join(OUT_DIR, "F_misclf_substruct_heatmap.png")
    plt.savefig(F_PATH, dpi=160); plt.close()
else:
    print("[SKIP] RDKit unavailable -> skip panel (F).")

# --------- Save working table & report paths ---------
WORK_TBL = os.path.join(OUT_DIR, "working_table.csv")
# RDKitが無い場合、max_simやsim_binが無いのでNaNで用意
if "max_sim_rest" not in df.columns:
    df["max_sim_rest"] = np.nan
    df["sim_bin"] = np.nan
df[["SMILES","y_true","p_ens","max_sim_rest","sim_bin"]].to_csv(WORK_TBL, index=False)

outputs = {
    "A_umap_gr.png": A_PATH,
    "B_novelty_vs_prauc.png": B_PATH,
    "C_reliability.png": C_PATH,
    "D_topk_precision.png": D_PATH,
    "E_attr_paths": E_PATHS,
    "F_misclf_heatmap.png": F_PATH,
    "working_table.csv": WORK_TBL,
    "note": "Panels B/E/F require RDKit. Index mismatch fixed by carrying gr_vec inside the merged DataFrame. p_ens is cv_pr-weighted M1+M4 probability."
}
print(json.dumps(outputs, indent=2))

{
  "A_umap_gr.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/A_umap_gr.png",
  "B_novelty_vs_prauc.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/B_novelty_vs_prauc.png",
  "C_reliability.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/C_reliability.png",
  "D_topk_precision.png": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/D_topk_precision.png",
  "E_attr_paths": [
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/E_attr_TP_79.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/E_attr_TP_80.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/E_attr_FP_9.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/E_attr_FP_19.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_m1m4_cvpr/E_attr_FN_225.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_

In [ ]:
# === Cell 2 (fixed): SHAP + PDP on g(7) & r(10) features (surrogate on p_ens) ===
# Target model: M4 (single family)  ※ p_ens は M4 の最終確率を想定
# Robust to scikit-learn version differences for partial_dependence outputs.

import os, sys, subprocess, json, numpy as np, pandas as pd, warnings
warnings.filterwarnings("ignore")

# -------- CONFIG --------
ROOT    = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PT = f"{ROOT}/data_graph_with_smiles.pt"
# ★ M4 で作成した内部評価の予測CSVを参照（p_ens=M4確率）
PRED_CSV = f"{ROOT}/reports_internal_MDR1-M4-HYB-v1/predictions_internal_MDR1-M4-HYB-v1.csv"
OUT_DIR  = f"{ROOT}/figs_shap_pdp_M4"
os.makedirs(OUT_DIR, exist_ok=True)

# ----- deps -----
def _pip(cmd):
    try:
        subprocess.check_call(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL); return True
    except Exception:
        return False

def _ensure(mod, pip_name=None):
    try:
        __import__(mod); return True
    except Exception:
        return _pip([sys.executable, "-m", "pip", "install", "-q", pip_name or mod])

_ensure("sklearn", "scikit-learn")
_ensure("shap", "shap")
_ensure("torch", "torch")
try:
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage
except Exception:
    _pip([sys.executable, "-m", "pip", "install", "-q", "torch-geometric"])
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage

import torch
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import partial_dependence
import shap
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ---- load ----
def load_pyg_list(path):
    add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

assert os.path.exists(DATA_PT), f"Missing: {DATA_PT}"
assert os.path.exists(PRED_CSV), f"Missing: {PRED_CSV}"
dl = load_pyg_list(DATA_PT)
pred = pd.read_csv(PRED_CSV)

# p_ens（= M4 の最終確率）が必須
if "p_ens" not in pred.columns:
    raise RuntimeError("predictions CSV に 'p_ens' 列がありません。M4 内部評価の predictions を指定してください。")

# carry (g,r) inside rows and merge by SMILES
rows = []
for d in dl:
    sm = getattr(d, "smiles", "")
    y  = int(getattr(d, "y", -1)); y  = y if y in (0,1) else -1
    g = np.asarray(getattr(d, "g")).reshape(-1)  # 7
    r = np.asarray(getattr(d, "r")).reshape(-1)  # 10
    gr = np.concatenate([g,r], axis=0)
    rows.append({"SMILES": sm, "y_true": y, "gr_vec": gr})
base = pd.DataFrame(rows)

df = base.merge(pred[["SMILES","p_ens"]], on="SMILES", how="inner")
df = df[df["y_true"].isin([0,1])].reset_index(drop=True)

X = np.vstack(df["gr_vec"].to_numpy())  # (n,17)
y = df["p_ens"].values.astype(float)
feat_names = [f"g{i+1}" for i in range(7)] + [f"r{i+1}" for i in range(10)]

# ---- surrogate (GBR) ----
sur = GradientBoostingRegressor(random_state=42)
sur.fit(X, y)

# ---- SHAP summary ----
explainer = shap.TreeExplainer(sur)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X, feature_names=feat_names, show=False, plot_size=(9,6))
SUM_PATH = os.path.join(OUT_DIR, "shap_summary_gr.png")
plt.tight_layout(); plt.savefig(SUM_PATH, dpi=180); plt.close()

# ---- PDP: top-3 features by |SHAP| ----
imp = np.abs(shap_values).mean(axis=0)
top_idx = np.argsort(-imp)[:3]

def _pd_xy(pd_bunch):
    """Handle sklearn version differences: grid_values / values / x_values."""
    if hasattr(pd_bunch, "grid_values"):
        xs = pd_bunch.grid_values[0]
    elif isinstance(pd_bunch, dict) and "grid_values" in pd_bunch:
        xs = pd_bunch["grid_values"][0]
    elif isinstance(pd_bunch, dict) and "values" in pd_bunch:
        xs = pd_bunch["values"][0]
    elif hasattr(pd_bunch, "values"):
        xs = pd_bunch.values[0]
    elif hasattr(pd_bunch, "x_values"):
        xs = pd_bunch.x_values[0]
    elif isinstance(pd_bunch, dict) and "x_values" in pd_bunch:
        xs = pd_bunch["x_values"][0]
    else:
        raise KeyError("No x grid found in partial_dependence result.")
    # average
    if hasattr(pd_bunch, "average"):
        ys = pd_bunch.average[0]
    elif isinstance(pd_bunch, dict) and "average" in pd_bunch:
        ys = pd_bunch["average"][0]
    else:
        raise KeyError("No 'average' in partial_dependence result.")
    return np.asarray(xs), np.asarray(ys)

pdp_paths = []
for k in top_idx:
    pd_res = partial_dependence(sur, X, [k], kind="average")
    xs, ys_pd = _pd_xy(pd_res)
    plt.figure()
    plt.plot(xs, ys_pd)
    plt.xlabel(feat_names[k]); plt.ylabel("p_ens (surrogate of M4)")
    plt.title(f"PDP: {feat_names[k]}")
    plt.tight_layout()
    pth = os.path.join(OUT_DIR, f"pdp_{feat_names[k]}.png")
    plt.savefig(pth, dpi=180); plt.close()
    pdp_paths.append(pth)

print(json.dumps({
    "out_dir": OUT_DIR,
    "shap_summary": SUM_PATH,
    "pdp_top3": pdp_paths,
}, indent=2))

{
  "out_dir": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M4",
  "shap_summary": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M4/shap_summary_gr.png",
  "pdp_top3": [
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M4/pdp_r5.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M4/pdp_r10.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M4/pdp_r9.png"
  ]
}


In [ ]:
# === Cell 2 (fixed): SHAP + PDP on g(7) & r(10) features (surrogate on p_ens) ===
# Target model: M1+M4 cv_pr ensemble  ※ p_ens は M1+M4（検証PR-AUC重み）アンサンブルの最終確率を想定
# Robust to scikit-learn version differences for partial_dependence outputs.

import os, sys, subprocess, json, numpy as np, pandas as pd, warnings
warnings.filterwarnings("ignore")

# -------- CONFIG --------
ROOT    = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PT = f"{ROOT}/data_graph_with_smiles.pt"
# ★ M1+M4(cv_pr) で作成した内部評価の予測CSVを参照（p_ens= M1+M4(cv_pr) 確率）
PRED_CSV = f"{ROOT}/reports_internal_MDR1-M1M4-cvpr/predictions_internal_MDR1-M1M4-cvpr.csv"
OUT_DIR  = f"{ROOT}/figs_shap_pdp_M1M4_cvpr"
os.makedirs(OUT_DIR, exist_ok=True)

# ----- deps -----
def _pip(cmd):
    try:
        subprocess.check_call(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL); return True
    except Exception:
        return False

def _ensure(mod, pip_name=None):
    try:
        __import__(mod); return True
    except Exception:
        return _pip([sys.executable, "-m", "pip", "install", "-q", pip_name or mod])

_ensure("sklearn", "scikit-learn")
_ensure("shap", "shap")
_ensure("torch", "torch")
try:
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage
except Exception:
    _pip([sys.executable, "-m", "pip", "install", "-q", "torch-geometric"])
    from torch.serialization import add_safe_globals
    from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
    from torch_geometric.data.storage import GlobalStorage

import torch
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import partial_dependence
import shap
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# ---- load ----
def load_pyg_list(path):
    add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

assert os.path.exists(DATA_PT), f"Missing: {DATA_PT}"
assert os.path.exists(PRED_CSV), f"Missing: {PRED_CSV}"
dl = load_pyg_list(DATA_PT)
pred = pd.read_csv(PRED_CSV)

# p_ens（= M1+M4(cv_pr) の最終確率）が必須
if "p_ens" not in pred.columns:
    raise RuntimeError("predictions CSV に 'p_ens' 列がありません。M1+M4(cv_pr) 内部評価の predictions を指定してください。")

# carry (g,r) inside rows and merge by SMILES
rows = []
for d in dl:
    sm = getattr(d, "smiles", "")
    y  = int(getattr(d, "y", -1)); y  = y if y in (0,1) else -1
    g = np.asarray(getattr(d, "g")).reshape(-1)  # 7
    r = np.asarray(getattr(d, "r")).reshape(-1)  # 10
    gr = np.concatenate([g,r], axis=0)
    rows.append({"SMILES": sm, "y_true": y, "gr_vec": gr})
base = pd.DataFrame(rows)

df = base.merge(pred[["SMILES","p_ens"]], on="SMILES", how="inner")
df = df[df["y_true"].isin([0,1])].reset_index(drop=True)

X = np.vstack(df["gr_vec"].to_numpy())  # (n,17)
y = df["p_ens"].values.astype(float)
feat_names = [f"g{i+1}" for i in range(7)] + [f"r{i+1}" for i in range(10)]

# ---- surrogate (GBR) ----
sur = GradientBoostingRegressor(random_state=42)
sur.fit(X, y)

# ---- SHAP summary ----
explainer = shap.TreeExplainer(sur)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X, feature_names=feat_names, show=False, plot_size=(9,6))
SUM_PATH = os.path.join(OUT_DIR, "shap_summary_gr.png")
plt.tight_layout(); plt.savefig(SUM_PATH, dpi=180); plt.close()

# ---- PDP: top-3 features by |SHAP| ----
imp = np.abs(shap_values).mean(axis=0)
top_idx = np.argsort(-imp)[:3]

def _pd_xy(pd_bunch):
    """Handle sklearn version differences: grid_values / values / x_values."""
    if hasattr(pd_bunch, "grid_values"):
        xs = pd_bunch.grid_values[0]
    elif isinstance(pd_bunch, dict) and "grid_values" in pd_bunch:
        xs = pd_bunch["grid_values"][0]
    elif isinstance(pd_bunch, dict) and "values" in pd_bunch:
        xs = pd_bunch["values"][0]
    elif hasattr(pd_bunch, "values"):
        xs = pd_bunch.values[0]
    elif hasattr(pd_bunch, "x_values"):
        xs = pd_bunch.x_values[0]
    elif isinstance(pd_bunch, dict) and "x_values" in pd_bunch:
        xs = pd_bunch["x_values"][0]
    else:
        raise KeyError("No x grid found in partial_dependence result.")
    # average
    if hasattr(pd_bunch, "average"):
        ys = pd_bunch.average[0]
    elif isinstance(pd_bunch, dict) and "average" in pd_bunch:
        ys = pd_bunch["average"][0]
    else:
        raise KeyError("No 'average' in partial_dependence result.")
    return np.asarray(xs), np.asarray(ys)

pdp_paths = []
for k in top_idx:
    pd_res = partial_dependence(sur, X, [k], kind="average")
    xs, ys_pd = _pd_xy(pd_res)
    plt.figure()
    plt.plot(xs, ys_pd)
    plt.xlabel(feat_names[k]); plt.ylabel("p_ens (surrogate of M1+M4 cv_pr)")
    plt.title(f"PDP: {feat_names[k]}")
    plt.tight_layout()
    pth = os.path.join(OUT_DIR, f"pdp_{feat_names[k]}.png")
    plt.savefig(pth, dpi=180); plt.close()
    pdp_paths.append(pth)

print(json.dumps({
    "out_dir": OUT_DIR,
    "shap_summary": SUM_PATH,
    "pdp_top3": pdp_paths,
}, indent=2))

{
  "out_dir": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M1M4_cvpr",
  "shap_summary": "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M1M4_cvpr/shap_summary_gr.png",
  "pdp_top3": [
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M1M4_cvpr/pdp_r5.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M1M4_cvpr/pdp_r10.png",
    "/content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_shap_pdp_M1M4_cvpr/pdp_r9.png"
  ]
}


In [ ]:
# ==== Digoxin mapping on MDR1-M4-HYB-v1 ====
# - 予測CSV（p_ens: M4単独）と内部グラフを結合して、ジゴキシンの p_ens を特定
# - ECFP(2048, r=2) サロゲート（ロジスティック回帰）で原子重要度を描画
# - データ集合に対する最大Tanimoto類似度（新規性）と近傍Top-5を出力
# 出力物: 図 (PNG) / 近傍CSV / サマリーTXT

import os, numpy as np, pandas as pd
import warnings; warnings.filterwarnings("ignore")

# ----------------- CONFIG -----------------
ROOT = "/content/drive/MyDrive/Chemoinfo_MDR1_ver7"
DATA_PT = f"{ROOT}/data_graph_with_smiles.pt"
# ★ M4内部評価の予測CSV（p_ens=M4確率）に合わせてパスを設定
PRED_CSV = f"{ROOT}/reports_internal_MDR1-M4-HYB-v1/predictions_internal_MDR1-M4-HYB-v1.csv"
OUT_DIR  = f"{ROOT}/figs_mdr1_final_digoxin_M4"
os.makedirs(OUT_DIR, exist_ok=True)

# --- できるだけ堅牢な SMILES 候補（先頭から順に試行）
CANDIDATE_SMILES = [
    # 一例（必要なら置き換え）
    "C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[C@@H](O)C[C@H](O[C@H]4CC[C@@]5(C)[C@H](CC[C@@H]6[C@@H]5CC[C@]5(C)[C@@H](C7=CC(=O)OC7)CC[C@]65O)C4)O[C@@H]3C)O[C@@H]2C)C[C@H](O)[C@@H]1O"
]

# ----------------- Imports -----------------
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

import torch
from torch.serialization import add_safe_globals
from torch_geometric.data.data import DataEdgeAttr, DataTensorAttr
from torch_geometric.data.storage import GlobalStorage

from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit import DataStructs
from rdkit import RDLogger
# RDKit ログ抑止
RDLogger.DisableLog('rdApp.*')
# MorganGeneratorのDeprecation Warning抑止
import warnings as _pywarn
_pywarn.filterwarnings("ignore", message=".*MorganGenerator.*", category=UserWarning)

from sklearn.linear_model import LogisticRegression

# ----------------- Helpers -----------------
def load_pyg_list(path):
    add_safe_globals([DataEdgeAttr, DataTensorAttr, GlobalStorage])
    obj = torch.load(path, map_location="cpu")
    if isinstance(obj, list): return obj
    if isinstance(obj, dict) and "data_list" in obj: return obj["data_list"]
    return list(obj)

def best_parsable_smiles(candidates):
    for s in candidates:
        m = Chem.MolFromSmiles(s)
        if m is not None:
            return Chem.MolToSmiles(m)
    return None

def smiles_equal_or_inchikey_match(sm1, sm2):
    try:
        m1 = Chem.MolFromSmiles(sm1); m2 = Chem.MolFromSmiles(sm2)
        if m1 is None or m2 is None: return (sm1 == sm2)
        return Chem.MolToInchiKey(m1) == Chem.MolToInchiKey(m2)
    except Exception:
        return (sm1 == sm2)

def ecfp2048(sm):
    m = Chem.MolFromSmiles(sm)
    if m is None: return None, None
    fp = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048)
    arr = np.zeros((2048,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    bit_info = {}
    _ = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048, bitInfo=bit_info)
    return m, (arr, bit_info)

def atom_scores_from_bits(mol, bit_info, coef):
    n = mol.GetNumAtoms()
    scores = np.zeros(n, dtype=float)
    for b, envs in bit_info.items():
        w = float(coef[b])
        for (atom_idx, radius) in envs:
            amap = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom_idx)
            atoms = {atom_idx}
            for bond_idx in amap:
                bd = mol.GetBondWithIdx(bond_idx)
                atoms.add(bd.GetBeginAtomIdx()); atoms.add(bd.GetEndAtomIdx())
            for a in atoms: scores[a] += w
    if scores.max() > scores.min():
        scores = (scores - scores.min())/(scores.max() - scores.min())
    return scores

def draw_atom_heat(smiles, scores, out_png, legend):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return False
    atom_cols = {i: (float(scores[i]), 0.0, 0.0) for i in range(mol.GetNumAtoms())}
    d2d = Draw.MolDraw2DCairo(800, 520)
    Draw.PrepareAndDrawMolecule(d2d, mol, highlightAtoms=list(atom_cols.keys()),
                                highlightAtomColors=atom_cols, legend=legend)
    d2d.FinishDrawing()
    with open(out_png, "wb") as f: f.write(d2d.GetDrawingText())
    return True

# ----------------- Load data -----------------
assert os.path.exists(DATA_PT), f"Missing: {DATA_PT}"
assert os.path.exists(PRED_CSV), f"Missing: {PRED_CSV}"

dl = load_pyg_list(DATA_PT)
pred = pd.read_csv(PRED_CSV)

# p_ens（= M4 の最終確率）が必須
if "p_ens" not in pred.columns:
    raise RuntimeError("predictions CSV に 'p_ens' 列がありません。M4 の predictions CSV を指定してください。")

merge_cols = ["SMILES","p_ens"]

# PyG -> 基本表（SMILES, y_true）
rows = []
for d in dl:
    sm = getattr(d, "smiles", "")
    y  = int(getattr(d, "y", -1))
    y  = y if y in (0,1) else -1
    rows.append({"SMILES": sm, "y_true": y})
base = pd.DataFrame(rows)

df = base.merge(pred[merge_cols], on="SMILES", how="inner")
df = df[df["y_true"].isin([0,1])].reset_index(drop=True)

# ----------------- Pick DIGOXIN SMILES -----------------
SMILES_DIG = best_parsable_smiles(CANDIDATE_SMILES)
if SMILES_DIG is None:
    raise ValueError("ジゴキシンの SMILES 候補がすべてパースに失敗しました。CANDIDATE_SMILES を置き換えてください。")

mol_dig = Chem.MolFromSmiles(SMILES_DIG)

# ----------------- Find in internal table -----------------
match_idx = None
for i, sm in enumerate(df["SMILES"].tolist()):
    if smiles_equal_or_inchikey_match(sm, SMILES_DIG):
        match_idx = i; break

if match_idx is not None:
    p_ens_dig = float(df.loc[match_idx, "p_ens"])
    y_dig     = int(df.loc[match_idx, "y_true"])
    print(f"[FOUND] Digoxin present. p_ens={p_ens_dig:.4f}, y={y_dig}")
else:
    p_ens_dig = None
    y_dig = None
    print("[INFO] Digoxin not found in internal table (SMILES/InChIKey). 以降は重要度と類似度のみ出力。")

# ----------------- Build ECFP matrix & train surrogate -----------------
fps = []
for sm in df["SMILES"].tolist():
    m = Chem.MolFromSmiles(sm)
    if m is None:
        fps.append(None); continue
    fps.append(AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=2048))

valid = [(i,f) for i,f in enumerate(fps) if f is not None]
if not valid:
    raise RuntimeError("ECFP の生成に失敗しました。")

X = np.zeros((len(valid), 2048), dtype=np.uint8)
y = df.loc[[i for i,_ in valid], "y_true"].values.astype(int)
for r, (i, fp) in enumerate(valid):
    arr = np.zeros((2048,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    X[r] = arr

try:
    clf = LogisticRegression(max_iter=300, n_jobs=1)
except TypeError:
    clf = LogisticRegression(max_iter=300)
clf.fit(X, y)
coef = clf.coef_[0]

# ----------------- Digoxin: attribution & novelty -----------------
mt, bits = ecfp2048(SMILES_DIG)
if mt is None or bits is None:
    raise RuntimeError("Digoxin の ECFP を計算できません。")

arr_dig, bit_info_dig = bits
scores = atom_scores_from_bits(mt, bit_info_dig, coef)
fig_path = os.path.join(OUT_DIR, "digoxin_atom_importance.png")
_ = draw_atom_heat(
    SMILES_DIG, scores, fig_path,
    legend=f"Digoxin{'' if p_ens_dig is None else f'  p_ens={p_ens_dig:.3f}'}"
)
print("[SAVE] atom-importance figure ->", fig_path)

# 新規性（内部集合に対する最大Tanimoto類似度）
fp_dig = AllChem.GetMorganFingerprintAsBitVect(mt, 2, nBits=2048)
sims = [DataStructs.TanimotoSimilarity(fp, fp_dig) for _, fp in valid]
max_sim = float(np.max(sims)) if sims else float("nan")
print(f"[NOVELTY] Max Tanimoto similarity to internal set: {max_sim:.3f}")

# 近傍Top-5
nn_idx = np.argsort(-np.array(sims))[:5]
nn_rows = []
for k in nn_idx:
    i = valid[k][0]
    nn_rows.append({
        "rank": len(nn_rows)+1,
        "SMILES": df.loc[i, "SMILES"],
        "y_true": int(df.loc[i, "y_true"]),
        "p_ens": float(df.loc[i, "p_ens"]),
        "sim": float(sims[k]),
    })
nn_df = pd.DataFrame(nn_rows)
nn_csv = os.path.join(OUT_DIR, "digoxin_nearest_neighbors.csv")
nn_df.to_csv(nn_csv, index=False)
print("[SAVE] nearest neighbors ->", nn_csv)

# サマリー
summary_txt = os.path.join(OUT_DIR, "digoxin_summary.txt")
with open(summary_txt, "w") as f:
    f.write("Digoxin mapping on MDR1-M4-HYB-v1\n")
    f.write(f"Chosen SMILES: {SMILES_DIG}\n")
    if match_idx is not None:
        f.write(f"Found in internal set: YES | p_ens={p_ens_dig:.4f} | y_true={y_dig}\n")
    else:
        f.write("Found in internal set: NO (only surrogate attribution & novelty reported)\n")
    f.write(f"Max Tanimoto similarity to internal set: {max_sim:.3f}\n")
    f.write(f"Atom-importance figure: {fig_path}\n")
    f.write(f"Nearest neighbors CSV: {nn_csv}\n")

print("[DONE] Summary ->", summary_txt)

[FOUND] Digoxin present. p_ens=0.7565, y=1
[SAVE] atom-importance figure -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_digoxin_M4/digoxin_atom_importance.png
[NOVELTY] Max Tanimoto similarity to internal set: 1.000
[SAVE] nearest neighbors -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_digoxin_M4/digoxin_nearest_neighbors.csv
[DONE] Summary -> /content/drive/MyDrive/Chemoinfo_MDR1_ver7/figs_mdr1_final_digoxin_M4/digoxin_summary.txt
